<p style="font-family: Arial; font-size:2.75em;color:purple; font-style:bold"><br>
SwissStore - Thermal dummy data </p><br>

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Download the dataset </p><br>

This notebook was made as a part of the SwissStore project and tests a set of dummmy data on thermal energy storage technologies.

The objective is to test each technology for a set of energy consuming archetypes to obtain the least-costly configuration. The optimization will be donde following a brute-forece approach.


<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Importing Python libraries </p><br>

In [1]:
import gdxtools as gt
import pandas as pd
import numpy as np
import itertools


#Module that provides a portable way of using operating system dependent functionality
import os

from itertools import permutations, repeat

#Module to interact with the Notebook environment 
#https://ipywidgets.readthedocs.io/en/stable/
import ipywidgets as widgets

#Module for character encoding auto-detection 
#https://chardet.readthedocs.io/en/latest/index.html
import chardet 

#Plotting modules
import matplotlib
from matplotlib import pyplot as plt
#Allows plots to be shown on the Notebook
%matplotlib inline 

#Data visualization module based on matplotlib
#https://seaborn.pydata.org/
import seaborn as sns

#Cross-platform colored terminal text 
#https://pypi.org/project/colorama/
from colorama import Fore
from colorama import Style

from pandas.io.parsers import ParserError

from itertools import product

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Importing data and creating dataframes
</p><br>

In [414]:
#dir_source = './Data/Dummy data_dummy_v8'
#dir_source = './Data/Dummy data_dummy_tco_test'

dir_source = './Data/Dummy data_dummy_100_test'
dir_parsed = './Results'

In [535]:
#Creating a list with the names of the data tables stored in the source directory
data_tables = os.listdir(dir_source)

#Creating an empty dictionary to store the data from the csv files
tables = {}

#Creating a list with the dictionary key`s names
key_names = ['1_demand_profiles', '2_master_table', '3_retrofits', '4_new_technologies', \
             '5_energy_costs','6_power_bands','7_dhw_profile','8_cop_aw_35','9_cop_aw_60',\
             '10_elec_profiles', '11_pv_profiles','12_elec_dem_ind','13_base_energy_costs_scen',\
             '14_tax_scen','15_cop_aw_35_hr','16_cop_aw_60_hr','17_rcp_scen','18_avg_spot_prices',\
             '19_elec_tariff_struct']

count = 0

#Looping through the csv files and uploading the data to the `Tables` dictionary
for fn in data_tables:
    try:
        name = key_names[count]
        path = dir_source + "/" + fn

        try:
            table_df = pd.read_csv(path, sep = "\,", engine='python')
            
#             print(type(table_df))
#             df_comma = pd.read_csv(path, nrows=1,sep=",")
#             df_semi = pd.read_csv(path, nrows=1, sep=";")
            
#             if df_comma.shape[1]>df_semi.shape[1]:
#                 print("comma delimited")
#                 tables[name] = pd.read_csv(path, sep = "\,", engine='python')
#             else:
#                 print("semicolon delimited")
#                 tables[name] = pd.read_csv(path, sep = "\;", engine='python')
            
            
            #tables[name] = pd.read_csv(path, sep = "\,", engine='python') 
            
            table_shape = table_df.shape
          
            if table_shape[1] == 1:
                table_df = pd.read_csv(path, sep = "\;", engine='python')
            else:
                pass
            
            tables[name] = table_df

        except ParserError:
            tables[name] = pd.read_excel(path,engine = 'xlrd')
        except IndexError:
            pass
    except IndexError:
        pass
        
        
    count += 1
    
print(data_tables)    
print(tables)

['01.swisstore_table1_unige.csv', '02.swisstore_table2_unige.csv', '03.swisstore_table3_unige.csv', '04.swisstore_table4_hslu.xlsx', '05.swisstore_table5_hslu.xlsx', '06.swisstore_table6_hslu.csv', '07.swisstore_table7_hslu.csv', '08.swisstore_table8_hslu.csv', '09.swisstore_table9_hslu.csv', '10.swisstore_table10_unige.xlsx', '11.swisstore_table11_unige.csv', '12.swisstore_table12_unige.xlsx', '13.swisstore_table13_hslu.xlsx', '14.swisstore_table14_unibas.xlsx', '15.swisstore_table15_hslu.xlsx', '16.swisstore_table16_hslu.xlsx', '17.swisstore_table17_unige.csv', '18.swissstore_table18_avg_spot_prices.xlsx', '19.swissstore_table_19_etariff.xlsx', '~$02.swisstore_table2_unige.xlsx', '~$04.swisstore_table4_hslu.xlsx', '~$14.swisstore_table14_unibas.xlsx']
{'1_demand_profiles':         arch_id         1         2         3         4         5         6  \
0           481  0.009388  0.007851  0.005753  0.005672  0.008330  0.008666   
1           482  0.009164  0.007574  0.005614  0.005714 

In [536]:
#The function building_code is created to include the retrofit data from table_3 in table_2 
def building_code(df):
    """Returns a DataFrame with a column named 'build_code', aggregating the data from the 
    'constr_period', 'build_type' and 'area_typology' columns. The format is "year of construction-building type-typology"
    
    :param: df: <DataFrame> unedited table 2 or 3
    return: df: <DataFrame> Edited tables with the column 'build_code'"""
    
    df['build_code'] = df['constr_period'].map(str) + '-' + df['build_type'].map(str) + '-' \
                    + df['area_typology'].map(str)
    
    df['arch_typ'] = df['build_type'].map(str) + '-' + df['area_typology'].map(str)
                    
    
    #df = df.drop(columns = ['constr_period','build_type','area_typology'])
    
    #Moving the column 'build_code' to the first position
    cols = list(df.columns)
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]
    
    return df

In [537]:
def rename_cols(tables_key):#Creating table_n as a previous version of table_3 

    table_new = tables.get(tables_key)

    #Renaming table_n's columns to be consistent with table_2's column titles

    table_new.rename(columns={'age':'constr_period','buildingtype':'build_type',\
                        'typology':'area_typology'}, inplace=True)

    return(table_new)

In [538]:
# table_2 = tables.get('2_master_table')
# table_2 = table_2.sample(n=500)

# dir_csv = 'C:/Users/xigeby65/switchdrive/SwissStore/SNF_Storage/Thermal/Data/Dummy data_dummy_500_test/table_2.csv'

# table_2.to_csv(dir_csv, sep = ',', index = False)

In [539]:
#Creating a data frame for each table. For tables 2 and 3 the above function `archetype` 
#is applied
table_1 = tables.get('1_demand_profiles')
table_2 = building_code(tables.get('2_master_table'))
table_3 = building_code(rename_cols(tables_key = '3_retrofits'))
table_4 = tables.get('4_new_technologies')
table_5 = tables.get('5_energy_costs')
table_6 = tables.get('6_power_bands')
table_7 = tables.get('7_dhw_profile')
table_8 = tables.get('8_cop_aw_35')
table_9 = tables.get('9_cop_aw_60')
table_13 = tables.get('13_base_energy_costs_scen')
table_14 = tables.get('14_tax_scen')
table_15 = tables.get('15_cop_aw_35_hr')
table_16 = tables.get('16_cop_aw_60_hr')
table_17 = tables.get('17_rcp_scen')
table_17 = building_code(rename_cols(tables_key ='17_rcp_scen'))

table_8 = table_8.T.reset_index().T.reset_index(drop=True)
table_9 = table_9.T.reset_index().T.reset_index(drop=True)

In [540]:
table_18 = tables.get('18_avg_spot_prices')
table_19 = tables.get('19_elec_tariff_struct')

In [541]:
#print(table_1)
#display(table_2)
# print(table_3)
# print(table_4)
# print(table_5)
# print(table_6)
# print(table_7)
# print(table_8)
# print(table_9)
# print(table_13)
# print(table_14)
display(table_17)
display(table_18)
display(table_19)

,arch_typ,rcp,constr_period,build_type,area_typology,canton,hvac_system_org,scenario_year,useful_energy__kwh_per_m2,era__m2,build_code
0,MFH-Rural,8.5,1920,MFH,Rural,SH,DH,2020,127.422429,4772,1920-MFH-Rural
1,MFH-Rural,2.6,1920,MFH,Rural,BE,DH,2020,142.044250,21700,1920-MFH-Rural
2,MFH-Rural,2.6,1920,MFH,Rural,BL,DH,2020,129.987921,15774,1920-MFH-Rural
3,MFH-Rural,2.6,1920,MFH,Rural,FR,DH,2020,121.300314,3532,1920-MFH-Rural
4,MFH-Rural,2.6,1920,MFH,Rural,GL,DH,2020,190.903501,757,1920-MFH-Rural
...,...,...,...,...,...,...,...,...,...,...,...
157387,SFH-Urban,8.5,2010,SFH,Urban,BE,Electro,2090,49.475081,801,2010-SFH-Urban
157388,SFH-Urban,8.5,2010,SFH,Urban,SZ,Electro,2090,55.048837,2209,2010-SFH-Urban
157389,SFH-Urban,8.5,2010,SFH,Urban,TI,Electro,2090,46.152105,6623,2010-SFH-Urban
157390,SFH-Urban,8.5,2010,SFH,Urban,VS,Electro,2090,50.413620,4443,2010-SFH-Urban


,energy_price,2020,2035,2050
0,avg_spot,6.151168,13.852153,15.774472


,Canton,mean_%,Netznutzung_%,Abgabe_%,KEV_CHF,MWST_%
0,AR,1.007392,1.179748,0.106070,2.3,0.077
1,AI,0.978619,1.166849,0.010701,2.3,0.077
2,AG,0.984507,1.140954,0.090728,2.3,0.077
3,BE,1.153294,1.370025,0.225185,2.3,0.077
4,BL,1.160682,1.231389,0.061985,2.3,0.077
5,BS,1.001806,1.149444,0.959403,2.3,0.077
6,FR,1.143290,1.159414,0.193980,2.3,0.077
7,GL,0.977606,1.527373,0.075018,2.3,0.077
8,GE,1.030318,1.008983,0.133289,2.3,0.077
9,GR,0.974956,1.345607,0.195976,2.3,0.077


<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Climate scenarios and new total heating demand calculation
</p><br>

In [542]:
rcp_scen_list = pd.unique(table_17["rcp"]).tolist()
rcp_year_list = pd.unique(table_17["scenario_year"]).tolist()

rcp_scenario_year_list = [2020,2025,2030,2035,2040,2045,2050]

# print(rcp_scen_list)
# print(rcp_year_list)

In [543]:
rcp_scen_widget = widgets.Dropdown(
    options = rcp_scen_list,
    description = 'rcp scen ',
    disabled = False)
display(rcp_scen_widget)

Dropdown(description='rcp scen ', options=(8.5, 2.6, 4.5), value=8.5)

In [544]:
rcp_scen = rcp_scen_widget.value
rcp_scen

2.6

In [545]:
if rcp_scen == 8.5:
    rcp_file_name = 'rcp85'
if rcp_scen == 4.5:
    rcp_file_name = 'rcp45'
if rcp_scen == 2.6:
    rcp_file_name = 'rcp26'
    
rcp_file_name

'rcp26'

In [546]:
rcp_scenario_year_widget = widgets.Dropdown(
    options = rcp_scenario_year_list,
    description = 'rcp year ',
    disabled = False)
display(rcp_scenario_year_widget)

Dropdown(description='rcp year ', options=(2020, 2025, 2030, 2035, 2040, 2045, 2050), value=2020)

In [547]:
rcp_scenario_year = rcp_scenario_year_widget.value
rcp_scenario_year

2050

In [548]:
pd.options.mode.chained_assignment = None
table_2.loc[:,'rcp'] = rcp_scen
table_2.loc[:,'scenario_year'] = rcp_scenario_year
table_2.head(10)

,arch_typ,arch_id,clim_id,era_median,era_avg,tot_buildings,energy_specific,constr_period,build_type,area_typology,...,sys_enable_electro,sys_enable_gas,sys_enable_oil,n_dhw_cent,roof_era,era_roof_new,arch_entry_id,build_code,rcp,scenario_year
0,SFH-Rural,407,15023,110.0,110.000000,1,191,1920,SFH,Rural,...,0,0,1,0.785714,15.400000,47.600000,1030392,1920-SFH-Rural,2.6,2050
1,MFH-Rural,3962,920,433.0,517.250000,4,78,1990,MFH,Rural,...,0,0,1,3.694643,258.625000,107.333333,1226166,1990-MFH-Rural,2.6,2050
2,SFH-Suburban,2511,6884,184.0,191.760000,16,115,1960,SFH,Suburban,...,0,1,1,1.369714,95.880000,86.984211,1310946,1960-SFH-Suburban,2.6,2050
3,SFH-Suburban,5663,5776,224.5,224.500000,2,57,2010,SFH,Suburban,...,0,1,0,1.603571,62.860000,74.900000,1300476,2010-SFH-Suburban,2.6,2050
4,SFH-Suburban,509,1665,138.0,154.909091,8,121,1920,SFH,Suburban,...,0,1,0,1.106494,21.687273,92.506061,1320270,1920-SFH-Suburban,2.6,2050
5,SFH-Rural,4945,9704,183.0,183.000000,2,77,2000,SFH,Rural,...,0,0,0,1.307143,51.240000,75.250000,1272573,2000-SFH-Rural,2.6,2050
6,SFH-Suburban,4468,10500,146.0,146.000000,1,77,1990,SFH,Suburban,...,0,1,0,1.042857,73.000000,67.200000,1253269,1990-SFH-Suburban,2.6,2050
7,SFH-Rural,3588,7370,126.0,170.666667,3,125,1980,SFH,Rural,...,0,0,0,1.219048,42.666667,44.333333,1198268,1980-SFH-Rural,2.6,2050
8,SFH-Rural,4346,10523,195.0,195.000000,1,105,1990,SFH,Rural,...,0,0,0,1.392857,97.500000,39.666667,1245415,1990-SFH-Rural,2.6,2050
9,SFH-Urban,1952,4026,130.0,130.000000,1,133,1950,SFH,Urban,...,0,0,0,0.928571,26.000000,49.000000,1122003,1950-SFH-Urban,2.6,2050


In [549]:
#The function building_code is created to include the retrofit data from table_3 in table_2 
def rcp_code(df):
    """Returns a DataFrame with a column named 'build_code', aggregating the data from the 
    'constr_period', 'build_type' and 'area_typology' columns. The format is "year of construction-building type-typology"
    
    :param: df: <DataFrame> unedited table 2 or 3
    return: df: <DataFrame> Edited tables with the column 'build_code'"""
    
    df['rcp_code'] = df['build_code'].map(str) + '-' + df['canton'].map(str) + '-' + \
    df['hvac_system_org'].map(str) + '-' + df['rcp'].map(str)
                    
    
    #df = df.drop(columns = ['constr_period','build_type','area_typology'])
    
    #Moving the column 'build_code' to the first position
    cols = list(df.columns)
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]
    
    return df

In [550]:
table_17 = rcp_code(table_17)
table_2 = rcp_code(table_2)

In [551]:
"Calculating the new heating demand based on the chosen climate scenario"

rcp_code_list = pd.unique(table_2['rcp_code']).tolist()

table_2_rcp = table_2.copy()
master_rcp = pd.DataFrame()


for rcp_code in rcp_code_list:

    table_17_rcp = table_17.loc[table_17['rcp_code']== rcp_code]

    for rcp_year in rcp_scenario_year_list:
        if rcp_year not in rcp_year_list:
            table_17_rcp.loc[-1,'scenario_year'] = rcp_year
        else:
            pass
        table_17_rcp = table_17_rcp.sort_values('scenario_year').reset_index(drop=True)

        table_17_rcp = table_17_rcp.interpolate(method ='linear', limit_direction ='forward')
    
#     display('table_17_rcp',table_17_rcp)
#     display('rcp_scenario_year',rcp_scenario_year)
    
    
    energy_specific_base = table_17_rcp.loc[table_17_rcp['scenario_year'] == 2020, 'useful_energy__kwh_per_m2'].item()
    energy_specific_scen = table_17_rcp.loc[table_17_rcp['scenario_year'] == rcp_scenario_year, 'useful_energy__kwh_per_m2'].item()

#     display('base',energy_specific_base)
#     display('scen',energy_specific_scen)
    rcp_factor = energy_specific_scen / energy_specific_base
    
#     display('rcp_factor',rcp_factor)
    
    table_2_rcp_filt = table_2_rcp[table_2_rcp['rcp_code']== rcp_code]
#     display('table_2_rcp_filt_orig',table_2_rcp_filt)
    table_2_rcp_filt['energy_specific'] = table_2_rcp_filt['energy_specific'] * rcp_factor
#     display('table_2_rcp_filt_new',table_2_rcp_filt)
    
    master_rcp = master_rcp.append(table_2_rcp_filt)
    
table_2 = master_rcp

display(table_2)

,rcp_code,arch_typ,arch_id,clim_id,era_median,era_avg,tot_buildings,energy_specific,constr_period,build_type,...,sys_enable_electro,sys_enable_gas,sys_enable_oil,n_dhw_cent,roof_era,era_roof_new,arch_entry_id,build_code,rcp,scenario_year
0,1920-SFH-Rural-NE-Oil-2.6,SFH-Rural,407,15023,110.0,110.000000,1,183.709130,1920,SFH,...,0,0,1,0.785714,15.400000,47.600000,1030392,1920-SFH-Rural,2.6,2050
1,1990-MFH-Rural-LU-Oil-2.6,MFH-Rural,3962,920,433.0,517.250000,4,75.429296,1990,MFH,...,0,0,1,3.694643,258.625000,107.333333,1226166,1990-MFH-Rural,2.6,2050
2,1960-SFH-Suburban-TI-Oil-2.6,SFH-Suburban,2511,6884,184.0,191.760000,16,109.907851,1960,SFH,...,0,1,1,1.369714,95.880000,86.984211,1310946,1960-SFH-Suburban,2.6,2050
3,2010-SFH-Suburban-FR-Gas-2.6,SFH-Suburban,5663,5776,224.5,224.500000,2,55.122506,2010,SFH,...,0,1,0,1.603571,62.860000,74.900000,1300476,2010-SFH-Suburban,2.6,2050
4,1920-SFH-Suburban-BL-Wood-2.6,SFH-Suburban,509,1665,138.0,154.909091,8,116.503706,1920,SFH,...,0,1,0,1.106494,21.687273,92.506061,1320270,1920-SFH-Suburban,2.6,2050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1990-SFH-Rural-LU-Wood-2.6,SFH-Rural,4281,18016,184.0,200.000000,3,102.015351,1990,SFH,...,0,0,0,1.428571,100.000000,87.500000,1240710,1990-SFH-Rural,2.6,2050
96,2010-MFH-Suburban-SH-HP-2.6,MFH-Suburban,5389,5958,630.0,630.000000,1,40.602305,2010,MFH,...,0,0,0,4.500000,176.400000,122.500000,1290392,2010-MFH-Suburban,2.6,2050
97,1990-SFH-Suburban-TI-Electro-2.6,SFH-Suburban,4461,6742,98.0,98.000000,1,65.631219,1990,SFH,...,0,0,0,0.700000,49.000000,83.300000,1252449,1990-SFH-Suburban,2.6,2050
98,1945-SFH-Rural-BE-Wood-2.6,SFH-Rural,1024,16200,114.0,114.000000,1,162.256412,1945,SFH,...,0,0,0,0.814286,57.000000,199.500000,1069997,1945-SFH-Rural,2.6,2050


In [552]:
# table_17_rcp_set = pd.unique(table_17['rcp_code'])
# table_2_rcp_set = pd.unique(table_2['rcp_code'])

# print('table_17_rcp_set',len(table_17_rcp_set))
# print('table_2_rcp_set',len(table_2_rcp_set))

# set3 = [x for x in table_17_rcp_set if x not in table_2_rcp_set]
# print(set3)

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Lists of system solution technologies
</p><br>

In [553]:
#Creating a list of archetypes by ID
arch_id_list = pd.unique(table_2["arch_entry_id"]).tolist()
#print(arch_id_list)

#Creating a dataframe with system solution's id and technology types 
syst_sol_matrix = tables.get('4_new_technologies')

#Removing unnecesary rows and columns
#syst_sol_matrix = syst_sol_matrix.drop(columns = ['data_type','unit','parameter'])
syst_sol_tech = syst_sol_matrix.iloc[5:13]
syst_sol_decentral = syst_sol_matrix.iloc[13:14]
print(syst_sol_tech)
# print(syst_sol_decentral)

#Creating a list of system solution technologies
syst_sol_list = list(syst_sol_matrix.columns)

syst_sol_list.remove('th_syssolution_id')
print(syst_sol_list)


        th_syssolution_id 100000 100001 100002 100003 100004 100005 100006  \
5        th_systemtype_dh      0      0      0      0      0      0      0   
6      th_systemtype_hpaw      0      0      0      0      0      0      0   
7      th_systemtype_hpbw      0      0      0      0      0      0      0   
8      th_systemtype_wood      0      0      0      0      0      0      0   
9   th_systemtype_electro      0      0      0      0      0      0      0   
10      th_systemtype_gas      0      0      0      0      0      0      0   
11      th_systemtype_oil      0      1      1      1      1      1      1   
12     th_systemtype_stes      1      0      0      0      0      0      0   

   100007 100008  ... 100158 100159 100160 100161 100162 100163 100164 100165  \
5       0      0  ...      0      0      0      0      0      0      0      0   
6       0      0  ...      0      0      0      0      0      0      0      0   
7       0      0  ...      1      1      1      1     

In [554]:
def th_tech_list(syst_sol_df, syst_sol_list, tech_name):

    tech_list = []

    for syst in syst_sol_list:
        if tech_name != 'th_dhw_decentral':
            syst_sol_df = syst_sol_tech.copy()
        else:
            syst_sol_df = syst_sol_decentral.copy()
        
        
        syst_sol_df  = syst_sol_df[['th_syssolution_id', syst]]
            
        syst_sol_df.rename(columns={syst:'tech'}, inplace=True)
        syst_sol_df  = syst_sol_df[(syst_sol_df.th_syssolution_id == tech_name) & (syst_sol_df.tech == 1.0)]
         
        if syst_sol_df.empty:
            pass
            
        else:
            tech_list.append(syst)

    return(tech_list)

In [555]:
#Creating lists for each system solution technology using the 'th_tech_list' function
syst_sol_dict = {}
syst_sol_dict['th_dh_list']= th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_dh') #District heating

syst_sol_dict['th_hpaw_list'] = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_hpaw') #Air water heat pump
syst_sol_dict['th_hpbw_list'] = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_hpbw') #Brine water heat pump
syst_sol_dict['th_wood_list'] = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_wood') #Wood
syst_sol_dict['th_electro_list'] = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_electro') #Electric heaters
syst_sol_dict['th_gas_list'] = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_gas') #Gas
syst_sol_dict['th_oil_list'] = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_oil') #Oil
syst_sol_dict['th_stes_list'] = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_stes') #stes
syst_sol_dict['th_decentral_list'] = th_tech_list(syst_sol_decentral,syst_sol_list,'th_dhw_decentral') #Decentral

th_dh_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_dh') #District heating
th_hpaw_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_hpaw') #Air water heat pump
th_hpbw_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_hpbw') #Brine water heat pump
th_wood_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_wood') #Wood
th_electro_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_electro') #Electric heaters
th_gas_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_gas') #Gas
th_oil_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_oil') #Oil
th_stes_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_stes') #stes
th_decentral_list = th_tech_list(syst_sol_decentral,syst_sol_list,'th_dhw_decentral') #Decentralized
th_central_list = list([x for x in set(syst_sol_list) if x not in set(th_decentral_list)]) #Centralized 

syst_sol_dict['th_central_list'] = th_central_list

# print(th_dh_list)
# print(th_hpaw_list)
# print(th_wood_list)
# print(th_electro_list)
# print(th_gas_list)
# print(th_oil_list)
# print(th_decentral_list)

print(syst_sol_dict)

{'th_dh_list': [], 'th_hpaw_list': [100066, 100067, 100068, 100069, 100070, 100071, 100072, 100073, 100074, 100075, 100076, 100077, 100078, 100079, 100080, 100081, 100082, 100083, 100084, 100085, 100123, 100124, 100125, 100126, 100127, 100128, 100129, 100130, 100131, 100132, 100133, 100134, 100135, 100136, 100137, 100138, 100139, 100140, 100141, 100142], 'th_hpbw_list': [100086, 100087, 100088, 100089, 100090, 100091, 100092, 100093, 100094, 100095, 100096, 100097, 100098, 100099, 100100, 100101, 100102, 100103, 100104, 100105, 100106, 100107, 100108, 100109, 100110, 100143, 100144, 100145, 100146, 100147, 100148, 100149, 100150, 100151, 100152, 100153, 100154, 100155, 100156, 100157, 100158, 100159, 100160, 100161, 100162, 100163, 100164, 100165, 100166, 100167], 'th_wood_list': [100041, 100042, 100043, 100044, 100045, 100046, 100047, 100048, 100049, 100050, 100051, 100052, 100053, 100054, 100055, 100056, 100057, 100058, 100059, 100060, 100061, 100062, 100063, 100064, 100065], 'th_ele

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Retrofit and housing development scenarios
</p><br>

In [556]:
retrofit_scen_widget = widgets.Dropdown(
    options = ['Low','High'],
    description = 'Retrofit scen ',
    disabled = False)
display(retrofit_scen_widget)

Dropdown(description='Retrofit scen ', options=('Low', 'High'), value='Low')

In [557]:
retrofit_scen = retrofit_scen_widget.value

if retrofit_scen == 'High':
    retrofit_factor = 0.02
else:
    retrofit_factor = 0.01
    
retrofit_factor

0.01

In [558]:
#Creating an empty dataframe to copy the desired data from table_2 and table_3
retro_df = pd.DataFrame()

#Creating a list of all archetypes
build_code_list = pd.unique(table_2['build_code']).tolist()

for build_code in build_code_list:
    
    retro = table_2.loc[table_2['build_code'] == build_code]
    retro.insert(1,'investment_cost__chf_per_m2',table_3.loc[table_3['build_code'] == build_code, 'investment_cost__chf_per_m2'].item())
    retro.insert(2,'savings_relative',table_3.loc[table_3['build_code'] == build_code, 'savings_relative'].item())
#     display('x-loop',x)
    retro_df = pd.concat([retro_df,retro])
#     table_x = pd.DataFrame()
retro_df = retro_df.sort_values(by = 'energy_specific', ascending = False).reset_index(drop=True)        
# display('retro_df_final',retro_df)

retrofit_index = round(len(retro_df.index)*retrofit_factor)
print('retrofit_index',retrofit_index)

retro_df_n = retro_df[:retrofit_index]
retro_df_n['energy_specific'] = retro_df_n['energy_specific']*retro_df_n['savings_relative']
retro_df_n['arch_entry_id'] = retro_df_n['arch_entry_id'].apply(str) + '_retro'
retro_df_n

retro_df_x = retro_df[retrofit_index:]
retro_df_x['arch_entry_id'] = retro_df_x['arch_entry_id'].apply(str)

retro_df_fin = pd.concat([retro_df_n,retro_df_x])

table_2 = retro_df_fin

retrofit_index 1


In [559]:
table_2.head(20)

,rcp_code,investment_cost__chf_per_m2,savings_relative,arch_typ,arch_id,clim_id,era_median,era_avg,tot_buildings,energy_specific,...,sys_enable_electro,sys_enable_gas,sys_enable_oil,n_dhw_cent,roof_era,era_roof_new,arch_entry_id,build_code,rcp,scenario_year
0,1960-SFH-Rural-GR-DH-2.6,606,0.702,SFH-Rural,2313,10690,136.0,136.000000,1,139.283936,...,0,0,0,0.971429,68.000000,38.150000,1135719_retro,1960-SFH-Rural,2.6,2050
1,1920-SFH-Rural-NE-Oil-2.6,546,0.671,SFH-Rural,407,15023,110.0,110.000000,1,183.709130,...,0,0,1,0.785714,15.400000,47.600000,1030392,1920-SFH-Rural,2.6,2050
2,1960-SFH-Rural-BE-Oil-2.6,606,0.702,SFH-Rural,2291,5173,75.0,68.333333,3,176.990485,...,0,0,1,0.488095,34.166667,77.000000,1133034,1960-SFH-Rural,2.6,2050
3,1960-SFH-Rural-BE-HP-2.6,606,0.702,SFH-Rural,2290,12043,115.0,115.000000,1,176.974419,...,0,0,0,0.821429,57.500000,82.600000,1132705,1960-SFH-Rural,2.6,2050
4,1950-SFH-Rural-SZ-Oil-2.6,587,0.660,SFH-Rural,1738,7447,235.5,207.000000,4,175.780934,...,0,0,1,1.478571,41.400000,79.800000,1107559,1950-SFH-Rural,2.6,2050
5,1970-SFH-Rural-BE-Wood-2.6,616,0.642,SFH-Rural,2936,10771,103.0,103.000000,1,162.593847,...,0,0,0,0.735714,51.500000,26.833333,1165553,1970-SFH-Rural,2.6,2050
6,1945-SFH-Rural-BE-Wood-2.6,542,0.663,SFH-Rural,1024,16200,114.0,114.000000,1,162.256412,...,0,0,0,0.814286,57.000000,199.500000,1069997,1945-SFH-Rural,2.6,2050
7,1920-SFH-Rural-BE-Oil-2.6,546,0.671,SFH-Rural,361,16782,195.0,195.000000,1,159.840727,...,0,0,1,1.392857,27.300000,71.400000,1021432,1920-SFH-Rural,2.6,2050
8,1960-SFH-Rural-JU-Oil-2.6,606,0.702,SFH-Rural,2323,8665,138.0,138.000000,1,159.492485,...,0,0,1,0.985714,69.000000,28.000000,1136584,1960-SFH-Rural,2.6,2050
9,1980-SFH-Rural-SZ-Electro-2.6,539,0.608,SFH-Rural,3667,7988,114.0,113.000000,3,153.603688,...,0,0,0,0.807143,28.250000,70.962500,1206896,1980-SFH-Rural,2.6,2050


In [560]:
# retro_df_x = retro_df[retrofit_index:]
# retro_df_x

In [561]:
# print(retrofit_index)
# retro_df_n = retro_df[:retrofit_index]
# retro_df_n['energy_specific'] = retro_df_n['energy_specific']*retro_df_n['savings_relative']
# retro_df_n['arch_entry_id'] = retro_df_n['arch_entry_id'].apply(str) + '_retro'
# retro_df_n

In [562]:
# print(retrofit_index)
# retro_df_n = retro_df[:retrofit_index]
# retro_df_n['energy_specific'] = retro_df_n['energy_specific']*retro_df_n['savings_relative']
# retro_df_n['arch_entry_id'] = retro_df_n['arch_entry_id'].apply(str) + '_retro'
# retro_df_n

# retro_df_fin = pd.concat([retro_df_n,retro_df_x])
# retro_df_fin.head(10)

In [563]:
# dir_csv = 'C:/Users/xigeby65/switchdrive/SwissStore/SNF_Storage/Thermal/Data/Dummy data_dummy_500_test/table_2.csv'

# table_2.to_csv(dir_csv, sep = ';', index = False)

**Creating WACC and discounted fossil fuel costs dataframes**

In [564]:
"Creating the WACC dataframe to include the discount of the system's costs"

#Weighted average cost of capital
wacc = float(0.04)
print(wacc)
#Lifetime in years
lifetime = int(20)
start_year = int(2020)
end_year = start_year + lifetime + 1

discount_df = pd.DataFrame()

for year in range(start_year, end_year):
    
    discount_df[year] = [1 / ((1 + wacc))**(year - start_year)]
                             
print(discount_df)

0.04
   2020      2021      2022      2023      2024      2025      2026      2027  \
0   1.0  0.961538  0.924556  0.888996  0.854804  0.821927  0.790315  0.759918   

      2028      2029  ...      2031      2032      2033      2034      2035  \
0  0.73069  0.702587  ...  0.649581  0.624597  0.600574  0.577475  0.555265   

       2036      2037      2038      2039      2040  
0  0.533908  0.513373  0.493628  0.474642  0.456387  

[1 rows x 21 columns]


In [565]:
#'Creating the discounted fuel costs dataframe'

#Discounted fuel costs in CHF per kWh for all technologies
# cost_df = table_5.copy()

# hvac_list = list(cost_df['hvac_system'].tolist())

# #Creating a cost dataframe using table 5 data

# for year in range(start_year, end_year):
#     cost_list = []
#     for hvac in hvac_list:
        
#         if year == 2020:       
#             hvac_cost = cost_df.loc[cost_df['hvac_system'] == hvac, 'cost_chf_per_unit'].item() \
#                       / cost_df.loc[cost_df['hvac_system'] == hvac, 'energy_density_kWh_per_ unit'].item()
            
#             cost_list.append(hvac_cost)
#         else:
            
#             prev_year = year - 1
#             hvac_cost = (cost_df.loc[cost_df['hvac_system'] == hvac, prev_year].item()) \
#                         * (1+ cost_df.loc[cost_df['hvac_system'] == hvac, 'cost_rate'].item())
#             cost_list.append(hvac_cost)

#     cost_df[year] = cost_list


# print(cost_df)
        
            

**Selecting policy scenario**

In [566]:
# policy_scen_list = pd.unique(table_14['scenario']).tolist()
# tax_list = pd.unique(table_14['tax_type']).tolist()
# year_scen_list = [2020,2025,2030,2035,2040]

In [567]:
# policy_scen_widget = widgets.Dropdown(
#     options = policy_scen_list,
#     description = 'Policy scenario ',
#     disabled = False)
# display(policy_scen_widget)

In [568]:
# policy_scen = policy_scen_widget.value

In [569]:
# year_scen_widget = widgets.Dropdown(
#     options = year_scen_list,
#     description = 'Year scenario ',
#     disabled = False)
# display(year_scen_widget)

In [570]:
# year_scen =year_scen_widget.value

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
System solutions' preselection process 
</p><br>

In [571]:
# def syst_sol_preselec(table_2,table_4,table_6,arch_id_list,syst_sol_dict,bool_elec):
    
#     #Return dictionary
#     pre_sel_dict = {}

#     #Creating a copy of table_2
#     pre_sel_df = table_2.copy()
    
#     #Removing archetypes with existing district heating or Brine Water Heat Pump
#     pre_sel_df  = pre_sel_df[pre_sel_df.hvac_system != 'DH']
# #     pre_sel_df  = pre_sel_df[pre_sel_df.hvac_system != 'HPBW']
#     #print(pre_sel_df)
    
#     #Creating a list of archetypes withouth district heating nor electricity based system solutions
#     pre_sel_list = list(pd.unique(pre_sel_df["arch_entry_id"].tolist()))

         
#     for arch in pre_sel_list:
#         arch_df = pre_sel_df.copy()
#         arch_dict = syst_sol_dict.copy()
        
#         #Empty list to store unnecesary entries from the syst_sol_dict
#         #It will be used to delete the filtered entries in arch_dict
#         tech_del_list = []
        
#         #Empty list to store all the compatible technologies after being filtered
#         sol_conc_list = []
        
#         #Creating a one row dataframe, filtered by arch_entry_id
#         arch_df = arch_df[arch_df.arch_entry_id == arch]
        
#         #Creating the boolean variables for technology selection
#         enable_borehole = arch_df.head()['enable_borehole'].values[0] #HPAW or HPBW
#         enable_gas = arch_df.head()['enable_gas'].values[0] #Gas connection enabled
#         enable_district_heat = arch_df.head()['enable_district_heat'].values[0] #District heating connection enabled
        
#         #Variable of the existing technology
#         hvac_system = arch_df.head()['hvac_system'].values[0] 
#         #print(hvac_system)
        
#         #Adding decentral technologies for removal, since it is not a decision parameter at this point
# #         tech_del_list.append('th_decentral_list')
        
#         #Defining non-compatible technologies relative to the existing and 
#         #the technical parameters of its replacement. Discarded technologies are added to tech_del_list  
#         if hvac_system == 'HPBW' or hvac_system == 'HPAW' or hvac_system == 'Wood' \
#         or hvac_system == 'Electro' or hvac_system == 'Gas':
#             tech_del_list.append('th_electro_list')
#             tech_del_list.append('th_oil_list')
            
#         elif hvac_system == 'Oil':
#             tech_del_list.append('th_electro_list')
        
#         if enable_borehole == 0:
#             tech_del_list.append('th_hpbw_list')
        
#         elif enable_gas == 0:
#             tech_del_list.append('th_gas_list')
            
#         elif enable_district_heat == 0:
#             tech_del_list.append('th_dh_list')
        
#         for tech in tech_del_list:
#             arch_dict.pop(tech, None)
# #         print('arch_dict',arch_dict)
# #         print('tech_del_list',tech_del_list)
        
#         for key, value in arch_dict.items():
            
#             sol_conc_list = sol_conc_list + value
        
#         #Boolean variable for decentral and temperature levels of DHW systems
#         dhw_decentral = arch_df.head()['dhw_decentral'].values[0]
#         ht_temp_level = arch_df.head()['ht_temp_level'].values[0]
        
#         #Defining the heating power requirements of the archetype (thermal kW)
# #         if dhw_decentral == 0: #change to p_therm_total
# #             print('decentral is 0')
            
#         p_therm_sel = float(arch_df.head()['p_therm_tot'].values[0])
                
# #         else:
# #             print('decentral is 1')
# #             p_therm_sel = float(arch_df.head()['p_therm_heating'].values[0])
            
            
#         #Creating a list of for power bands
#         pwr_band_list = list(pd.unique(table_6['power_band'].tolist()))
        
              
#         for pwr_band in pwr_band_list:
                
#             lwr_pwr = float(table_6.loc[table_6['power_band'] == pwr_band, 'min_power'])
            
#             hgr_pwr = float(table_6.loc[table_6['power_band'] == pwr_band, 'max_power'])
                                
#             if lwr_pwr <= p_therm_sel <= hgr_pwr:
                    
#                 pwr_band_arch = pwr_band
                
#                 print('pwr_band_arch',pwr_band_arch)
           
#                 break
                
# ##### START FUNCTION       
#         def syst_sol_list(table_4,sol_conc_list,ht_temp_level,p_therm_sel,hgr_pwr):
            
#             #Creating a copy of table_4
#             syst_sol_df = table_4.copy()
        
#             #Empty list to store all compatible system solutions per archetype
#             sol_final_list = []
        
#             for sol_id in sol_conc_list:
            
#                 if ht_temp_level == 0:
                
#                     th_max_power = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_max_power_lt', sol_id].item()
                
#                 else:
#                     th_max_power = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_max_power_ht', sol_id].item()
                
#                 if sol_id == 100017:
#                     print('th_max_power',th_max_power)
#                     print('p_therm_sel',p_therm_sel)
#                     print('hgr_pwr',hgr_pwr)

        
#                 if ((p_therm_sel <= th_max_power) & (th_max_power <= hgr_pwr)):
#                     sol_final_list.append(sol_id)
                
#                 #print('before dhw_decentral',arch,p_therm_sel,sol_final_list)

#                 #Removing system solutions according to centralized or decentralized dhw systems
#                 if dhw_decentral == 0: #If the arch has a centralized system
#                     if sol_id in th_decentral_list and sol_id in sol_final_list:  #but the system solution is decentralized
#                         sol_final_list.remove(sol_id) #remove
#                     else:
#                         pass
                    
#                 if dhw_decentral == 1: #If the arch has a decentralized system
#                     if sol_id in th_central_list and sol_id in sol_final_list: #but syst sol is centralized
#                         sol_final_list.remove(sol_id) #remove
#                     else:
#                         pass 
                    
# #                 if arch == '1110432_retro':
# # #                 if arch == '1071585_retro':
# #                     print('MIDDLE',arch,'sol_final_list',sol_final_list) 
               
#                 if sol_id in sol_final_list:
#                     if bool_elec == 0:
#                         if sol_id in th_dh_list or sol_id in th_hpaw_list or sol_id in th_hpbw_list \
#                            or sol_id in th_electro_list:
#                             sol_final_list.remove(sol_id)
                
#                     if bool_elec == 1:
#                         if sol_id in th_dh_list or sol_id in th_wood_list or sol_id in th_gas_list \
#                           or sol_id in th_oil_list: 
#                             sol_final_list.remove(sol_id)
                            
#                 else:
#                     pass

                
#             return(sol_final_list)
        
# ##### END FUNCTION 
        
#         sol_final_list = syst_sol_list(table_4,sol_conc_list,ht_temp_level,p_therm_sel,hgr_pwr)  
        
#         print('sol_final_list',sol_final_list)
        
#         if len(sol_final_list) == 0:
            
#             pwr_band_tech = pwr_band_arch + 1
            
#             hgr_pwr = float(table_6.loc[table_6['power_band'] == pwr_band_tech, 'max_power'])
            
#             sol_final_list = syst_sol_list(table_4,sol_conc_list,ht_temp_level,p_therm_sel,hgr_pwr)
            
#         else:
#             pass
        
# ##### START FUNCTION 

#         def unique_sol(count,sol_id,sol_final_list):
#             if count >= 1:
#                 sol_final_del_list.append(sol_id)
                
#             elif count == 0:
#                 count += 1
#                 #pass
            
#             return (count,sol_final_list)
        
# ##### END FUNCTION             
            
#         #having one system solution per type of technology
#         count_dh = 0
#         count_hpaw = 0
#         count_hpbw = 0
#         count_electro = 0
#         count_wood = 0
#         count_gas = 0
#         count_oil = 0
#         sol_final_del_list = []
        
        
#         for sol_id in sol_final_list:
    
#             if sol_id in th_dh_list:
#                 count_dh,sol_final_list = unique_sol(count_dh,sol_id,sol_final_list)
                
#             if sol_id in th_hpaw_list:
#                 count_hpaw,sol_final_list = unique_sol(count_hpaw,sol_id,sol_final_list)
                
#             if sol_id in th_hpbw_list:
#                 count_hpbw,sol_final_list = unique_sol(count_hpbw,sol_id,sol_final_list)
                        
#             if sol_id in th_electro_list:
#                 count_electro,sol_final_list = unique_sol(count_electro,sol_id,sol_final_list)
                
#             if sol_id in th_wood_list:
#                 #print('before the wood',arch,sol_final_list,count_wood)
#                 count_wood,sol_final_list = unique_sol(count_wood,sol_id,sol_final_list)
#                 #print('after the wood',arch,sol_final_list,count_wood)
            
#             if sol_id in th_gas_list:
#                 count_gas,sol_final_list = unique_sol(count_gas,sol_id,sol_final_list)
            
#             if sol_id in th_oil_list:
#                 count_oil,sol_final_list = unique_sol(count_oil,sol_id,sol_final_list)
            
            
#         for tech_del in sol_final_del_list:
#             sol_final_list.remove(tech_del)
              
#         #print(sol_final_del_list)
#         pre_sel_dict[arch] = sol_final_list
        
#     return(pre_sel_dict)

In [572]:
def syst_sol_preselec(table_2,table_4,table_6,arch_id_list,syst_sol_dict,bool_elec):
    
    #Return dictionary
    pre_sel_dict = {}

    #Creating a copy of table_2
    pre_sel_df = table_2.copy()
    
    #Removing archetypes with existing district heating or Brine Water Heat Pump
    pre_sel_df  = pre_sel_df[pre_sel_df.hvac_system != 'DH']
#     pre_sel_df  = pre_sel_df[pre_sel_df.hvac_system != 'HPBW']
    #print(pre_sel_df)
    
    #Creating a list of archetypes withouth district heating nor electricity based system solutions
    pre_sel_list = list(pd.unique(pre_sel_df["arch_entry_id"].tolist()))
         
    for arch in pre_sel_list:
        arch_df = pre_sel_df.copy()
        arch_dict = syst_sol_dict.copy()
        
        #Empty list to store unnecesary entries from the syst_sol_dict
        #It will be used to delete the filtered entries in arch_dict
        tech_del_list = []
        
        #Empty list to store all the compatible technologies after being filtered
        sol_conc_list = []
        
        #Creating a one row dataframe, filtered by arch_entry_id
        arch_df = arch_df[arch_df.arch_entry_id == arch]
        
        #Creating the boolean variables for technology selection
        enable_borehole = arch_df.head()['enable_borehole'].values[0] #HPAW or HPBW
        enable_gas = arch_df.head()['enable_gas'].values[0] #Gas connection enabled
        enable_district_heat = arch_df.head()['enable_district_heat'].values[0] #District heating connection enabled
        
        #Variable of the existing technology
        hvac_system = arch_df.head()['hvac_system'].values[0] 
        #print(hvac_system)
        
        #Adding decentral technologies for removal, since it is not a decision parameter at this point
#         tech_del_list.append('th_decentral_list')
        
        #Defining non-compatible technologies relative to the existing and 
        #the technical parameters of its replacement. Discarded technologies are added to tech_del_list  
        if hvac_system == 'HPBW' or hvac_system == 'HPAW' or hvac_system == 'Wood' \
        or hvac_system == 'Electro' or hvac_system == 'Gas':
            tech_del_list.append('th_electro_list')
            tech_del_list.append('th_oil_list')
            
        elif hvac_system == 'Oil':
            tech_del_list.append('th_electro_list')
        
        if enable_borehole == 0:
            tech_del_list.append('th_hpbw_list')
        
        elif enable_gas == 0:
            tech_del_list.append('th_gas_list')
            
        elif enable_district_heat == 0:
            tech_del_list.append('th_dh_list')
            print('th_dh_list',th_dh_list)
        
        for tech in tech_del_list:
            arch_dict.pop(tech, None)
#         print('arch_dict',arch_dict)
#         print('tech_del_list',tech_del_list)
        
        for key, value in arch_dict.items():
            
            sol_conc_list = sol_conc_list + value
            sol_conc_list = (pd.Series(sol_conc_list,dtype = int).unique()).tolist()
        
#         print('sol_conc_list',len(sol_conc_list))
        #Boolean variable for decentral and temperature levels of DHW systems
        dhw_decentral = arch_df.head()['dhw_decentral'].values[0]
        ht_temp_level = arch_df.head()['ht_temp_level'].values[0]
        
        #Defining the heating power requirements of the archetype (thermal kW)            
        p_therm_sel = float(arch_df.head()['p_therm_tot'].values[0])
                
                
##### START FUNCTION       
        def syst_sol_list(table_4,sol_conc_list,ht_temp_level,p_therm_sel):
            
            #Creating a copy of table_4
            syst_sol_df = table_4.copy()
        
            #Empty list to store all compatible system solutions per archetype
            sol_final_list = []
        
            for sol_id in sol_conc_list:
            
                if ht_temp_level == 0:
                
                    th_max_power = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_max_power_lt', sol_id].item()
                
                else:
                    th_max_power = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_max_power_ht', sol_id].item()
                
#                 if sol_id == 100017:
#                     print('th_max_power',th_max_power)
#                     print('p_therm_sel',p_therm_sel)
        
                if (p_therm_sel <= th_max_power):
                    sol_final_list.append(sol_id)
                
                #print('before dhw_decentral',arch,p_therm_sel,sol_final_list)

                #Removing system solutions according to centralized or decentralized dhw systems
                if dhw_decentral == 0: #If the arch has a centralized system
                    if sol_id in th_decentral_list and sol_id in sol_final_list:  #but the system solution is decentralized
                        sol_final_list.remove(sol_id) #remove
                    else:
                        pass
                    
                if dhw_decentral == 1: #If the arch has a decentralized system
                    if sol_id in th_central_list and sol_id in sol_final_list: #but syst sol is centralized
                        sol_final_list.remove(sol_id) #remove
                    else:
                        pass 
                    
#                 if arch == '1110432_retro':
# #                 if arch == '1071585_retro':
#                     print('MIDDLE',arch,'sol_final_list',sol_final_list) 
               
                if sol_id in sol_final_list:
                    if bool_elec == 0:
                        if sol_id in th_dh_list or sol_id in th_hpaw_list or sol_id in th_hpbw_list \
                           or sol_id in th_electro_list:
                            sol_final_list.remove(sol_id)
                
                    if bool_elec == 1:
                        if sol_id in th_dh_list or sol_id in th_wood_list or sol_id in th_gas_list \
                          or sol_id in th_oil_list: 
                            sol_final_list.remove(sol_id)
                            
                else:
                    pass

                
            return(sol_final_list)
        
##### END FUNCTION 
        
        sol_final_list = syst_sol_list(table_4,sol_conc_list,ht_temp_level,p_therm_sel)  
        
#         print('sol_final_list',sol_final_list)
        
        if len(sol_final_list) == 0:
            
            pwr_band_tech = pwr_band_arch + 1
            
            hgr_pwr = float(table_6.loc[table_6['power_band'] == pwr_band_tech, 'max_power'])
            
            sol_final_list = syst_sol_list(table_4,sol_conc_list,ht_temp_level,p_therm_sel,hgr_pwr)
            
        else:
            pass
        
##### START FUNCTION 

        def unique_sol(count,sol_id,sol_final_list):
            if count >= 1:
                sol_final_del_list.append(sol_id)
                
            elif count == 0:
                count += 1
                #pass
            
            return (count,sol_final_list)
        
##### END FUNCTION             
            
        #having one system solution per type of technology
        count_dh = 0
        count_hpaw = 0
        count_hpbw = 0
        count_electro = 0
        count_wood = 0
        count_gas = 0
        count_oil = 0
        sol_final_del_list = []
        
        
        for sol_id in sol_final_list:
    
            if sol_id in th_dh_list:
                count_dh,sol_final_list = unique_sol(count_dh,sol_id,sol_final_list)
                
            if sol_id in th_hpaw_list:
                count_hpaw,sol_final_list = unique_sol(count_hpaw,sol_id,sol_final_list)
                
            if sol_id in th_hpbw_list:
                count_hpbw,sol_final_list = unique_sol(count_hpbw,sol_id,sol_final_list)
                        
            if sol_id in th_electro_list:
                count_electro,sol_final_list = unique_sol(count_electro,sol_id,sol_final_list)
                
            if sol_id in th_wood_list:
                #print('before the wood',arch,sol_final_list,count_wood)
                count_wood,sol_final_list = unique_sol(count_wood,sol_id,sol_final_list)
                #print('after the wood',arch,sol_final_list,count_wood)
            
            if sol_id in th_gas_list:
                count_gas,sol_final_list = unique_sol(count_gas,sol_id,sol_final_list)
            
            if sol_id in th_oil_list:
                count_oil,sol_final_list = unique_sol(count_oil,sol_id,sol_final_list)
            
            
        for tech_del in sol_final_del_list:
            sol_final_list.remove(tech_del)
              
        #print(sol_final_del_list)
        pre_sel_dict[arch] = sol_final_list
        
    return(pre_sel_dict)

In [573]:
syst_fuel_dict = syst_sol_preselec(table_2,table_4,table_6,arch_id_list,syst_sol_dict,0)
syst_fuel_dict

{'1030392': [100056, 100032],
 '1133034': [100056, 100032],
 '1132705': [100056, 100032],
 '1107559': [100056, 100032],
 '1165553': [100056, 100032],
 '1069997': [100056, 100032],
 '1021432': [100056, 100032],
 '1136584': [100056, 100032],
 '1206896': [100042, 100001, 100017],
 '1023478': [100056, 100032],
 '1110917': [100042, 100017, 100001],
 '1107180': [100044, 100018, 100001],
 '1118480': [100056, 100032],
 '1035864': [100056, 100032],
 '1101288': [100043, 100001, 100018],
 '1059152': [100056, 100032],
 '1110364': [100056, 100032],
 '1116978': [100056, 100032],
 '1025186': [100056, 100032],
 '1031611': [100056, 100032],
 '1084861': [100056, 100032],
 '1026107': [100043, 100017, 100001],
 '1192251': [100047, 100019, 100003],
 '1173736': [100056, 100032],
 '1065871': [100043, 100017, 100001],
 '1031558': [100056, 100032],
 '1105291': [100056, 100032],
 '1116960': [100056, 100032],
 '1003479': [100044, 100002, 100018],
 '1033028': [100056, 100032],
 '1017477': [100042, 100001, 100017]

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Wood and fossil fuel demand and cost calculation 
</p><br>

In [574]:
# def tco_fuel_func(syst_fuel_dict,table_2,table_4, discount_df):
#     '''fuel_presel_dict is the result of the preselection process. The dictionary includes all archetypes 
#     compatible with fuel based system solutions, namely oil, wood and gas. The preselection process filters 
#     system solutions based on currently installed heating techbologies and its compatibility with possible 
#     alternatives, considering power operation bands and geographic and technical limitations 
#     (e.g. gass connection availability, borehole suitability, district heating in place, type of DHW system, etc )
    
#     key: arch_entry_id
#     value: list of all suitable technologies (syst_sol_id)'''
    
#     tco_df = pd.DataFrame()
#     master_df = table_2.copy()
#     syst_sol_df = table_4.copy()
#     year_cols = list(discount_df.columns)
#     del year_cols[0]
    
#     'Calculating heating demand'
#     for arch_entry_id, syst_sol_list in syst_fuel_dict.items():
        
#         era_avg = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'era_avg'].item()
#         energy_specific = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'energy_specific'].item()
#         dhw_daily_energy = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'dhw_daily_energy'].item()
#         dhw_decentral = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'dhw_decentral'].item()
#         inv_cost_m2 = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'investment_cost__chf_per_m2'].item()
        
# #         print('arch_entry_id is',arch_entry_id)
# #         print('era_avg',era_avg)
# #         print('energy_specific',energy_specific)
# #         print('dhw_daily_energy',dhw_daily_energy)
# #         print('dhw_decentral',dhw_decentral)
# #         print('inv_cost_m2',inv_cost_m2)
# #         print('savings_relative',savings_relative)     
        
#         for syst_sol in syst_sol_list:
            
#             retro_id = arch_entry_id[-5:]
            
# #             if retro_id == 'retro':
# #                 name_scenario = arch_entry_id[:-6] + '_' + str(syst_sol) + arch_entry_id[-6:]
# #             else:
#             name_scenario = arch_entry_id + '_' + str(syst_sol)
            
#             tco_df_loop = pd.DataFrame(np.array([name_scenario]),columns =['scenario'])
#             #print('tco_df', tco_df)

            
#             '''CAPEX calculation'''
#             th_capex_f = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_capex_f', syst_sol].item()
# #            print('th_capex_f',th_capex_f)     
    
#             tco_df_loop['capex'] = th_capex_f
       
#             '''OPEX calculation'''
            
#             th_opex_f = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_opex_f', syst_sol].item()
# #             print('th_opex_f',th_opex_f)
            
#             th_opex_p = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_opex_p', syst_sol].item()
# #             print('th_opex_p', th_opex_p)
            
#             opex_cost_yr = th_opex_f  + (th_opex_p*th_capex_f)
            
#             opex_df = discount_df * opex_cost_yr
#             #print('opex_df', opex_df)
                  
#             opex_cost_final  = opex_df.iloc[0].sum()
#             #print('opex_cost',opex_cost)
            
#             tco_df_loop['opex'] = opex_cost_final            
            
#             '''Replacement costs'''
#             rep_cost_df = pd.DataFrame(np.array(['th_rc_1','th_rc_2','th_rc_3','th_rc_4','th_rc_5','total']),\
#                                                  columns =['rep'])
#             #print('rep_cost_df',rep_cost_df)
            
            
#             th_rc_1 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_1', syst_sol].item()
#             th_rc_1t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_1t', syst_sol].item()
            
#             th_rc_2 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_2', syst_sol].item()
#             th_rc_2t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_2t', syst_sol].item()
            
#             th_rc_3 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_3', syst_sol].item()
#             th_rc_3t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_3t', syst_sol].item()
            
#             th_rc_4 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_4', syst_sol].item()
#             th_rc_4t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_4t', syst_sol].item()
            
#             th_rc_5 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_5', syst_sol].item()
#             th_rc_5t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_5t', syst_sol].item()
            
#             th_rc_list = [th_rc_1,th_rc_2,th_rc_3,th_rc_4,th_rc_5]
            
            
#             for year in year_cols:
#                 year_diff = year - year_cols[0] + 1
#                 rep_cost_list = []
                
#                 try:
#                     rep_mod_1 = year_diff % th_rc_1t
#                 except ZeroDivisionError:
#                     rep_mod_1 = 0.5
#                 try:
#                     rep_mod_2 = year_diff % th_rc_2t
#                 except ZeroDivisionError:
#                     rep_mod_2 = 0.5
                    
#                 try:    
#                     rep_mod_3 = year_diff % th_rc_3t
#                 except ZeroDivisionError:
#                     rep_mod_3 = 0.5
                    
#                 try:    
#                     rep_mod_4 = year_diff % th_rc_4t
#                 except ZeroDivisionError:
#                     rep_mod_4 = 0.5
                
#                 try:
#                     rep_mod_5 = year_diff % th_rc_5t
#                 except ZeroDivisionError:
#                     rep_mod_5 = 0.5
                    
#                 rep_mod_list = [rep_mod_1,rep_mod_2,rep_mod_3,rep_mod_4,rep_mod_5]
                
#                 for rep_mod, th_rc in zip(rep_mod_list, th_rc_list):
                    
#                     if rep_mod == 0:
                        
#                         rep_cost = th_rc * discount_df.loc[0][year]
                        
#                     else:
#                         rep_cost = 0
                        
#                     rep_cost_list.append(rep_cost)
                    
                   
#                 rep_cost = sum(rep_cost_list)
                
#                 rep_cost_list.append(rep_cost)
                
#                 rep_cost_df[year] = rep_cost_list
        
# #             print(rep_cost_df)
            
#             rep_cost_total = rep_cost_df.sum(axis = 1).get(key = 5)
# #             print('rep_cost_total',rep_cost_total)
# #             print(type(rep_cost_total))

#             tco_df_loop['replacement'] = rep_cost_total
            
#             '''Fuel & energy demand'''
            
#             th_systemtype_oil = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_systemtype_oil', syst_sol].item()
#             th_systemtype_gas = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_systemtype_gas', syst_sol].item()
#             th_systemtype_wood = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_systemtype_wood', syst_sol].item()
            
#             th_dhw_decentral = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_dhw_decentral', syst_sol].item()
#             th_efficiency = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_efficiency', syst_sol].item()
#             th_l_t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_l_t', syst_sol].item()
#             fuel_list = ['Oil','Gas','Wood']
            
# #             print('era_avg',era_avg)
# #             print('energy_specific',energy_specific)
# #             print('dhw_daily_energy',dhw_daily_energy)
# #             print('th_efficiency',th_efficiency)
            
#             if dhw_decentral == 0: #energy_specific*era_avg

#                 fuel_demand_yr = ((energy_specific * era_avg) + (dhw_daily_energy * 365)) / th_efficiency
                
#             else:
#                 fuel_demand_yr = (energy_specific * era_avg) / th_efficiency
                
#             '''Creating the fuel cost dataframe'''
            
# ####START FUNCTION                       
#             def syst_cost_func(fuel, fuel_demand,price_band_arch):
        
# #                 print('fuel',fuel)
            
# #                 print('fuel_demand',fuel_demand)
                
#                 syst_cost_df = pd.DataFrame()
        
#                 price_df = table_5.copy()
#                 price_scen_df = table_13.copy()
#                 tax_scen_df = table_14.copy()
                
#                 price_df = price_df[price_df.hvac_system == fuel]
                
# #                 print('price_df',price_df)
            
#                 price_band_list = list(pd.unique(price_df['price_band'].tolist()))
                
# #                 print('price_band_list',price_band_list)
            
#                 for price_band in price_band_list:
                
#                     lwr_fuel = float(price_df.loc[price_df['price_band'] == price_band, 'min_fuel'])
            
#                     hgr_fuel = float(price_df.loc[price_df['price_band'] == price_band, 'max_fuel'])
                
# #                     print('lwr_fuel',lwr_fuel,'hgr_fuel',hgr_fuel)
                                
#                     if lwr_fuel <= fuel_demand_yr <= hgr_fuel:
                    
#                         price_band_arch = price_band
                    
# #                         print('price_band_arch',price_band_arch)

                
#                 base_price = price_df.loc[price_df['price_band'] == price_band_arch, 'base_price_chf_per_kWh'].item()
#                 base_mos = price_df.loc[price_df['price_band'] == price_band_arch, 'mos_chf_per_kWh'].item()
#                 base_co2 = price_df.loc[price_df['price_band'] == price_band_arch, 'co2_tax_chf_per_kWh'].item()
#                 base_mwst = price_df.loc[price_df['price_band'] == price_band_arch, 'mwst_%'].item()
                
# #                 print('base_price',base_price)
# #                 print('base_mos',base_mos)
# #                 print('base_co2',base_co2)
# #                 print('base_mwst',base_mwst)
                
#                 price_scen_df = price_scen_df[price_scen_df.hvac_system == fuel]
#                 price_scen_df = price_scen_df[price_scen_df.scenario == policy_scen]
                
#                 price_scen_df = price_scen_df.drop(columns=['hvac_system','scenario',2017])
            
#                 base_price_df = (price_scen_df*base_price).astype(float)
#                 base_price_df = base_price_df.interpolate(method = 'linear',axis=1)\
#                                 .reset_index(drop=True).reset_index(drop = True)

                
#                 tax_scen_df = tax_scen_df[tax_scen_df.scenario == policy_scen]
                
#                 mos_tax_df = tax_scen_df[tax_scen_df.tax_type == 'mos_tax']
#                 mos_tax_df = mos_tax_df.drop(columns=['tax_type','scenario',2017])
#                 co2_tax_df = tax_scen_df[tax_scen_df.tax_type == 'co2_tax']
#                 co2_tax_df = co2_tax_df.drop(columns=['tax_type','scenario',2017])
                
#                 mos_tax_df = (mos_tax_df*base_mos).astype(float).reset_index(drop = True)
#                 co2_tax_df = (co2_tax_df*base_co2).astype(float).reset_index(drop = True)
                
# #                 print('base_price_df',base_price_df)
# #                 print('mos_tax_df',mos_tax_df)
# #                 print('co2_tax_df',co2_tax_df)
                
#                 syst_cost_df = pd.DataFrame()
                
# #                 syst_cost_df = pd.concat([base_price_df, mos_tax_df,co2_tax_df]).reset_index()
# #                 syst_cost_df = syst_cost_df.sum(axis = 1)
                
#                 syst_cost_df = (base_price_df + mos_tax_df + co2_tax_df)*(1/(1-base_mwst))
                
# #                 print('syst_cost_df',syst_cost_df)
                 
#                 syst_cost_df = syst_cost_df*discount_df*fuel_demand
        
# #                 display('syst_cost_df',syst_cost_df)  
            
#                 return(syst_cost_df)
# ####END FUNCTION
   
#             if th_systemtype_oil == 1:
                
#                 price_band_arch = 0
                
#                 #Creating a one row dataframe, filtered by th_systemtype_oil
#                 oil_cost_df = syst_cost_func('Oil',fuel_demand_yr,price_band_arch)

#                 #Summing all yearly costs
#                 fuel_cost_total = oil_cost_df.sum(axis = 1).get(key = 0)                
                
#             elif th_systemtype_gas == 1:
                
#                 price_band_arch = 0
                
#                 #Creating a one row dataframe, filtered by th_systemtype_oil
#                 gas_cost_df = syst_cost_func('Gas',fuel_demand_yr,price_band_arch)

#                 #Summing all yearly costs
#                 fuel_cost_total = gas_cost_df.sum(axis = 1).get(key = 0)
            
#             elif th_systemtype_wood == 1:
                
#                 price_band_arch = 0
                
#                 #Creating a one row dataframe, filtered by th_systemtype_oil
#                 wood_cost_df = syst_cost_func('Wood',fuel_demand_yr,price_band_arch)

#                 #Summing all yearly costs
#                 fuel_cost_total = wood_cost_df.sum(axis = 1).get(key = 0)
                
#             else:
#                 pass
            
#             cost_retro = era_avg * inv_cost_m2
#             tco_df_loop['fuel'] = fuel_cost_total
            
#             if retro_id == 'retro':
#                 tco_df_loop['retrofits'] = cost_retro
#             else:
#                 tco_df_loop['retrofits'] = 0
                
            
#             tco_df_loop['total_cost'] = th_capex_f + opex_cost_final + rep_cost_total + fuel_cost_total + cost_retro
            
#             #print(tco_df_loop)
            
#             tco_df = tco_df.append(tco_df_loop,sort = False)
    
#     tco_df = tco_df.reset_index().drop(columns = 'index')
#     return(tco_df)

In [575]:
# tco_fuel_df = tco_fuel_func(syst_fuel_dict,table_2,table_4, discount_df)
# tco_fuel_df

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Electricity demand and cost calculation 
</p><br>

**Preselection process of electricity based system solutions**

In [576]:
syst_elec_dict = syst_sol_preselec(table_2,table_4,table_6,arch_id_list,syst_sol_dict,1)
syst_elec_dict

{'1030392': [100128, 100147],
 '1133034': [100125, 100145],
 '1132705': [100127, 100147],
 '1107559': [100130, 100150],
 '1165553': [100126, 100146],
 '1069997': [100126, 100146],
 '1021432': [100129, 100148],
 '1136584': [100127, 100147],
 '1206896': [100069, 100089],
 '1023478': [100125, 100145],
 '1110917': [100069, 100089],
 '1107180': [100071, 100091],
 '1118480': [100126, 100146],
 '1035864': [100126, 100146],
 '1101288': [100073, 100092],
 '1059152': [100132, 100151],
 '1110364': [100128, 100147],
 '1116978': [100129, 100148],
 '1025186': [100130, 100150],
 '1031611': [100125, 100145],
 '1084861': [100128, 100148],
 '1026107': [100072, 100091],
 '1192251': [100078, 100096],
 '1173736': [100135, 100154],
 '1065871': [100070, 100090],
 '1031558': [100129, 100148],
 '1105291': [100128, 100148],
 '1116960': [100130, 100149],
 '1003479': [100075, 100093],
 '1033028': [100129, 100148],
 '1017477': [100068, 100088],
 '1053713': [100127, 100147],
 '1122003': [100126, 100146],
 '1099367'

In [577]:
syst_fuel_dict

{'1030392': [100056, 100032],
 '1133034': [100056, 100032],
 '1132705': [100056, 100032],
 '1107559': [100056, 100032],
 '1165553': [100056, 100032],
 '1069997': [100056, 100032],
 '1021432': [100056, 100032],
 '1136584': [100056, 100032],
 '1206896': [100042, 100001, 100017],
 '1023478': [100056, 100032],
 '1110917': [100042, 100017, 100001],
 '1107180': [100044, 100018, 100001],
 '1118480': [100056, 100032],
 '1035864': [100056, 100032],
 '1101288': [100043, 100001, 100018],
 '1059152': [100056, 100032],
 '1110364': [100056, 100032],
 '1116978': [100056, 100032],
 '1025186': [100056, 100032],
 '1031611': [100056, 100032],
 '1084861': [100056, 100032],
 '1026107': [100043, 100017, 100001],
 '1192251': [100047, 100019, 100003],
 '1173736': [100056, 100032],
 '1065871': [100043, 100017, 100001],
 '1031558': [100056, 100032],
 '1105291': [100056, 100032],
 '1116960': [100056, 100032],
 '1003479': [100044, 100002, 100018],
 '1033028': [100056, 100032],
 '1017477': [100042, 100001, 100017]

In [578]:
syst_all_list = [syst_elec_dict,syst_fuel_dict]
syst_all_list

[{'1030392': [100128, 100147],
  '1133034': [100125, 100145],
  '1132705': [100127, 100147],
  '1107559': [100130, 100150],
  '1165553': [100126, 100146],
  '1069997': [100126, 100146],
  '1021432': [100129, 100148],
  '1136584': [100127, 100147],
  '1206896': [100069, 100089],
  '1023478': [100125, 100145],
  '1110917': [100069, 100089],
  '1107180': [100071, 100091],
  '1118480': [100126, 100146],
  '1035864': [100126, 100146],
  '1101288': [100073, 100092],
  '1059152': [100132, 100151],
  '1110364': [100128, 100147],
  '1116978': [100129, 100148],
  '1025186': [100130, 100150],
  '1031611': [100125, 100145],
  '1084861': [100128, 100148],
  '1026107': [100072, 100091],
  '1192251': [100078, 100096],
  '1173736': [100135, 100154],
  '1065871': [100070, 100090],
  '1031558': [100129, 100148],
  '1105291': [100128, 100148],
  '1116960': [100130, 100149],
  '1003479': [100075, 100093],
  '1033028': [100129, 100148],
  '1017477': [100068, 100088],
  '1053713': [100127, 100147],
  '11220

In [579]:
syst_all_dict = {}

for arch_entry_id, syst_fuel_list in syst_fuel_dict.items():
    syst_elec_list = syst_elec_dict[arch_entry_id]
    syst_all_list = syst_fuel_list + syst_elec_list
    syst_all_dict[arch_entry_id] = syst_all_list

syst_all_dict

{'1030392': [100056, 100032, 100128, 100147],
 '1133034': [100056, 100032, 100125, 100145],
 '1132705': [100056, 100032, 100127, 100147],
 '1107559': [100056, 100032, 100130, 100150],
 '1165553': [100056, 100032, 100126, 100146],
 '1069997': [100056, 100032, 100126, 100146],
 '1021432': [100056, 100032, 100129, 100148],
 '1136584': [100056, 100032, 100127, 100147],
 '1206896': [100042, 100001, 100017, 100069, 100089],
 '1023478': [100056, 100032, 100125, 100145],
 '1110917': [100042, 100017, 100001, 100069, 100089],
 '1107180': [100044, 100018, 100001, 100071, 100091],
 '1118480': [100056, 100032, 100126, 100146],
 '1035864': [100056, 100032, 100126, 100146],
 '1101288': [100043, 100001, 100018, 100073, 100092],
 '1059152': [100056, 100032, 100132, 100151],
 '1110364': [100056, 100032, 100128, 100147],
 '1116978': [100056, 100032, 100129, 100148],
 '1025186': [100056, 100032, 100130, 100150],
 '1031611': [100056, 100032, 100125, 100145],
 '1084861': [100056, 100032, 100128, 100148],
 '

In [580]:
time_res_widget = widgets.Dropdown(
    options = ['t8760','t48'],
    description = 'time resolution ',
    disabled = False)
display(time_res_widget)

Dropdown(description='time resolution ', options=('t8760', 't48'), value='t8760')

In [581]:
time_res = time_res_widget.value

In [582]:
heat_profiles_df = table_1.copy()

if time_res == 't48':

    column_list = list(range(3,366))
    new_column_list = []
    for i in column_list:
        x = str(i)
        new_column_list.append(x)

    heat_profiles_df = heat_profiles_df.drop(columns = new_column_list)

else:
    pass

print(heat_profiles_df)

        arch_id         1         2         3         4         5         6  \
0           481  0.009388  0.007851  0.005753  0.005672  0.008330  0.008666   
1           482  0.009164  0.007574  0.005614  0.005714  0.008100  0.008251   
2           483  0.009426  0.008040  0.006103  0.005645  0.008365  0.009075   
3           484  0.009277  0.007840  0.005971  0.005649  0.008226  0.008790   
4           485  0.009026  0.007668  0.005656  0.005551  0.007912  0.008051   
...         ...       ...       ...       ...       ...       ...       ...   
304627      476  0.009003  0.007602  0.005884  0.005970  0.007591  0.007938   
304628      477  0.009005  0.007654  0.005793  0.005800  0.007890  0.008302   
304629      478  0.009301  0.007811  0.005858  0.005649  0.008205  0.008790   
304630      479  0.009432  0.007921  0.005790  0.005660  0.008394  0.008813   
304631      480  0.009024  0.007641  0.005730  0.005778  0.007909  0.008324   

               7         8         9  ...       356

**Creating a heating demand dataframe**

In [583]:
'''A DataFrame of net heating demand (kWh_ht) per archetype is created'''

#Creating a list of 365 days
d_list = []
if time_res == 't48':
    d_range_list = list(range(1,3))
else:
    d_range_list = list(range(1,366))
    
    
for i in d_range_list:
    d = 'd' + str(i)
    d_list.append(d)


heat_dem_th = pd.DataFrame() #Creating a empty DataFrame for heating demand profiles 
heat_profiles_df['arch_id'] =  heat_profiles_df['arch_id'].apply(str) #Converting the column arch_entry_id to strings
#display('heat_profiles_df',heat_profiles_df)

for arch_entry_id, syst_sol_list in syst_elec_dict.items():
    
    if arch_entry_id[-5:] == 'retro':
        arch = arch_entry_id[:-6]
    else: 
        arch = arch_entry_id
    
    arch_id = str(table_2.loc[table_2['arch_entry_id'] == arch_entry_id, 'arch_id'].item())
    #print('arch_entry_id',arch_entry_id,'arch_id',arch_id,'arch',arch)
    #A new Heating demand DataFrame is created, filtered by the building ID. 
    heat_df = heat_profiles_df[heat_profiles_df.arch_id == arch_id].set_index('arch_id').transpose()
    #display(heat_df)
    
    '''Creating filtered DataFrames per building ID for demand and costs from Table 2''' 
    
    energy_specific = table_2.loc[table_2['arch_entry_id'] == arch_entry_id, 'energy_specific'].item()
    
#     dhw_decentral = table_2.loc[table_2['arch_entry_id'] == arch_entry_id, 'dhw_decentral'].item()
    
#     dhw_daily_energy = table_2.loc[table_2['arch_entry_id'] == arch_entry_id, 'dhw_daily_energy'].item()
    
    era_avg = table_2.loc[table_2['arch_entry_id'] == arch_entry_id, 'era_avg'].item()
 
    #print('arch_id',arch_id)
    #print('arch_entry_id',arch_entry_id)
    
    for syst_sol in syst_sol_list:
        
        scen = str(arch_entry_id) + '_' + str(syst_sol)
        
        '''Adding the heating demand', including the DHW demand if the system is centralized''' 
    
        heat_dem_th[scen] = heat_df[arch_id] * energy_specific * era_avg
        
for arch_entry_id, syst_sol_list in syst_fuel_dict.items():
    
    if arch_entry_id[-5:] == 'retro':
        arch = arch_entry_id[:-6]
    else: 
        arch = arch_entry_id
        
    for syst_sol in syst_sol_list:
        
        scen = str(arch_entry_id) + '_' + str(syst_sol)
        
        heat_dem_th[scen] = 0
        
# heat_dem_th['d'] = d_list
heat_dem_th = heat_dem_th.reset_index()
del heat_dem_th['index']

display('heat_dem_th',heat_dem_th)

'heat_dem_th'

,1030392_100128,1030392_100147,1133034_100125,1133034_100145,1132705_100127,1132705_100147,1107559_100130,1107559_100150,1165553_100126,1165553_100146,...,1265060_100056,1265060_100032,1294820_100056,1294820_100032,1316897_100056,1316897_100032,1290392_100056,1290392_100032,1292871_100056,1292871_100032
0,180.661833,180.661833,113.947257,113.947257,194.323899,194.323899,314.081086,314.081086,158.193044,158.193044,...,0,0,0,0,0,0,0,0,0,0
1,153.251472,153.251472,91.558155,91.558155,147.947405,147.947405,248.879021,248.879021,141.427799,141.427799,...,0,0,0,0,0,0,0,0,0,0
2,115.062774,115.062774,68.962465,68.962465,112.172033,112.172033,181.295969,181.295969,95.844255,95.844255,...,0,0,0,0,0,0,0,0,0,0
3,120.904879,120.904879,63.946193,63.946193,107.283906,107.283906,185.906009,185.906009,77.655049,77.655049,...,0,0,0,0,0,0,0,0,0,0
4,152.981619,152.981619,95.267177,95.267177,151.054679,151.054679,246.791885,246.791885,130.309459,130.309459,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,110.810855,110.810855,80.640975,80.640975,123.260250,123.260250,189.324384,189.324384,130.727127,130.727127,...,0,0,0,0,0,0,0,0,0,0
361,135.980461,135.980461,98.457044,98.457044,147.186382,147.186382,205.270527,205.270527,140.562705,140.562705,...,0,0,0,0,0,0,0,0,0,0
362,124.912588,124.912588,81.972141,81.972141,125.576828,125.576828,176.925658,176.925658,120.206679,120.206679,...,0,0,0,0,0,0,0,0,0,0
363,132.760167,132.760167,66.521118,66.521118,112.270802,112.270802,189.776908,189.776908,103.713100,103.713100,...,0,0,0,0,0,0,0,0,0,0


In [584]:
def cop_table_func(cop_df):
    
    cop_table = cop_df.copy()
    
    cop_table = cop_table.T.reset_index()
    
    cop_table['index'] = pd.to_numeric(cop_table['index'],errors='coerce')    
    
    cop_columns_list = cop_table.iloc[0].tolist()

    cop_columns_list = [int(i) for i in cop_columns_list]

    #cop_columns_list = [str(i) for i in cop_columns_list]
    


    cop_table.columns =  cop_columns_list
    cop_table = cop_table.drop([0])
    #cop_table = cop_table.apply(pd.to_numeric)
    

    
    #cop_table = cop_table.reindex(cop_table.index.repeat(24))
    
    cop_table = cop_table.reset_index()
    del cop_table['index']
    
    cop_table = cop_table.interpolate(method ='linear', limit_direction ='forward') 
    
    return(cop_table)

cop_aw35_df = cop_table_func(table_8)
cop_aw60_df = cop_table_func(table_9)

# cop_aw35_df = cop_aw35_df.apply(pd.to_numeric)
# cop_aw60_df = cop_aw60_df.apply(pd.to_numeric)

# cop_aw35_df[5154] = cop_aw35_df[5154].astype(float)
# cop_aw60_df[5154] = cop_aw60_df[5154].astype(float)


print('cop_aw35',cop_aw35_df)
print(type(cop_aw35_df))
print('cop_aw60',cop_aw60_df)

cop_aw35       0         1       2       3       4       5       6       7       8      \
0         1    3.2915  3.2734  3.2945  3.2716  3.2992  3.2938  3.2929  3.2965   
1         2     3.226  3.2085  3.2285  3.2128  3.2338  3.2266  3.2263  3.2302   
2         3    3.1607  3.1438  3.1627  3.1542  3.1686  3.1597  3.1599   3.164   
3         4    3.1055  3.0891   3.107  3.1046  3.1135  3.1031  3.1038  3.1081   
4         5    3.0686  3.0526  3.0699  3.0715  3.0768  3.0654  3.0664  3.0708   
...     ...       ...     ...     ...     ...     ...     ...     ...     ...   
8755   8756  3.0557.1  3.0398  3.0569  3.0599  3.0639  3.0522  3.0533  3.0578   
8756   8757  3.1122.1  3.0964  3.1138  3.1151    3.12  3.1088  3.1096   3.114   
8757   8758  3.2672.1  3.2516    3.27  3.2666   3.274  3.2641  3.2644  3.2683   
8758   8759   3.481.1  3.4658  3.4855  3.4755  3.4864  3.4785  3.4779  3.4812   
8759   8760   3.698.2  3.6831  3.7043  3.6875   3.702  3.6961  3.6945  3.6973   

       9      ... 

In [585]:
cop_aw60_df = cop_aw60_df.replace(to_replace=0, method='ffill')
cop_aw35_df = cop_aw35_df.replace(to_replace=0, method='ffill')

In [586]:
# cop_aw60_df.eq(0).sum().sum()

In [587]:
# cop_aw35_df.eq(0).sum().sum()

In [588]:
'''Creating a DataFrame of thermal energy centralized DHW systems'''

dhw_prof_tab = table_7.copy().transpose()
scen_id_list = list(pd.unique(table_2['arch_entry_id'].tolist())) #Creating a list of archetypes

dhw_cen_th = pd.DataFrame()

for arch_entry_id in scen_id_list: #Looping through all archetypes
    
    dhw_decentral = table_2.loc[table_2['arch_entry_id'] == arch_entry_id, 'dhw_decentral'].item()
    
    if dhw_decentral == 0:
        
        dhw_daily_energy = table_2.loc[table_2['arch_entry_id'] == arch_entry_id, 'dhw_daily_energy'].item()    
    
        dhw_cen_th[arch_entry_id] = dhw_prof_tab[0] * dhw_daily_energy
        
    else:

        dhw_cen_th[arch_entry_id] = int(0)
   
                                  
dhw_cen_th = pd.concat([dhw_cen_th]*365,ignore_index=False)


days = list(range(0,365))
days = list(itertools.chain.from_iterable(itertools.repeat(x, 24) for x in days))
hours = list(range(0,8760))

dhw_cen_th['days'] = days
dhw_cen_th['hours'] = hours
dhw_cen_th = dhw_cen_th.set_index(['days', 'hours'])

dhw_cen_th = dhw_cen_th.fillna(0)

                                   
display(dhw_cen_th)

1135719_retro   1030392   1133034   1132705   1107559   1165553  \
days hours                                                                    
0    0           0.009282  0.007508  0.004664  0.007849  0.014128  0.007030   
     1           0.009827  0.007948  0.004938  0.008310  0.014957  0.007442   
     2           0.008911  0.007208  0.004478  0.007535  0.013564  0.006749   
     3           0.008699  0.007036  0.004371  0.007355  0.013240  0.006588   
     4           0.112667  0.091128  0.056610  0.095270  0.171486  0.085329   
...                   ...       ...       ...       ...       ...       ...   
364  8755        0.425271  0.343969  0.213678  0.359604  0.647287  0.322080   
     8756        0.274860  0.222314  0.138104  0.232419  0.418354  0.208166   
     8757        0.185557  0.150083  0.093233  0.156905  0.282429  0.140532   
     8758        0.095560  0.077291  0.048014  0.080805  0.145448  0.072373   
     8759        0.009492  0.007677  0.004769  0.008026  0.014447  0.007189   

             1069997   1021432   1136584  1206896  ...   1282703   1228993  \
days hours                                         ...                       
0    0      0.007781  0.013309  0.009419        0  ...  0.011534  0.037572   
     1      0.008237  0.014090  0.009971        0  ...  0.012211  0.039778   
     2      0.007470  0.012777  0.009042        0  ...  0.011074  0.036072   
     3      0.007291  0.012472  0.008826        0  ...  0.010809  0.035210   
     4      0.094441  0.161545  0.114324        0  ...  0.140005  0.456053   
...              ...       ...       ...      ...  ...       ...       ...   
364  8755   0.356477  0.609763  0.431525        0  ...  0.528461  1.721408   
     8756   0.230398  0.394101  0.278903        0  ...  0.341555  1.112579   
     8757   0.155541  0.266056  0.188286        0  ...  0.230582  0.751097   
     8758   0.080102  0.137017  0.096966        0  ...  0.118748  0.386808   
     8759   0.007957  0.013610  0.009632        0  ...  0.011795  0.038422   

             1294746   1300476  1297375   1265060   1294820   1316897  \
days hours                                                              
0    0      0.015288  0.015322        0  0.122237  0.017438  0.013485   
     1      0.016186  0.016222        0  0.129413  0.018462  0.014276   
     2      0.014678  0.014710        0  0.117355  0.016742  0.012946   
     3      0.014327  0.014359        0  0.114552  0.016342  0.012637   
     4      0.185569  0.185983        0  1.483725  0.211665  0.163678   
...              ...       ...      ...       ...       ...       ...   
364  8755   0.700446  0.702009        0  5.600440  0.798946  0.617815   
     8756   0.452711  0.453722        0  3.619670  0.516374  0.399306   
     8757   0.305624  0.306306        0  2.443625  0.348602  0.269570   
     8758   0.157393  0.157745        0  1.258444  0.179527  0.138826   
     8759   0.015634  0.015669        0  0.125001  0.017832  0.013790   

             1290392    1292871  
days hours                       
0    0      0.064497   0.232778  
     1      0.068283   0.246442  
     2      0.061921   0.223482  
     3      0.060442   0.218143  
     4      0.782870   2.825478  
...              ...        ...  
364  8755   2.955006  10.664993  
     8756   1.909876   6.892985  
     8757   1.289350   4.653427  
     8758   0.664003   2.396472  
     8759   0.065956   0.238042  

[8760 rows x 100 columns]

In [589]:
# display('heat_dem_th',heat_dem_th)

In [590]:
# def tco_elec_func(syst_elec_dict,table_2,table_4,dhw_cen_th,cop_aw35_df,cop_aw60_df,heat_dem_th,discount_df):
#     '''fuel_presel_dict is the result of the preselection process. The dictionary includes all archetypes 
#     compatible with fuel based system solutions, namely oil, wood and gas. The preselection process filters 
#     system solutions based on currently installed heating techbologies and its compatibility with possible 
#     alternatives, considering power operation bands and geographic and technical limitations 
#     (e.g. gass connection availability, borehole suitability, district heating in place, type of DHW system, etc )
    
#     key: arch_entry_id
#     value: list of all suitable technologies (syst_sol_id)'''
    
#     tco_df = pd.DataFrame()
#     heat_dem_el = pd.DataFrame()
#     dhw_cen_el = pd.DataFrame()
#     dhw_dec_th = pd.DataFrame()    
    
#     master_df = table_2.copy()
#     syst_sol_df = table_4.copy()
#     year_cols = list(discount_df.columns)
#     del year_cols[0]
    
#     'Calculating heating demand'
#     for arch_entry_id, syst_sol_list in syst_elec_dict.items():
        
# #         print(type(arch_entry_id),arch_entry_id)
# #         print(syst_sol_list)
# #         display('master_df',master_df)
        
        
#         era_avg = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'era_avg'].item()
#         energy_specific = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'energy_specific'].item()
#         dhw_daily_energy = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'dhw_daily_energy'].item()
#         dhw_decentral = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'dhw_decentral'].item()
#         inv_cost_m2 = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'investment_cost__chf_per_m2'].item()
        
# #         print('arch_entry_id is',arch_entry_id)
# #         print('era_avg',era_avg)
# #         print('energy_specific',energy_specific)
# #         print('dhw_daily_energy',dhw_daily_energy)
# #         print('dhw_decentral',dhw_decentral)
# #         print('inv_cost_m2',inv_cost_m2)
# #         print('savings_relative',savings_relative)     
        
#         for syst_sol in syst_sol_list:
            
#             retro_id = arch_entry_id[-5:]
            
# #             if retro_id == 'retro':
# #                 name_scenario = arch_entry_id[:-6] + '_' + str(syst_sol) + arch_entry_id[-6:]
# #             else:
#             name_scenario = arch_entry_id + '_' + str(syst_sol)
            
#             tco_df_loop = pd.DataFrame(np.array([name_scenario]),columns =['scenario'])
#             #print('tco_df', tco_df)

            
#             '''CAPEX calculation'''
#             th_capex_f = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_capex_f', syst_sol].item()
# #            print('th_capex_f',th_capex_f)     
    
#             tco_df_loop['capex'] = th_capex_f
       
#             '''OPEX calculation'''
            
#             th_opex_f = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_opex_f', syst_sol].item()
# #             print('th_opex_f',th_opex_f)
            
#             th_opex_p = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_opex_p', syst_sol].item()
# #             print('th_opex_p', th_opex_p)
            
#             opex_cost_yr = th_opex_f  + (th_opex_p*th_capex_f)
            
#             opex_df = discount_df * opex_cost_yr
#             #print('opex_df', opex_df)
                  
#             opex_cost_final  = opex_df.iloc[0].sum()
#             #print('opex_cost',opex_cost)
            
#             tco_df_loop['opex'] = opex_cost_final            
            
#             '''Replacement costs'''
#             rep_cost_df = pd.DataFrame(np.array(['th_rc_1','th_rc_2','th_rc_3','th_rc_4','th_rc_5','total']),\
#                                                  columns =['rep'])
#             #print('rep_cost_df',rep_cost_df)
            
            
#             th_rc_1 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_1', syst_sol].item()
#             th_rc_1t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_1t', syst_sol].item()
            
#             th_rc_2 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_2', syst_sol].item()
#             th_rc_2t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_2t', syst_sol].item()
            
#             th_rc_3 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_3', syst_sol].item()
#             th_rc_3t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_3t', syst_sol].item()
            
#             th_rc_4 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_4', syst_sol].item()
#             th_rc_4t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_4t', syst_sol].item()
            
#             th_rc_5 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_5', syst_sol].item()
#             th_rc_5t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_5t', syst_sol].item()
            
#             th_rc_list = [th_rc_1,th_rc_2,th_rc_3,th_rc_4,th_rc_5]
            
            
#             for year in year_cols:
#                 year_diff = year - year_cols[0] + 1
#                 rep_cost_list = []
                
#                 try:
#                     rep_mod_1 = year_diff % th_rc_1t
#                 except ZeroDivisionError:
#                     rep_mod_1 = 0.5
#                 try:
#                     rep_mod_2 = year_diff % th_rc_2t
#                 except ZeroDivisionError:
#                     rep_mod_2 = 0.5
                    
#                 try:    
#                     rep_mod_3 = year_diff % th_rc_3t
#                 except ZeroDivisionError:
#                     rep_mod_3 = 0.5
                    
#                 try:    
#                     rep_mod_4 = year_diff % th_rc_4t
#                 except ZeroDivisionError:
#                     rep_mod_4 = 0.5
                
#                 try:
#                     rep_mod_5 = year_diff % th_rc_5t
#                 except ZeroDivisionError:
#                     rep_mod_5 = 0.5
                
#                 rep_mod_list = [rep_mod_1,rep_mod_2,rep_mod_3,rep_mod_4,rep_mod_5]
                
#                 for rep_mod, th_rc in zip(rep_mod_list, th_rc_list):
                    
#                     if rep_mod == 0:
                        
#                         rep_cost = th_rc * discount_df.loc[0][year]
                        
#                     else:
#                         rep_cost = 0
                        
#                     rep_cost_list.append(rep_cost)
                    
                   
#                 rep_cost = sum(rep_cost_list)
                
#                 rep_cost_list.append(rep_cost)
                
#                 rep_cost_df[year] = rep_cost_list
        
# #             print(rep_cost_df)
            
#             rep_cost_total = rep_cost_df.sum(axis = 1).get(key = 5)
# #             print('rep_cost_total',rep_cost_total)
# #             print(type(rep_cost_total))

#             tco_df_loop['replacement'] = rep_cost_total
            
#             '''DHW & heating energy demand'''

            
#             dhw_df = pd.DataFrame()
            
#             cop_bw35 = (308/(308-278))*0.49
#             cop_bw60 = (333/(333-278))*0.44
            
#             ht_temp_level = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'ht_temp_level'].item()
#             th_systemtype_hpaw = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_systemtype_hpaw', syst_sol].item()
#             #print('arch_entry_id',arch_entry_id,'syst_sol',syst_sol,'th_systemtype_hpaw',th_systemtype_hpaw)
#             th_systemtype_hpbw = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_systemtype_hpbw', syst_sol].item()
#             #print('arch_entry_id',arch_entry_id,'syst_sol',syst_sol,'th_systemtype_hpbw',th_systemtype_hpbw)
#             th_systemtype_electro = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_systemtype_electro', syst_sol].item()
#             clim_id = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'clim_id'].item()
            
#             #Creating 
#             if dhw_decentral == 1:
                    
#                 dhw_df[arch_entry_id] = dhw_prof_tab[0] * dhw_daily_energy
#                 dhw_df = pd.concat([dhw_df]*365, ignore_index = True)
#                 display('dhw_df',name_scenario,dhw_df.head(24))
#                 dhw_dec_th[str(name_scenario)] = dhw_df[arch_entry_id]

#             else:
#                 #pass
#                 dhw_dec_th[str(name_scenario)] = 0.0
                
            
#             if th_systemtype_hpaw == 1:
#                 if dhw_decentral == 0:
                    
#                     print('hpaw aquí') 
        
#                     dhw_cen_el[str(name_scenario)] = dhw_cen_th[arch_entry_id].div(cop_aw60_df[clim_id],axis='index',level='hours') 
                    

#                 else:
#                     #pass
#                     dhw_cen_el[str(name_scenario)] = 0.0
                
#                 if ht_temp_level == 0:
                    
#                     #display('heat_dem_th',heat_dem_th)
                    
#                     heat_dem_el[str(name_scenario)] = heat_dem_th[str(name_scenario)] / cop_aw35_df[clim_id]
#                 else:
#                     heat_dem_el[str(name_scenario)] = heat_dem_th[str(name_scenario)] / cop_aw60_df[clim_id]
                    
#             if th_systemtype_hpbw == 1:
                
#                 if dhw_decentral == 0:
                                                                                
#                     print('hpbw aquí')
#                     dhw_cen_el[str(name_scenario)] = dhw_cen_th[arch_entry_id] / cop_bw60_df
                    
# #                     dhw_cen_el[str(name_scenario)] = dhw_cen_th[arch_entry_id].div(cop_bw60,axis='index',level='days')
                    
#                 else:
#                     #pass
#                     dhw_cen_el[str(name_scenario)] = 0.0
                
#                 if ht_temp_level == 0:
                    
#                     #display('heat_dem_th',heat_dem_th)

#                     heat_dem_el[str(name_scenario)] = heat_dem_th[str(name_scenario)] / cop_bw35
#                 else:

#                     heat_dem_el[str(name_scenario)] = heat_dem_th[str(name_scenario)] / cop_bw60
            
            
#             rep_cost_total = 0
#             tco_df_loop['replacement'] = rep_cost_total
#             fuel_cost_total = 0
#             cost_retro = era_avg * inv_cost_m2
#             tco_df_loop['energy'] = fuel_cost_total
            
#             if retro_id == 'retro':
#                 tco_df_loop['retrofits'] = cost_retro
#             else:
#                 tco_df_loop['retrofits'] = 0
                
            
#             tco_df_loop['total_cost'] = th_capex_f + opex_cost_final + rep_cost_total + fuel_cost_total + cost_retro
            
#             #print(tco_df_loop)
            
#             tco_df = tco_df.append(tco_df_loop,sort = False)
    
    

#     tco_df = tco_df.reset_index().drop(columns = 'index')
    
    
    
#     dhw_cen_el = dhw_cen_el.reset_index()
#     del dhw_cen_el['days']
#     del dhw_cen_el['hours']
    
#     return(tco_df,heat_dem_el,dhw_dec_th,dhw_cen_el)

In [591]:
# tco_elec_df,heat_dem_el,dhw_dec_th,dhw_cen_el = \
#               tco_elec_func(syst_elec_dict,table_2,table_4,dhw_cen_th,cop_aw35_df,cop_aw60_df, \
#                             heat_dem_th,discount_df)


# for arch_entry_id, syst_sol_list in syst_fuel_dict.items():
    
#     if arch_entry_id[-5:] == 'retro':
#         arch = arch_entry_id[:-6]
#     else: 
#         arch = arch_entry_id
        
#     dhw_decentral = table_2.loc[table_2['arch_entry_id'] == arch_entry_id, 'dhw_decentral'].item()
        
#     for syst_sol in syst_sol_list:
        
#         scen = str(arch_entry_id) + '_' + str(syst_sol)
        
#         heat_dem_el[scen] = 0
#         dhw_cen_el[scen] = 0
        
#         dhw_df = pd.DataFrame()
        
#         if dhw_decentral == 1:
                                
#             dhw_df[arch_entry_id] = dhw_prof_tab[0] * dhw_daily_energy
#             dhw_df = pd.concat([dhw_df]*365, ignore_index = True)
#             dhw_dec_th[scen] = dhw_df[arch_entry_id]
#             display('scen',scen,'dhw_daily_energy',dhw_daily_energy)
            
#         else:
#             dhw_dec_th[scen] = 0

# if time_res == 't48':
#     heat_col_list = []        
#     for i in range (2,365):
#         heat_col_list.append(i)
        
#     heat_dem_el = heat_dem_el.drop(heat_col_list)
    
#     dhwdec_col_list = []        
#     for i in range (48,8760):
#         dhwdec_col_list.append(i)
        
#     dhw_dec_th = dhw_dec_th.drop(dhwdec_col_list)
#     dhw_cen_el = dhw_cen_el.drop(dhwdec_col_list)
    
# dhw_dec_th.fillna(0,inplace = True)
# dhw_cen_el.fillna(0,inplace = True)

In [592]:
# heat_dem_el.head()

In [593]:
dhw_cen_th.head(5)

1135719_retro   1030392   1133034   1132705   1107559   1165553  \
days hours                                                                    
0    0           0.009282  0.007508  0.004664  0.007849  0.014128  0.007030   
     1           0.009827  0.007948  0.004938  0.008310  0.014957  0.007442   
     2           0.008911  0.007208  0.004478  0.007535  0.013564  0.006749   
     3           0.008699  0.007036  0.004371  0.007355  0.013240  0.006588   
     4           0.112667  0.091128  0.056610  0.095270  0.171486  0.085329   

             1069997   1021432   1136584  1206896  ...   1282703   1228993  \
days hours                                         ...                       
0    0      0.007781  0.013309  0.009419        0  ...  0.011534  0.037572   
     1      0.008237  0.014090  0.009971        0  ...  0.012211  0.039778   
     2      0.007470  0.012777  0.009042        0  ...  0.011074  0.036072   
     3      0.007291  0.012472  0.008826        0  ...  0.010809  0.035210   
     4      0.094441  0.161545  0.114324        0  ...  0.140005  0.456053   

             1294746   1300476  1297375   1265060   1294820   1316897  \
days hours                                                              
0    0      0.015288  0.015322        0  0.122237  0.017438  0.013485   
     1      0.016186  0.016222        0  0.129413  0.018462  0.014276   
     2      0.014678  0.014710        0  0.117355  0.016742  0.012946   
     3      0.014327  0.014359        0  0.114552  0.016342  0.012637   
     4      0.185569  0.185983        0  1.483725  0.211665  0.163678   

             1290392   1292871  
days hours                      
0    0      0.064497  0.232778  
     1      0.068283  0.246442  
     2      0.061921  0.223482  
     3      0.060442  0.218143  
     4      0.782870  2.825478  

[5 rows x 100 columns]

In [594]:
def dhw_dem_func(syst_all_dict,table_2,table_4,dhw_cen_th,cop_aw35_df,cop_aw60_df):
    '''fuel_presel_dict is the result of the preselection process. The dictionary includes all archetypes 
    compatible with fuel based system solutions, namely oil, wood and gas. The preselection process filters 
    system solutions based on currently installed heating techbologies and its compatibility with possible 
    alternatives, considering power operation bands and geographic and technical limitations 
    (e.g. gass connection availability, borehole suitability, district heating in place, type of DHW system, etc )
    
    key: arch_entry_id
    value: list of all suitable technologies (syst_sol_id)'''
    
#     tco_df = pd.DataFrame()
#     heat_dem_el = pd.DataFrame()
    dhw_cen_el = pd.DataFrame()
    dhw_dec_th = pd.DataFrame()    
    
    master_df = table_2.copy()
    syst_sol_df = table_4.copy()
#     year_cols = list(discount_df.columns)
#     del year_cols[0]
    
    'Calculating DHW demand'
    for arch_entry_id, syst_sol_list in syst_all_dict.items():
#         print('arch_entry_id',arch_entry_id)

        dhw_daily_energy = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'dhw_daily_energy'].item()
        dhw_decentral = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'dhw_decentral'].item()
   
        
        for syst_sol in syst_sol_list:
            
            retro_id = arch_entry_id[-5:]
                        
            if retro_id == 'retro':
                name_scenario = arch_entry_id[:-6] + '_' + arch_entry_id[-5:] + '_' + str(syst_sol) 

            else:
                name_scenario = arch_entry_id + '_' + str(syst_sol)
#             print('name_scenario',name_scenario)
            
            '''DHW & heating energy demand'''

            
            dhw_df = pd.DataFrame()
            
            cop_bw35 = (308/(308-278))*0.49
            cop_bw60 = (333/(333-278))*0.44
            

            th_systemtype_hpaw = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_systemtype_hpaw', syst_sol].item()
            th_systemtype_hpbw = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_systemtype_hpbw', syst_sol].item()
            th_electric_system = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_electric_system', syst_sol].item()
           
            th_systemtype_electro = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_systemtype_electro', syst_sol].item()
            clim_id = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'clim_id'].item()
#             print('clim_id',clim_id)
            
            if dhw_decentral == 1:
                    
                dhw_df[arch_entry_id] = dhw_prof_tab[0] * dhw_daily_energy
                dhw_df = pd.concat([dhw_df]*365, ignore_index = True)
#                 display('dhw_df',name_scenario,dhw_df.head(24))
                dhw_dec_th[str(name_scenario)] = dhw_df[arch_entry_id]
                dhw_cen_el[str(name_scenario)] = 0

            else: #centralized systems
                dhw_dec_th[str(name_scenario)] = 0.0
#                 print('dhw_dec_th',dhw_dec_th)
                if th_electric_system == 0: #centralized and fuel based
#                     print('th_electric_system',name_scenario,th_electric_system)
                    dhw_cen_el[str(name_scenario)] = 0
                else:
                    pass
                
            if th_systemtype_hpaw == 1:
#                 print('hpaw aquí',th_electric_system)
            
                if dhw_decentral == 0:
#                     cop_aw60_filt = cop_aw60_df[clim_id]
                
#                     if arch_entry_id == str(1145583):
#                         print('cop_aw60_filt',cop_aw60_filt)
                    
#                     dhw_cen_el[str(name_scenario)] = dhw_cen_th[arch_entry_id] / cop_aw60_filt
        
                    dhw_cen_el[str(name_scenario)] = dhw_cen_th[arch_entry_id].div(cop_aw60_df[clim_id],axis='index',level='hours') 
                    
                else:

                    dhw_cen_el[str(name_scenario)] = 0
                    
            if th_systemtype_hpbw == 1:
#                 print('hpbw aquí',th_electric_system)
                
                if dhw_decentral == 0:

                    dhw_cen_el[str(name_scenario)] = dhw_cen_th[arch_entry_id] / cop_bw60
                    
                else:
                    #pass
                    dhw_cen_el[str(name_scenario)] = 0
                
           
    dhw_cen_el = dhw_cen_el.reset_index().fillna(0)
    del dhw_cen_el['days']
    del dhw_cen_el['hours']
    
    return(dhw_dec_th,dhw_cen_el)

In [595]:
dhw_dec_th,dhw_cen_el = \
              dhw_dem_func(syst_all_dict,table_2,table_4,dhw_cen_th,cop_aw35_df,cop_aw60_df)

In [596]:
dhw_cen_el.head(24)

,1030392_100056,1030392_100032,1030392_100128,1030392_100147,1133034_100056,1133034_100032,1133034_100125,1133034_100145,1132705_100056,1132705_100032,...,1316897_100125,1316897_100144,1290392_100056,1290392_100032,1290392_100128,1290392_100148,1292871_100056,1292871_100032,1292871_100135,1292871_100156
0,0.0,0.0,0.003850,0.002818,0,0,0.002672,0.001751,0,0,...,0.007221,0.005062,0,0,0.036383,0.024211,0,0,0.131342,0.087379
1,0.0,0.0,0.004355,0.002984,0,0,0.002995,0.001853,0,0,...,0.007864,0.005359,0,0,0.040344,0.025632,0,0,0.146126,0.092508
2,0.0,0.0,0.004239,0.002706,0,0,0.002885,0.001681,0,0,...,0.007342,0.004860,0,0,0.038406,0.023244,0,0,0.139615,0.083889
3,0.0,0.0,0.004411,0.002641,0,0,0.002973,0.001641,0,0,...,0.007350,0.004744,0,0,0.039134,0.022688,0,0,0.142754,0.081885
4,0.0,0.0,0.059772,0.034207,0,0,0.039996,0.021250,0,0,...,0.096851,0.061441,0,0,0.522192,0.293870,0,0,1.909752,1.060615
5,0.0,0.0,0.185134,0.104235,0,0,0.123549,0.064752,0,0,...,0.296931,0.187220,0,0,1.608265,0.895473,0,0,5.886985,3.231875
6,0.0,0.0,0.309556,0.177158,0,0,0.207136,0.110053,0,0,...,0.501589,0.318200,0,0,2.704426,1.521950,0,0,9.890582,5.492911
7,0.0,0.0,0.297306,0.178004,0,0,0.200395,0.110578,0,0,...,0.495366,0.319719,0,0,2.637636,1.529215,0,0,9.621732,5.519132
8,0.0,0.0,0.149895,0.095682,0,0,0.102033,0.059439,0,0,...,0.259629,0.171858,0,0,1.358180,0.821995,0,0,4.937367,2.966683
9,0.0,0.0,0.091424,0.062638,0,0,0.062881,0.038911,0,0,...,0.165105,0.112506,0,0,0.846995,0.538115,0,0,3.067789,1.942127


In [597]:
dhw_dec_th = dhw_dec_th.replace(np.NaN,0)
dhw_dec_th.head(24)

,1030392_100056,1030392_100032,1030392_100128,1030392_100147,1133034_100056,1133034_100032,1133034_100125,1133034_100145,1132705_100056,1132705_100032,...,1316897_100125,1316897_100144,1290392_100056,1290392_100032,1290392_100128,1290392_100148,1292871_100056,1292871_100032,1292871_100135,1292871_100156
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
DataFrames to GDX data 
</p><br>

In [598]:
if time_res == 't48':
    data_scen_list = ['stes_t48','base_t48']
else:
    data_scen_list = ['stes_t8760','base_t8760']
    

data_scen_widget = widgets.Dropdown(
    options = data_scen_list,
    description = 'data scen ',
    disabled = False)
display(data_scen_widget)


Dropdown(description='data scen ', options=('stes_t8760', 'base_t8760'), value='stes_t8760')

In [599]:
data_scen = data_scen_widget.value

In [600]:
#Data directory
dir_param = './Data/Parameters' + '/parameters_dummy_test.xlsx'

xls = pd.ExcelFile(dir_param)

if data_scen == 'base_t8760':
    price_tab = 'prices_base_t8760' 
    pv_tab = 'pv_cf_t8760'
    #elec_dem_tab = 'elec_dem_t8760'

elif data_scen == 'stes_t8760':
    price_tab = 'prices_stes_t8760'
    pv_tab = 'pv_cf_t8760'
    #elec_dem_tab = 'elec_dem_t8760'

elif data_scen == 'base_t48':
#     price_tab = 'prices_base_t48'
    pv_tab = 'pv_cf_t48'
    #elec_dem_tab = 'elec_dem_t48'
    
elif data_scen == 'stes_t48':
#     price_tab = 'prices_stes_t48'
    pv_tab = 'pv_cf_t48'
    #elec_dem_tab = 'elec_dem_t48'

#Importing the data from the "parameters" excel sheet as DataFrames
solar_df = pd.read_excel(xls,'solar').round(2)
solar_df_var = pd.read_excel(xls,'solar_var').round(4)
bat_df = pd.read_excel(xls,'batteries').round(2)
price_df = pd.read_excel(xls,price_tab).round(2) 
pv_df = pd.read_excel(xls, pv_tab)  
    
# elec_df = pd.read_excel(xls, elec_dem_tab)
# elec_df.columns =  elec_df.columns.astype(str)

# elec_dem_df = pd.DataFrame()


# for scen in scen_id_list: #Looping through all archetypes
    
#     if scen[-5:] == 'retro':
        
#         arch = scen[:-6]
#     else: 
#         arch = scen 
    
#     elec_dem_df[scen] = elec_df[arch]

# elec_dem_df = elec_dem_df.round(3)

print('solar_df',solar_df)
print('bat_df',bat_df)
print('price_df',price_df)
print('pv_df',pv_df)
print('solar_df_var',solar_df_var)
#print('elec_dem_df',elec_dem_df)

solar_df    pv  pv_pel
0  PV       0
bat_df      b  bat_soc_min  bat_eff  capex_kW  opex_kW  bat_l_t  bat_en_pwr_rat  \
0  LiB            0      0.9   1580.98    23.71       15               2   
1  LiB            0      0.9    391.40     5.87       15               2   
2  LiB            0      0.9    391.40     5.87       15               2   

   rcp_scenario_year  
0               2020  
1               2035  
2               2050  
price_df           t  price_grid  price_supply
0        t1        0.27           0.1
1        t2        0.27           0.1
2        t3        0.27           0.1
3        t4        0.27           0.1
4        t5        0.27           0.1
...     ...         ...           ...
8755  t8756        0.27           0.1
8756  t8757        0.27           0.1
8757  t8758        0.27           0.1
8758  t8759        0.27           0.1
8759  t8760        0.27           0.1

[8760 rows x 3 columns]
pv_df           t  pv_cf
0        t1    0.0
1        t2    0.0
2     

In [601]:
t_list = []

for i in range (1,8761):
    t = 't' + str(i)
    t_list.append(t)

In [602]:
elec_price = table_18.loc[table_18.energy_price == 'avg_spot', rcp_scenario_year].item()
elec_price_df = table_19[['Canton']].copy()

elec_price

15.774471843711671

In [603]:
elec_price_df['avg_elec_price'] = (elec_price*table_19['mean_%']+ elec_price*table_19['Netznutzung_%']+ \
                             elec_price*table_19['Abgabe_%']+ table_19['KEV_CHF'])*(1+table_19['MWST_%'])
                           
elec_price_df

,Canton,avg_elec_price
0,AR,41.436686
1,AI,39.108495
2,AG,40.128181
3,BE,49.171732
4,BL,44.169320
5,BS,55.324316
6,FR,44.893518
7,GL,46.308932
8,GE,39.387480
9,GR,45.230853


In [604]:
tariff_widget = widgets.Dropdown(
    options = ['flat','double'],
    description = 'Tariff struct ',
    disabled = False)
display(tariff_widget)

Dropdown(description='Tariff struct ', options=('flat', 'double'), value='flat')

In [605]:
tariff_struct = tariff_widget.value
tariff_struct

'flat'

In [606]:
table_2.canton

0     GR
1     NE
2     BE
3     BE
4     SZ
      ..
95    ZH
96    BL
97    ZH
98    SH
99    ZH
Name: canton, Length: 100, dtype: object

In [607]:
canton_list = elec_price_df.Canton.tolist()
arch_id_list = table_2.arch_entry_id.tolist()
price_grid = pd.DataFrame()
price_supply = pd.DataFrame()

for arch_id in arch_id_list:
    canton = table_2.loc[table_2['arch_entry_id'] == arch_id,'canton'].item()
    canton_price = elec_price_df.loc[elec_price_df.Canton == canton,'avg_elec_price'].item()
    if tariff_struct == 'flat':
        price_grid['t'] = t_list
        price_grid[arch_id] = canton_price
        price_supply['t'] = t_list
        price_supply[arch_id] = 0.1
        
    if tariff_struct == 'double':
        double_price_list = []
        price_day = canton_price*1.07416
        price_night = canton_price*0.656934
        
        for h in range(1,25):
            if (h<=6) or (h>=21):   
                double_price_list.append(price_night)
            else:
                double_price_list.append(price_day)
                
        double_price_list = double_price_list*365
        
        price_grid['t'] = t_list    
        price_grid[arch_id] = double_price_list
        
        price_supply['t'] = t_list
        price_supply[arch_id] = 0.1
               
price_grid

,t,1135719_retro,1030392,1133034,1132705,1107559,1165553,1069997,1021432,1136584,...,1282703,1228993,1294746,1300476,1297375,1265060,1294820,1316897,1290392,1292871
0,t1,45.230853,45.140181,49.171732,49.171732,40.523565,49.171732,49.171732,49.171732,52.230595,...,45.325228,44.16932,49.171732,44.893518,40.866229,38.13574,44.16932,38.13574,43.358392,38.13574
1,t2,45.230853,45.140181,49.171732,49.171732,40.523565,49.171732,49.171732,49.171732,52.230595,...,45.325228,44.16932,49.171732,44.893518,40.866229,38.13574,44.16932,38.13574,43.358392,38.13574
2,t3,45.230853,45.140181,49.171732,49.171732,40.523565,49.171732,49.171732,49.171732,52.230595,...,45.325228,44.16932,49.171732,44.893518,40.866229,38.13574,44.16932,38.13574,43.358392,38.13574
3,t4,45.230853,45.140181,49.171732,49.171732,40.523565,49.171732,49.171732,49.171732,52.230595,...,45.325228,44.16932,49.171732,44.893518,40.866229,38.13574,44.16932,38.13574,43.358392,38.13574
4,t5,45.230853,45.140181,49.171732,49.171732,40.523565,49.171732,49.171732,49.171732,52.230595,...,45.325228,44.16932,49.171732,44.893518,40.866229,38.13574,44.16932,38.13574,43.358392,38.13574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,t8756,45.230853,45.140181,49.171732,49.171732,40.523565,49.171732,49.171732,49.171732,52.230595,...,45.325228,44.16932,49.171732,44.893518,40.866229,38.13574,44.16932,38.13574,43.358392,38.13574
8756,t8757,45.230853,45.140181,49.171732,49.171732,40.523565,49.171732,49.171732,49.171732,52.230595,...,45.325228,44.16932,49.171732,44.893518,40.866229,38.13574,44.16932,38.13574,43.358392,38.13574
8757,t8758,45.230853,45.140181,49.171732,49.171732,40.523565,49.171732,49.171732,49.171732,52.230595,...,45.325228,44.16932,49.171732,44.893518,40.866229,38.13574,44.16932,38.13574,43.358392,38.13574
8758,t8759,45.230853,45.140181,49.171732,49.171732,40.523565,49.171732,49.171732,49.171732,52.230595,...,45.325228,44.16932,49.171732,44.893518,40.866229,38.13574,44.16932,38.13574,43.358392,38.13574


In [608]:
'''Creating the lists that will be used to create the set and parameters of the GDX file'''

#Creating the lists for GAMS's sets
a_list = list(syst_elec_dict.keys()) + list(syst_fuel_dict.keys())
a_list = list(dict.fromkeys(a_list))
pv_list = list(pd.unique(solar_df['pv'].tolist())) 
b_list = list(pd.unique(bat_df['b'].tolist()))
# t_list = list(pd.unique(price_df['t'].tolist()))
ah_scen_list = list(dhw_dec_th)
print(ah_scen_list)


#Creating empty lists for each parameter, the will be filled with the model's inputa data
solar_list = []
solar_var_list = []
bat_list = []
grid_list = []
heat_par_list = []
stes_par_list = []
dhw_par_list = []
price_list = []
pvcf_list = []
elec_dem_list = []
heat_dem_th_list = []
heat_dem_el_list = []
dhw_cen_list = []
dhw_dec_list = []
cop_aw35_list = []
cop_aw60_list = []


['1030392_100056', '1030392_100032', '1030392_100128', '1030392_100147', '1133034_100056', '1133034_100032', '1133034_100125', '1133034_100145', '1132705_100056', '1132705_100032', '1132705_100127', '1132705_100147', '1107559_100056', '1107559_100032', '1107559_100130', '1107559_100150', '1165553_100056', '1165553_100032', '1165553_100126', '1165553_100146', '1069997_100056', '1069997_100032', '1069997_100126', '1069997_100146', '1021432_100056', '1021432_100032', '1021432_100129', '1021432_100148', '1136584_100056', '1136584_100032', '1136584_100127', '1136584_100147', '1206896_100042', '1206896_100001', '1206896_100017', '1206896_100069', '1206896_100089', '1023478_100056', '1023478_100032', '1023478_100125', '1023478_100145', '1110917_100042', '1110917_100017', '1110917_100001', '1110917_100069', '1110917_100089', '1107180_100044', '1107180_100018', '1107180_100001', '1107180_100071', '1107180_100091', '1118480_100056', '1118480_100032', '1118480_100126', '1118480_100146', '1035864_

**Creating electric demand profiles per archetype**

In [609]:
# master_df_elec = building_code(tables.get('2_master_table'))

# tot_elec_dem = tables.get('12_elec_dem_total')

# elec_dem_df = tables.get('10_elec_profiles').dropna().reset_index().drop(columns = ['time','index'])
# #print('elec_dem_df',elec_dem_df)

# arch_entry_id_list = list(pd.unique(table_2["arch_entry_id"].tolist()))

# arch_typ_list = list(pd.unique(table_3['arch_typ'].tolist()))
# print('arch_typ_list',arch_typ_list)

# elec_dem_ind_df = pd.DataFrame()

# elec_dem_ind_list = []

# arch_dem_ind_list = []

# for arch_typ in arch_typ_list:
    
#     arch_typ_dem = tot_elec_dem.loc[tot_elec_dem['arch_typ'] == arch_typ,'tot_elec_dem'].item()
    
#     arch_typ_df  = master_df_elec[master_df_elec.arch_typ == arch_typ]
      
#     arch_typ_df['tot_area'] = arch_typ_df['era_avg']*arch_typ_df['tot_buildings']
    
#     print('arch_typ_df',arch_typ_df)
    
#     tot_area = arch_typ_df['tot_area'].sum()
#     print('arch_typ','tot_area',tot_area)
    
#     print('arch_typ_dem',arch_typ_dem)
    
#     arch_dem_ind_list.append(arch_typ) 
    
#     if tot_area != 0:
    
#         elec_dem_ind = arch_typ_dem / tot_area
        
#         print('elec_dem_ind',elec_dem_ind)
        
#         elec_dem_ind_list.append(elec_dem_ind)
        
#         print('elec_dem_ind',elec_dem_ind)
    
#     else:
#         elec_dem_ind_list.append(0)

    

# print('arch_typ_list',arch_typ_list)

# print('elec_dem_ind_list',elec_dem_ind_list)


# elec_dem_ind_df['arch_typ'] = arch_dem_ind_list

# elec_dem_ind_df['elec_dem_ind'] = elec_dem_ind_list 

# elec_dem_ind_df

In [610]:
master_df = table_2.copy()
elec_dem_df = tables.get('10_elec_profiles').dropna().reset_index().drop(columns = ['time','index'])
display(elec_dem_df)
elec_dem_ind_df = tables.get('12_elec_dem_ind')
arch_entry_id_list = list(pd.unique(table_2["arch_entry_id"].tolist()))
arch_typ_list = list(pd.unique(table_3['arch_typ'].tolist()))

for arch_entry_id in arch_entry_id_list:
    
    arch_typ = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'arch_typ'].item()
    
    era_avg = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'era_avg'].item()
    
    elec_dem_ind = elec_dem_ind_df.loc[elec_dem_ind_df['arch_typ'] == arch_typ,'elec_dem_ind_era'].item()
    
    arch_typ = arch_typ[0:3]
    
    elec_dem_df[arch_entry_id] = elec_dem_df[arch_typ]*elec_dem_ind*era_avg
    

if time_res == 't48':
    
    elec_del_row = []
    
    for i in range (48,8760):
        
        elec_del_row.append(i)

    elec_dem_df = elec_dem_df.drop(elec_del_row)
    
elec_dem_df['t'] = t_list

elec_dem_df = elec_dem_df.drop(columns=['MFH','SFH'])

elec_dem_df

,MFH,SFH
0,0.000106,0.000067
1,0.000095,0.000056
2,0.000084,0.000059
3,0.000076,0.000061
4,0.000067,0.000061
...,...,...
8755,0.000198,0.000164
8756,0.000184,0.000128
8757,0.000157,0.000111
8758,0.000135,0.000092


,1135719_retro,1030392,1133034,1132705,1107559,1165553,1069997,1021432,1136584,1206896,...,1228993,1294746,1300476,1297375,1265060,1294820,1316897,1290392,1292871,t
0,0.272213,0.220172,0.136774,0.230180,0.414324,0.206161,0.228178,0.390305,0.276216,0.226177,...,1.946409,0.448351,0.449351,0.184144,6.332458,0.511400,0.395459,3.341246,12.058983,t1
1,0.228529,0.184840,0.114825,0.193242,0.347835,0.173077,0.191561,0.327671,0.231890,0.189881,...,1.744887,0.376401,0.377242,0.154593,5.676826,0.429333,0.331998,2.995310,10.810455,t2
2,0.240259,0.194327,0.120718,0.203160,0.365689,0.181961,0.201394,0.344489,0.243792,0.199627,...,1.545524,0.395721,0.396604,0.162528,5.028216,0.451369,0.349038,2.653079,9.575298,t3
3,0.244708,0.197926,0.122954,0.206923,0.372461,0.185331,0.205123,0.350869,0.248307,0.203324,...,1.397373,0.403049,0.403949,0.165538,4.546221,0.459728,0.355502,2.398760,8.657429,t4
4,0.245517,0.198580,0.123360,0.207607,0.373692,0.185943,0.205801,0.352029,0.249128,0.203996,...,1.219958,0.404382,0.405284,0.166085,3.969018,0.461248,0.356677,2.094205,7.558253,t5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,0.663342,0.536527,0.333297,0.560914,1.009646,0.502384,0.556037,0.951115,0.673097,0.551159,...,3.630590,1.092563,1.095002,0.448731,11.811785,1.246205,0.963675,6.232349,22.493338,t8756
8756,0.517730,0.418752,0.260134,0.437787,0.788016,0.392105,0.433980,0.742334,0.525344,0.430173,...,3.372113,0.852732,0.854636,0.350229,10.970853,0.972648,0.752137,5.788641,20.891940,t8757
8757,0.448969,0.363137,0.225585,0.379643,0.683358,0.340028,0.376342,0.643743,0.455572,0.373041,...,2.873448,0.739479,0.741129,0.303715,9.348495,0.843468,0.652243,4.932623,17.802462,t8758
8758,0.373737,0.302287,0.187784,0.316027,0.568849,0.283050,0.313279,0.535872,0.379233,0.310531,...,2.471539,0.615566,0.616940,0.252822,8.040920,0.702130,0.542949,4.242697,15.312431,t8759


In [611]:
elec_dem_ind_df

,arch_typ,elec_dem_ind_cecb,tot_elec_dem,tot_area,tot_era_avg,elec_dem_ind_era,era_roof_new
0,SFH-Rural,17.902191,2376702210,1.327604e+08,75869547,29.740943,3.892998e+07
1,SFH-Suburban,20.412897,2051545090,1.005024e+08,92487158,29.740943,3.880819e+07
2,SFH-Urban,20.338770,1149095790,5.649780e+07,19174106,29.740943,7.577956e+06
3,MFH-Rural,25.204493,1916393390,7.603380e+07,39237742,49.837113,1.401543e+07
4,MFH-Suburban,29.112098,3038621470,1.043766e+08,108732184,49.837113,2.902574e+07
5,MFH-Urban,27.380277,6483847050,2.368072e+08,81555047,49.837113,1.965745e+07


In [612]:
def alpha_func(wacc,lifetime):
    
    try:
    
        alpha = wacc/(1-((1+wacc)**(-lifetime))) 
    
        return(alpha)

    except ZeroDivisionError:
        return(0)

In [613]:
alpha_pv = alpha_func(wacc = 0.04,lifetime = 30) 
alpha_pv

0.057830099133661314

**Creating the final battery parameters table**

In [614]:
bat_df

,b,bat_soc_min,bat_eff,capex_kW,opex_kW,bat_l_t,bat_en_pwr_rat,rcp_scenario_year
0,LiB,0,0.9,1580.98,23.71,15,2,2020
1,LiB,0,0.9,391.40,5.87,15,2,2035
2,LiB,0,0.9,391.40,5.87,15,2,2050


In [615]:
b_tech_list = pd.unique(bat_df['b']).tolist()
bat_df = bat_df[bat_df.rcp_scenario_year == rcp_scenario_year]
#rcp_scenario_year 

for b in b_tech_list:
    bat_l_t = bat_df.loc[bat_df['b'] == b, 'bat_l_t'].item()
    bat_en_pwr_rat = bat_df.loc[bat_df['b'] == b, 'bat_en_pwr_rat'].item()
    alpha_bat = alpha_func(wacc = 0.04,lifetime = bat_l_t) 
    
    bat_df['eac_capex_kWh_CHF'] = ((bat_df.loc[bat_df['b'] == b, 'capex_kW'].item())*alpha_bat)/bat_en_pwr_rat
    bat_df['eac_opex_kWh_CHF'] = (bat_df.loc[bat_df['b'] == b, 'opex_kW'].item())/bat_en_pwr_rat

    bat_df = bat_df.drop(columns = ['capex_kW','opex_kW'])
    bat_df = bat_df[['b','bat_soc_min','bat_eff','bat_l_t','bat_en_pwr_rat','eac_capex_kWh_CHF','eac_opex_kWh_CHF','rcp_scenario_year']]
                        

bat_df

,b,bat_soc_min,bat_eff,bat_l_t,bat_en_pwr_rat,eac_capex_kWh_CHF,eac_opex_kWh_CHF,rcp_scenario_year
2,LiB,0,0.9,15,2,17.601473,2.935,2050


**Creating PV profiles and parameters table**

In [616]:
pv_var_df = pd.DataFrame()

pv_var_arch_list = []

pv_var_roof_list = []

for arch_entry_id in arch_entry_id_list:
    
    roof_era = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'era_roof_new'].item()
    
    pv_var_arch_list.append(arch_entry_id)
    
    pv_var_roof_list.append(roof_era) 
    
pv_var_df['arch_entry_id']= pv_var_arch_list

pv_var_df['roof_era']= pv_var_roof_list

pv_pel_max_m2 =  solar_df_var.loc[solar_df_var['rcp_scenario_year'] == rcp_scenario_year, 'pv_pel_max_m2'].item()

capex_kWp_CHF =  solar_df_var.loc[solar_df_var['rcp_scenario_year'] == rcp_scenario_year, 'capex_kWp_CHF'].item()

opex_kWp_CHF =  solar_df_var.loc[solar_df_var['rcp_scenario_year'] == rcp_scenario_year, 'opex_kWp_CHF'].item()

pv_var_df['pv_pel_max_m2']= pv_pel_max_m2

pv_var_df['eac_capex_kWp_CHF']= capex_kWp_CHF*alpha_pv

pv_var_df['eac_opex_kWp_CHF']= opex_kWp_CHF

pv_var_df

,arch_entry_id,roof_era,pv_pel_max_m2,eac_capex_kWp_CHF,eac_opex_kWp_CHF
0,1135719_retro,38.150000,0.1875,63.670939,62.5
1,1030392,47.600000,0.1875,63.670939,62.5
2,1133034,77.000000,0.1875,63.670939,62.5
3,1132705,82.600000,0.1875,63.670939,62.5
4,1107559,79.800000,0.1875,63.670939,62.5
...,...,...,...,...,...
95,1265060,222.600000,0.1875,63.670939,62.5
96,1294820,65.800000,0.1875,63.670939,62.5
97,1316897,41.521667,0.1875,63.670939,62.5
98,1290392,122.500000,0.1875,63.670939,62.5


In [617]:
#pv_prof_df = tables.get('11_pv_profiles').dropna().reset_index().drop(columns = ['time','index'])
table_11 = tables.get('11_pv_profiles')
pv_prof_df = pd.DataFrame()
#canton_full_list = list(pd.unique(table_11["canton"].tolist()))
canton_dict = {'AG':'Aargau',
               'AI':'SGallen',
               'AR':'SGallen',
               'BE':'Bern',
               'BL':'BL',
               'BS':'BL',
               'FR':'Freiburg',
               'GE':'Geneva',
               'GL':'Glarus',
               'GR':'Grissons',
               'JU':'Jura',
               'LU':'Lucerne',
               'NE':'Neuchetal',
               'NW':'Nidwalden',
               'OW':'Obwald',
               'SG':'SGallen',
               'SH':'Schaffhausen',
               'SO':'Solothurn',
               'SZ':'Schwyz',
               'TG':'Thargau',
               'TI':'Ticino',
               'UR':'Uri',
               'VD':'Vaud',
               'VS':'Valais',
               'ZG':'Zug',
               'ZH':'Zurich'}

for canton_abv, canton_name in canton_dict.items():
    pv_canton_df = table_11[table_11.canton == canton_name].reset_index() 
    #print(pv_canton_df)
    pv_canton_df = pv_canton_df.drop(columns = ['date_trunc','canton','year','index'])
    pv_canton_df = pv_canton_df.rename(columns={'kwh': canton_abv}).round(4)
    #print(pv_canton_df)
    
    #pv_prof_df = pv_prof_df + pv_canton_df 
    pv_prof_df = pd.concat([pv_prof_df,pv_canton_df],axis = 1 ,sort=True)
    
pv_prof_df

,AG,AI,AR,BE,BL,BS,FR,GE,GL,GR,...,SH,SO,SZ,TG,TI,UR,VD,VS,ZG,ZH
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [618]:
pv_prof_df.sum(axis=0)

AG    1310.2132
AI    1286.7506
AR    1286.7506
BE    1435.1614
BL    1357.6477
BS    1357.6477
FR    1456.7787
GE    1428.8793
GL    1211.3037
GR    1483.8707
JU    1316.6757
LU    1252.6281
NE    1463.1743
NW    1156.4425
OW    1247.2441
SG    1286.7506
SH    1286.4672
SO    1377.6450
SZ    1013.0783
TG    1150.4720
TI    1524.8014
UR    1246.0138
VD    1419.4708
VS    1583.9416
ZG    1264.3627
ZH    1301.8276
dtype: float64

In [619]:
arch_entry_id_list = pd.unique(table_2["arch_entry_id"]).tolist()

pv_canton_df = pd.DataFrame()

for arch_entry_id in arch_entry_id_list:
     
    canton = table_2.loc[table_2['arch_entry_id'] == arch_entry_id, 'canton'].item()
    
    pv_canton_df[arch_entry_id] = pv_prof_df[canton]
       
if time_res == 't48':
    
    pv_del_row = []
    
    for i in range (48,8760):
        
        pv_del_row.append(i)

    pv_canton_df = pv_canton_df.drop(pv_del_row)
    
pv_canton_df['t'] = t_list

pv_canton_df

,1135719_retro,1030392,1133034,1132705,1107559,1165553,1069997,1021432,1136584,1206896,...,1228993,1294746,1300476,1297375,1265060,1294820,1316897,1290392,1292871,t
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,t1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,t2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,t3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,t4
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,t5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,t8756
8756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,t8757
8757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,t8758
8758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,t8759


**Heating and DHW tables**

In [620]:
'''Creating heat_par & dhw_par dataframes'''
grid_df = pd.DataFrame(columns = a_list).transpose()

grid_df['grid_max_dem'] = 15
grid_df['grid_max_sup'] = 5

grid_df = grid_df.rename_axis('a').reset_index().round(2)

print(grid_df)

          a  grid_max_dem  grid_max_sup
0   1030392            15             5
1   1133034            15             5
2   1132705            15             5
3   1107559            15             5
4   1165553            15             5
..      ...           ...           ...
92  1265060            15             5
93  1294820            15             5
94  1316897            15             5
95  1290392            15             5
96  1292871            15             5

[97 rows x 3 columns]


In [621]:
h_list = []

for arch_entry_id, syst_sol_list in syst_fuel_dict.items():
    for syst_sol in syst_sol_list:
        h_list.append(syst_sol)
        
for arch_entry_id, syst_sol_list in syst_elec_dict.items():
    for syst_sol in syst_sol_list:
        h_list.append(syst_sol)

In [622]:
h_list = pd.unique(h_list).tolist()
len(h_list)

61

In [623]:
'''Creating heat_par & dhw_par dataframes'''
# heat_scen_list = list(dhw_dec_th) 

heat_dict = {}
dhw_dict = {}

dhw_soc_min = float(0)
dhw_eff = 1

for h_heat in h_list:
    heat_list = []
    th_pel_hp = table_4.loc[table_4['th_syssolution_id'] == 'th_pel_hp', h_heat].item()
    th_efficiency = table_4.loc[table_4['th_syssolution_id'] == 'th_efficiency', h_heat].item()
    heat_list.extend([th_pel_hp,th_efficiency])
    heat_dict[h_heat] = heat_list


for a_heat in a_list:
   
    dhw_list = []
    dhw_pel = table_2.loc[table_2['arch_entry_id'] == a_heat, 'dhw_power_decentral'].item()
    dhw_st_cap = table_2.loc[table_2['arch_entry_id'] == a_heat, 'dhw_storage_decentral'].item()
    dhw_decentral = table_2.loc[table_2['arch_entry_id'] == a_heat, 'dhw_decentral'].item()
    dhw_list.extend([dhw_soc_min,dhw_eff,dhw_pel,dhw_st_cap,dhw_decentral])
    dhw_dict[a_heat] = dhw_list
    

heat_par = pd.DataFrame.from_dict(heat_dict, orient='index',
                       columns = ['th_pel_hp','th_efficiency'])

heat_par = heat_par.rename_axis('h').reset_index().round(2)

dhw_par = pd.DataFrame.from_dict(dhw_dict, orient='index',
                       columns = ['dhw_soc_min','dhw_eff','dhw_pel','dhw_st_cap','dhw_decentral'])

dhw_par = dhw_par.rename_axis('a').reset_index().round(2)
   
print(heat_par)
print(dhw_par)

         h  th_pel_hp  th_efficiency
0   100056       0.00           0.94
1   100032       0.00           0.93
2   100042       0.00           0.93
3   100001       0.00           0.91
4   100017       0.00           0.93
..     ...        ...            ...
56  100153      26.60           0.00
57  100097      31.50           0.00
58  100066       1.88           0.00
59  100124       2.24           0.00
60  100156      43.22           0.00

[61 rows x 3 columns]
          a  dhw_soc_min  dhw_eff  dhw_pel  dhw_st_cap  dhw_decentral
0   1030392          0.0        1      0.0         0.0              0
1   1133034          0.0        1      0.0         0.0              0
2   1132705          0.0        1      0.0         0.0              0
3   1107559          0.0        1      0.0         0.0              0
4   1165553          0.0        1      0.0         0.0              0
..      ...          ...      ...      ...         ...            ...
92  1265060          0.0        1      0.0 

In [624]:
# dhw_dec_th_copy = dhw_dec_th.copy()
# dhw_cen_el_copy = dhw_cen_el.copy()
# heat_dem_th_copy = heat_dem_th.copy()

# dhw_scen_df = pd.DataFrame()

# dhw_dec_th_copy = dhw_dec_th_copy.transpose()
# dhw_cen_el_copy = dhw_cen_el_copy.transpose()
# heat_dem_th_copy = heat_dem_th_copy.transpose()

# a_dhw_list = []
# h_dhw_list = []

# print('ah_scen_list',ah_scen_list)
# for x in ah_scen_list:
    
#     a_dhw = x[:-7]
#     a_dhw_list.append(a_dhw)
    
#     h_dhw = x[-6:]
#     h_dhw_list.append(h_dhw)

    
# print('a_dhw_list',a_dhw_list, len(a_dhw_list))
# print('h_dhw_list',h_dhw_list, len(h_dhw_list))
# dhw_dec_th_copy.columns = t_list
# dhw_cen_el_copy.columns = t_list
# heat_dem_th_copy.columns = d_list

# dhw_dec_th_copy['a'] = a_dhw_list 
# dhw_dec_th_copy['h'] = h_dhw_list

# dhw_cen_el_copy['a'] = a_dhw_list 
# dhw_cen_el_copy['h'] = h_dhw_list 

# heat_dem_th_copy['a'] = a_dhw_list 
# heat_dem_th_copy['h'] = h_dhw_list

# dhw_dec_th_copy = dhw_dec_th_copy.rename_axis('heat_scen').reset_index().round(3)
# dhw_cen_el_copy = dhw_cen_el_copy.rename_axis('heat_scen').reset_index().round(3)
# heat_dem_th_copy = heat_dem_th_copy.rename_axis('heat_scen').reset_index().round(3)

# display('dhw_dec_th',dhw_dec_th_copy.head(30))
# display('dhw_cen_el_copy',dhw_cen_el_copy.head(30))
# display('heat_dem_th_copy',heat_dem_th_copy.head(30))

In [625]:
def heat_dhw_format(heat_dhw_df,t_res_list):
    
    heat_dhw_df_copy = heat_dhw_df.copy()
    heat_dhw_col_list = list(heat_dhw_df_copy.columns.values)
    heat_dhw_df_copy = heat_dhw_df_copy.transpose()
    
    a_heat_dhw_list = []
    h_heat_dhw_list = []
 
    for ah_scen in heat_dhw_col_list:
        
        a_heat_dhw = ah_scen[:-7]
        a_heat_dhw_list.append(a_heat_dhw)

        h_heat_dhw = ah_scen[-6:]
        h_heat_dhw_list.append(h_heat_dhw)
        
    heat_dhw_df_copy.columns = t_res_list
    
    heat_dhw_df_copy['a'] = a_heat_dhw_list 
    heat_dhw_df_copy['h'] = h_heat_dhw_list

    heat_dhw_df_copy = heat_dhw_df_copy.rename_axis('heat_scen').reset_index().round(3)
    
    display(heat_dhw_df_copy.head(30))
    
    return(heat_dhw_df_copy)
    


In [626]:
dhw_dec_th_copy = heat_dhw_format(heat_dhw_df = dhw_dec_th,
                                  t_res_list = t_list)
dhw_cen_el_copy = heat_dhw_format(heat_dhw_df = dhw_cen_el,
                                  t_res_list = t_list)
heat_dem_th_copy = heat_dhw_format(heat_dhw_df = heat_dem_th,
                                   t_res_list = d_list)

,heat_scen,t1,t2,t3,t4,t5,t6,t7,t8,t9,...,t8753,t8754,t8755,t8756,t8757,t8758,t8759,t8760,a,h
0,1030392_100056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1030392,100056
1,1030392_100032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1030392,100032
2,1030392_100128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1030392,100128
3,1030392_100147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1030392,100147
4,1133034_100056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1133034,100056
5,1133034_100032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1133034,100032
6,1133034_100125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1133034,100125
7,1133034_100145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1133034,100145
8,1132705_100056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1132705,100056
9,1132705_100032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1132705,100032


,heat_scen,t1,t2,t3,t4,t5,t6,t7,t8,t9,...,t8753,t8754,t8755,t8756,t8757,t8758,t8759,t8760,a,h
0,1030392_100056,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1030392,100056
1,1030392_100032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1030392,100032
2,1030392_100128,0.004,0.004,0.004,0.004,0.060,0.185,0.310,0.297,0.150,...,0.083,0.114,0.189,0.229,0.143,0.087,0.040,0.004,1030392,100128
3,1030392_100147,0.003,0.003,0.003,0.003,0.034,0.104,0.177,0.178,0.096,...,0.061,0.074,0.111,0.129,0.083,0.056,0.029,0.003,1030392,100147
4,1133034_100056,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1133034,100056
5,1133034_100032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1133034,100032
6,1133034_100125,0.003,0.003,0.003,0.003,0.040,0.124,0.207,0.200,0.102,...,0.054,0.075,0.126,0.153,0.095,0.057,0.026,0.002,1133034,100125
7,1133034_100145,0.002,0.002,0.002,0.002,0.021,0.065,0.110,0.111,0.059,...,0.038,0.046,0.069,0.080,0.052,0.035,0.018,0.002,1133034,100145
8,1132705_100056,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1132705,100056
9,1132705_100032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1132705,100032


,heat_scen,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,d358,d359,d360,d361,d362,d363,d364,d365,a,h
0,1030392_100128,180.662,153.251,115.063,120.905,152.982,157.765,144.236,100.590,89.402,...,84.052,78.547,101.376,110.811,135.980,124.913,132.760,116.948,1030392,100128
1,1030392_100147,180.662,153.251,115.063,120.905,152.982,157.765,144.236,100.590,89.402,...,84.052,78.547,101.376,110.811,135.980,124.913,132.760,116.948,1030392,100147
2,1133034_100125,113.947,91.558,68.962,63.946,95.267,102.748,91.059,67.692,52.147,...,60.320,66.582,77.589,80.641,98.457,81.972,66.521,64.919,1133034,100125
3,1133034_100145,113.947,91.558,68.962,63.946,95.267,102.748,91.059,67.692,52.147,...,60.320,66.582,77.589,80.641,98.457,81.972,66.521,64.919,1133034,100145
4,1132705_100127,194.324,147.947,112.172,107.284,151.055,154.461,139.864,110.188,86.757,...,93.519,100.779,118.567,123.260,147.186,125.577,112.271,106.103,1132705,100127
5,1132705_100147,194.324,147.947,112.172,107.284,151.055,154.461,139.864,110.188,86.757,...,93.519,100.779,118.567,123.260,147.186,125.577,112.271,106.103,1132705,100147
6,1107559_100130,314.081,248.879,181.296,185.906,246.792,237.221,221.601,177.950,144.710,...,153.777,158.557,182.636,189.324,205.271,176.926,189.777,191.624,1107559,100130
7,1107559_100150,314.081,248.879,181.296,185.906,246.792,237.221,221.601,177.950,144.710,...,153.777,158.557,182.636,189.324,205.271,176.926,189.777,191.624,1107559,100150
8,1165553_100126,158.193,141.428,95.844,77.655,130.309,150.571,130.597,93.021,70.791,...,87.649,90.849,121.328,130.727,140.563,120.207,103.713,88.049,1165553,100126
9,1165553_100146,158.193,141.428,95.844,77.655,130.309,150.571,130.597,93.021,70.791,...,87.649,90.849,121.328,130.727,140.563,120.207,103.713,88.049,1165553,100146


**STES preselection process and parameters**

In [627]:
# stes_factor_widget = widgets.Dropdown(
#     options = [0.1,0.2,0.3,0.5,0.8,1],
#     description = 'STES factor',
#     disabled = False)
# display(stes_factor_widget)

In [628]:
# stes_factor = stes_factor_widget.value

In [629]:
# arch_stes_list = []
# sol_id_stes_list = []
# ahs_scen_list = []
# s_scen_list = []

# print('ah_scen_list',ah_scen_list)


# for ah in ah_scen_list:
    
#     syst_sol_df = table_4.copy()
    
#     arch = ah[:-7]
    
#     sol_id = int(ah[-6:])
    
#     energy_specific = table_2.loc[table_2['arch_entry_id'] == arch, 'energy_specific'].item()
#     era_avg = table_2.loc[table_2['arch_entry_id'] == arch, 'era_avg'].item()
#     dhw_daily_energy = table_2.loc[table_2['arch_entry_id'] == arch, 'dhw_daily_energy'].item()
#     dhw_decentral = table_2.loc[table_2['arch_entry_id'] == arch, 'dhw_decentral'].item()
    
#     th_tot_dem = (energy_specific*era_avg) + (dhw_daily_energy*365*dhw_decentral)
    
#     th_stes_emax = stes_factor*th_tot_dem
    
# #     stes_capex = -0.0003*(th_stes_emax*th_stes_emax) + 14.774*(th_stes_emax) + 13039
    
# #     stes_code = str(int(100000 + th_stes_emax))
    
# #     print(stes_code)
    
#     stes_count = 0
    
#     if sol_id in th_hpaw_list or sol_id in th_hpbw_list:
            
#         for stes_id in th_stes_list:
#             stes_count += 1 
                        
#             th_stes_emaxtot = syst_sol_df.loc[syst_sol_df ['th_syssolution_id'] == 'th_stes_emaxtot', stes_id].item()
            
#             if th_stes_emaxtot >= th_stes_emax:
# #                 print('passed',arch,sol_id,'stes_id',stes_id,'th_stes_emax',th_stes_emax,'th_stes_emaxtot',th_stes_emaxtot)
#                 ahs_scen = str(arch) + '_' + str(sol_id) + '_' + str(stes_id)
#                 s_scen_list.append(str(stes_id))
#                 ahs_scen_list.append(ahs_scen)
#                 arch_stes_list.append(ahs_scen)
#                 break
                
#             if stes_count == len(th_stes_list):
# #                 print('empty',arch,sol_id)
#                 ahs_scen = str(arch) + '_' + str(sol_id) + '_' + '100122'
#                 s_scen_list.append('100122')
#                 ahs_scen_list.append(ahs_scen)
        
    
#     if sol_id in th_dh_list or sol_id in th_electro_list or sol_id in th_wood_list or sol_id in th_gas_list \
#                           or sol_id in th_oil_list:
        
#         ahs_scen  = str(arch) + '_' + str(sol_id) + '_' + '100000'
#         s_scen_list.append('100000')
#         ahs_scen_list.append(ahs_scen)
        
# ahs_scen_list = list(dict.fromkeys(ahs_scen_list))
        
# # print('arch_stes_list',arch_stes_list)
# # print('sol_id_stes_list',sol_id_stes_list)
# print('ahs_scen_list',ahs_scen_list)

In [630]:
stes_factor_list = [0,0.2,0.8]
stes_factor_dict = {0:100000,0.2:200000,0.8:800000}

In [631]:
arch_stes_list = []
sol_id_stes_list = []
ahs_scen_list = []
s_scen_list = []

stes_df = table_4.copy()

# print('stes_list',th_stes_list)

# print('ah_scen_list',ah_scen_list)

for stes_factor in stes_factor_list:
    
    for ah in ah_scen_list:

        syst_sol_df = table_4.copy()
    
        try:
            arch = ah[:-7]
            sol_id = int(ah[-6:])
            
        except ValueError:
            arch = ah[:-13] + '_retro'
            sol_id = int(ah[-12:-6])
        
        energy_specific = table_2.loc[table_2['arch_entry_id'] == arch, 'energy_specific'].item()
        era_avg = table_2.loc[table_2['arch_entry_id'] == arch, 'era_avg'].item()
    #         dhw_daily_energy = table_2.loc[table_2['arch_entry_id'] == arch, 'dhw_daily_energy'].item()
    #         dhw_decentral = table_2.loc[table_2['arch_entry_id'] == arch, 'dhw_decentral'].item()
    
    
        th_tot_dem = (energy_specific*era_avg) 
    
        th_stes_emax = stes_factor*th_tot_dem
        
        '''Creating list of stes data'''
        
        stes_id = int(stes_factor_dict[stes_factor] + th_stes_emax)
        th_stes_list.append(stes_id)
        
        if stes_id == 100000:
            pass
        else:   
            th_heatingtechnology = 'stes'
            th_min_power_ht = 0
            th_max_power_ht = 0
            th_min_power_lt = 0
            th_max_power_lt = 0
            th_systemtype_dh = 0
            th_systemtype_hpaw = 0
            th_systemtype_hpbw = 0
            th_systemtype_wood = 0
            th_systemtype_electro = 0
            th_systemtype_gas = 0
            th_systemtype_oil = 0
            th_systemtype_stes = 1
            th_dhw_decentral= 1
            th_electric_system= 1
            th_capex_f = 348.15*((th_stes_emax)**0.6408) 
            th_opex_p = 0
            th_opex_f = 0
            th_rc_1 = 0
            th_rc_1t = 0
            th_rc_2 = 0
            th_rc_2t = 0
            th_rc_3 = 0
            th_rc_3t = 0
            th_rc_4 = 0
            th_rc_4t = 0
            th_rc_5 = 0
            th_rc_5t = 0
            th_l_t = 50
            th_efficiency = 0
            th_capex_dhw_1 = 0
            th_capex_dhw_2 = 0
            th_capex_dhw_3 = 0
            th_capex_dhw_4 = 0
            th_capex_dhw_5 = 0
            th_capex_dhw_6 = 0
            th_pel_hp = 0
            th_stes_volume = th_stes_emax / 81.667
            th_stes_relativedailyloss = 0.0085537737*((th_stes_emax)**0.5) 
            th_stes_placedinsidehouse = 1
            th_stes_ambienttemperature = 20
            th_stes_tmax = 90
            th_stes_dtmax = 70
            th_stes_emaxtot = th_stes_emax
            th_stes_emax1 = th_stes_emax*0.21429
            th_stes_emax2 = th_stes_emax*0.35714
            th_stes_emax3 = th_stes_emax*0.42857
            th_stes_chargeefficiency1 = 'COP35'
            th_stes_chargeefficiency2 = 'COP60'
            th_stes_chargeefficiency3 = 1
            th_stes_pmaxcharge3 = 1.7319*(np.log(th_stes_emax))-7.4565
            th_stes_socmin = 0.11

            stes_cat_list = [th_heatingtechnology,th_min_power_ht,th_max_power_ht,th_min_power_lt,th_max_power_lt,\
                            th_systemtype_dh,th_systemtype_hpaw,th_systemtype_hpbw,th_systemtype_wood,\
                            th_systemtype_electro, th_systemtype_gas,th_systemtype_oil,th_systemtype_stes,\
                            th_dhw_decentral,th_electric_system,th_capex_f,th_opex_p,th_opex_f,th_rc_1,th_rc_1t,\
                            th_rc_2,th_rc_2t,th_rc_3,th_rc_3t,th_rc_4,th_rc_4t,th_rc_5,th_rc_5t,th_l_t,th_efficiency,\
                            th_capex_dhw_1,th_capex_dhw_2,th_capex_dhw_3,th_capex_dhw_4,th_capex_dhw_5,th_capex_dhw_6,\
                             th_pel_hp,\
                            th_stes_volume,th_stes_relativedailyloss,th_stes_placedinsidehouse,th_stes_ambienttemperature,\
                            th_stes_tmax,th_stes_dtmax,th_stes_emaxtot,th_stes_emax1,th_stes_emax2,th_stes_emax3,\
                            th_stes_chargeefficiency1,th_stes_chargeefficiency2,th_stes_chargeefficiency3,
                            th_stes_pmaxcharge3,th_stes_socmin]


            stes_df[stes_id] = stes_cat_list
        
        stes_count = 0
    
        if sol_id in th_hpaw_list or sol_id in th_hpbw_list:
            ahs_scen = str(arch) + '_' + str(sol_id) + '_' + str(stes_id)
            s_scen_list.append(str(stes_id))
            ahs_scen_list.append(ahs_scen)
            arch_stes_list.append(ahs_scen)       
    
        if sol_id in th_dh_list or sol_id in th_electro_list or sol_id in th_wood_list or sol_id in th_gas_list \
                              or sol_id in th_oil_list:

            ahs_scen  = str(arch) + '_' + str(sol_id) + '_' + '100000'
            s_scen_list.append('100000')
            ahs_scen_list.append(ahs_scen)

    ahs_scen_list = list(dict.fromkeys(ahs_scen_list))

# print('arch_stes_list',arch_stes_list)
# print('sol_id_stes_list',sol_id_stes_list)
print(len(ahs_scen_list))
display(stes_df)

801


,th_syssolution_id,100000,100001,100002,100003,100004,100005,100006,100007,100008,...,808693,817679,810757,809900,804054,851730,810866,808089,820463,866943
0,th_heatingtechnology,stes,oil,oil,oil,oil,oil,oil,oil,oil,...,stes,stes,stes,stes,stes,stes,stes,stes,stes,stes
1,th_min_power_ht,0,11,18,24,30,53,65,104,145,...,0,0,0,0,0,0,0,0,0,0
2,th_max_power_ht,0,18,30,40,53,76,105,146,205,...,0,0,0,0,0,0,0,0,0,0
3,th_min_power_lt,0,11,18,25,31,55,68,109,152,...,0,0,0,0,0,0,0,0,0,0
4,th_max_power_lt,0,18,30,40,53,76,105,146,205,...,0,0,0,0,0,0,0,0,0,0
5,th_systemtype_dh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,th_systemtype_hpaw,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,th_systemtype_hpbw,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,th_systemtype_wood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,th_systemtype_electro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [632]:
'''Creating the stes parameters dataframe'''

#Creating the stes list
s_list = th_stes_list.copy()
#s_list.append(100000)
# s_list.append(th_stes_list[0])  #dummy
#s_list = th_stes_list.copy()  #full


stes_dict = {}

for s_heat in s_list:

    stes_list = []
    th_stes_volume = stes_df.loc[stes_df['th_syssolution_id'] == 'th_stes_volume', s_heat].item()
    th_stes_relativedailyloss = stes_df.loc[stes_df['th_syssolution_id'] == 'th_stes_relativedailyloss', s_heat].item()
    th_stes_placedinsidehouse = stes_df.loc[stes_df['th_syssolution_id'] == 'th_stes_placedinsidehouse', s_heat].item()
    th_stes_ambienttemperature = stes_df.loc[stes_df['th_syssolution_id'] == 'th_stes_ambienttemperature', s_heat].item()
    th_stes_tmax = stes_df.loc[stes_df['th_syssolution_id'] == 'th_stes_tmax', s_heat].item()
    th_stes_dtmax = stes_df.loc[stes_df['th_syssolution_id'] == 'th_stes_dtmax', s_heat].item()
    stes_loss_tot_th = th_stes_dtmax*(th_stes_relativedailyloss/24)
    stes_loss_tot_e1_th = (15)*(th_stes_relativedailyloss/24)
    stes_loss_tot_e2_th = (40)*(th_stes_relativedailyloss/24)
    stes_loss_tot_e3_th = (70)*(th_stes_relativedailyloss/24)
    th_stes_emaxtot = stes_df.loc[stes_df['th_syssolution_id'] == 'th_stes_emaxtot', s_heat].item()
    th_stes_emax1 = stes_df.loc[stes_df['th_syssolution_id'] == 'th_stes_emax1', s_heat].item()
    th_stes_emax2 = stes_df.loc[stes_df['th_syssolution_id'] == 'th_stes_emax2', s_heat].item()
    th_stes_emax3 = stes_df.loc[stes_df['th_syssolution_id'] == 'th_stes_emax3', s_heat].item()
    #th_stes_chargeefficiency1 = table_4.loc[table_4['name'] == 'th_stes_chargeefficiency1', s_heat].item()
    #th_stes_chargeefficiency2 = table_4.loc[table_4['name'] == 'th_stes_chargeefficiency2', s_heat].item()
    th_stes_chargeefficiency3 = stes_df.loc[stes_df['th_syssolution_id'] == 'th_stes_chargeefficiency3', s_heat].item()
    th_stes_pmaxcharge3 = stes_df.loc[stes_df['th_syssolution_id'] == 'th_stes_pmaxcharge3', s_heat].item()
    th_stes_socmin = stes_df.loc[stes_df['th_syssolution_id'] == 'th_stes_socmin', s_heat].item()
    
    stes_l_t = stes_df.loc[stes_df['th_syssolution_id'] == 'th_l_t', s_heat].item()
    stes_capex = stes_df.loc[stes_df['th_syssolution_id'] == 'th_capex_f', s_heat].item()
    
    alpha_stes = alpha_func(wacc = 0.04,lifetime = stes_l_t)
    stes_eac_capex = stes_capex*alpha_stes

    
    
    stes_list.extend([th_stes_volume,th_stes_relativedailyloss,th_stes_placedinsidehouse,\
                      th_stes_ambienttemperature,stes_loss_tot_th,stes_loss_tot_e1_th,stes_loss_tot_e2_th,\
                      stes_loss_tot_e3_th,th_stes_tmax,th_stes_dtmax,th_stes_emaxtot,th_stes_emax1,th_stes_emax2,\
                      th_stes_emax3,th_stes_chargeefficiency3,th_stes_pmaxcharge3,th_stes_socmin,stes_eac_capex])
                      
    
    stes_dict[s_heat] = stes_list

stes_par = pd.DataFrame.from_dict(stes_dict, orient='index',
                       columns = ['th_stes_volume','th_stes_relativedailyloss','th_stes_placedinsidehouse', \
                                  'th_stes_ambienttemperature','stes_loss_tot_th','stes_loss_tot_e1_th',\
                                  'stes_loss_tot_e2_th','stes_loss_tot_e3_th','th_stes_tmax','th_stes_dtmax',\
                                  'th_stes_emaxtot', 'th_stes_emax1','th_stes_emax2','th_stes_emax3',\
                                  'th_stes_chargeefficiency3','th_stes_pmaxcharge3','th_stes_socmin','stes_eac_capex']) 
                                  

stes_par = stes_par.rename_axis('s').reset_index().round(4)
print(stes_par)

          s  th_stes_volume  th_stes_relativedailyloss  \
0    100000          0.0000                     0.0000   
1    100111          5.3300                     0.1785   
2    100112          6.2500                     0.1933   
3    100113          7.3000                     0.2089   
4    100114          8.3700                     0.2236   
..      ...             ...                        ...   
201  851730        633.4348                     1.9455   
202  810866        133.0555                     0.8917   
203  808089         99.0572                     0.7694   
204  820463        250.5732                     1.2236   
205  866943        819.7189                     2.2132   

     th_stes_placedinsidehouse  th_stes_ambienttemperature  stes_loss_tot_th  \
0                            0                           0            0.0000   
1                            1                          20            0.5205   
2                            1                          20     

**Creating COP profiles**

**Daily COP**

In [633]:
# cop_profile = tables.get('13_cop_profile')

# cop_profile = pd.concat([cop_profile]*365,ignore_index=False).reset_index(drop=True)
# cop_profile['t'] = t_list
# print(cop_profile)

In [634]:
cop_aw35_df

,0,1,2,3,4,5,6,7,8,9,...,20092,20093,20094,20095,20096,20097,20098,20099,20100,20101
0,1,3.2915,3.2734,3.2945,3.2716,3.2992,3.2938,3.2929,3.2965,3.2474,...,3.3144,3.3345,3.3425,3.2461,3.3358,3.328,3.2199,3.2201,3.2667,3.2765
1,2,3.226,3.2085,3.2285,3.2128,3.2338,3.2266,3.2263,3.2302,3.1983,...,3.2375,3.2705,3.2388,3.1584,3.2352,3.1712,3.0578,3.0766,3.2102,3.193
2,3,3.1607,3.1438,3.1627,3.1542,3.1686,3.1597,3.1599,3.164,3.1492,...,3.1609,3.2067,3.1356,3.0711,3.1351,3.0157,2.8968,2.934,3.1538,3.1097
3,4,3.1055,3.0891,3.107,3.1046,3.1135,3.1031,3.1038,3.1081,3.1077,...,3.0963,3.1528,3.0486,2.9973,3.0506,2.8846,2.7612,2.8137,3.1061,3.0394
4,5,3.0686,3.0526,3.0699,3.0715,3.0768,3.0654,3.0664,3.0708,3.08,...,3.0531,3.1168,2.9906,2.9481,2.9943,2.7974,2.6708,2.7336,3.0743,2.9926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,8756,3.0557.1,3.0398,3.0569,3.0599,3.0639,3.0522,3.0533,3.0578,3.0703,...,3.038,3.1042,2.9702,2.9308,2.9746,2.7668,2.6392,2.7056,3.0631,2.9761
8756,8757,3.1122.1,3.0964,3.1138,3.1151,3.12,3.1088,3.1096,3.114,3.1257,...,3.1051,3.1639,3.0336,3,3.0384,2.8315,2.6983,2.7801,3.1177,3.0357
8757,8758,3.2672.1,3.2516,3.27,3.2666,3.274,3.2641,3.2644,3.2683,3.2777,...,3.2895,3.3279,3.2077,3.1901,3.2137,3.009,2.8604,2.9846,3.2674,3.1994
8758,8759,3.481.1,3.4658,3.4855,3.4755,3.4864,3.4785,3.4779,3.4812,3.4874,...,3.5445,3.5545,3.448,3.4527,3.4558,3.2537,3.0835,3.2669,3.4738,3.4252


In [635]:
'''Creating the cop dataframes'''

clim_id_list = []
cop_aw35_par = pd.DataFrame()
cop_aw60_par = pd.DataFrame()
cop_aw35_df_copy = cop_aw35_df.copy()
cop_aw60_df_copy = cop_aw60_df.copy()
master_df = table_2.copy()

for a in a_list:
    clim_id = master_df.loc[master_df['arch_entry_id'] == a, 'clim_id'].item()
    cop_aw35_par[a] = cop_aw35_df_copy[clim_id]
    cop_aw60_par[a] = cop_aw60_df_copy[clim_id]

# cop_aw35_par = cop_aw35_par.loc[cop_aw35_par.index.repeat(24)]
# cop_aw60_par = cop_aw60_par.loc[cop_aw60_par.index.repeat(24)]

if time_res == 't48':
    cop_aw35_par = cop_aw35_par.loc[['0','1']]
    cop_aw60_par = cop_aw60_par.loc[['0','1']]
else:
    pass

cop_aw35_par['t'] = t_list
cop_aw60_par['t'] = t_list

# cop_aw35_par = cop_aw35_par.round(2)
# cop_aw60_par = cop_aw60_par.round(2)

display('cop_aw35_par',cop_aw35_par)

'cop_aw35_par'

,1030392,1133034,1132705,1107559,1165553,1069997,1021432,1136584,1206896,1023478,...,1228993,1294746,1300476,1297375,1265060,1294820,1316897,1290392,1292871,t
0,3.4254,3.2267,3.2035,3.1906,3.3446,3.4284,3.3007,3.1719,3.3636,3.3664,...,3.4028,3.2619,3.289,3.3892,3.239,3.3632,3.3449,3.2532,3.2529,t1
1,3.304,3.1333,3.0999,3.1052,3.2425,3.286,3.186,3.0801,3.264,3.2546,...,3.3018,3.1564,3.1973,3.3343,3.1504,3.3026,3.2945,3.1757,3.1699,t2
2,3.1835,3.0404,2.9968,3.0201,3.141,3.1447,3.0719,2.9886,3.1649,3.1435,...,3.2013,3.0513,3.1059,3.2795,3.0622,3.2422,3.2442,3.0985,3.0873,t3
3,3.0819,2.9619,2.9097,2.9482,3.0553,3.0256,2.9756,2.9114,3.0813,3.0497,...,3.1164,2.9626,3.0288,3.2332,2.9876,3.1911,3.2017,3.0333,3.0175,t4
4,3.0142,2.9096,2.8517,2.9003,2.9982,2.9464,2.9115,2.8599,3.0256,2.9872,...,3.06,2.9036,2.9774,3.2023,2.9379,3.1571,3.1734,2.9898,2.971,t5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2.9905,2.8913,2.8313,2.8835,2.9782,2.9187,2.889,2.8418,3.0061,2.9654,...,3.0402,2.8828,2.9594,3.1915,2.9205,3.1451,3.1634,2.9746,2.9547,t8756
8756,3.0481,2.9518,2.9039,2.9529,3.0382,2.9908,2.9585,2.9135,3.0683,3.0255,...,3.0961,2.9507,3.029,3.2319,2.9813,3.1943,3.2136,3.0268,3.0133,t8757
8757,3.2062,3.118,3.103,3.1434,3.2029,3.1891,3.1492,3.1103,3.2392,3.1908,...,3.2497,3.1369,3.2205,3.3426,3.1483,3.3291,3.3513,3.1701,3.1742,t8758
8758,3.4244,3.347,3.3782,3.4065,3.4301,3.4632,3.4126,3.3821,3.4752,3.4187,...,3.4615,3.394,3.4851,3.495,3.3787,3.515,3.5413,3.3675,3.3962,t8759


**Hourly COP**

In [636]:
# '''Creating the cop dataframes'''

# clim_id_list = []
# cop_aw35_d = pd.DataFrame()
# cop_aw60_d = pd.DataFrame()
# cop_aw35_par = pd.DataFrame(np.nan, index = list(range(0,8760)),columns = ['0'])
# cop_aw60_par = pd.DataFrame(np.nan, index = list(range(0,8760)),columns = ['0'])
# cop_aw35_df_copy = cop_aw35_df.copy()
# cop_aw60_df_copy = cop_aw60_df.copy()
# master_df = table_2.copy()

# for a in a_list:
#     clim_id = master_df.loc[master_df['arch_entry_id'] == a, 'clim_id'].item()
#     cop_aw35_par = cop_aw35_par.reset_index().rename(columns = {'index':'t'})
#     cop_aw60_par = cop_aw60_par.reset_index().rename(columns = {'index':'t'})
    
#     cop_aw35_d[a] = cop_aw35_df_copy[clim_id]
#     cop_aw60_d[a] = cop_aw60_df_copy[clim_id]
    
#     cop_aw35_d = cop_aw35_d.reset_index()
#     cop_aw35_d['t'] = (cop_aw35_d['index']*24)+12
#     cop_aw35_d = cop_aw35_d.drop(columns='index').set_index('t')
    
#     cop_aw60_d = cop_aw60_d.reset_index()
#     cop_aw60_d['t'] = (cop_aw60_d['index']*24)+12
#     cop_aw60_d = cop_aw60_d.drop(columns='index').set_index('t')
    
    
#     cop_aw35_par = pd.concat([cop_aw35_par,cop_aw35_d],axis = 1).drop(columns= ['t','0'])
#     cop_aw60_par = pd.concat([cop_aw60_par,cop_aw60_d],axis = 1).drop(columns= ['t','0'])
    
#     cop_aw35_par = cop_aw35_par.interpolate(method = 'linear')\
#                                .interpolate(method = 'linear',limit_direction = 'backward')
#     cop_aw60_par = cop_aw60_par.interpolate(method = 'linear')\
#                                .interpolate(method = 'linear',limit_direction = 'backward')

# if time_res == 't48':
#     cop_aw35_par = cop_aw35_par.loc[list(range(0,24))]
#     cop_aw60_par = cop_aw60_par.loc[list(range(0,24))]
# else:
#     pass

# cop_aw35_par['t'] = t_list
# cop_aw60_par['t'] = t_list                
    
# cop_aw35_par

**Hourly generated COP**

In [637]:
# cop_aw35_par = table_15.copy().T
# cop_aw60_par = table_16.copy().T

# cop_aw35_par['t'] = t_list
# cop_aw60_par['t'] = t_list

# cop_aw35_par = cop_aw35_par.rename(columns={0: '1260522'})
# cop_aw60_par = cop_aw60_par.rename(columns={0: '1260522'})

# cop_aw35_par

**Scenario definition for testing all STES systems**

**For SFH**

In [638]:
# ahs_scen_list = ['1213068_100070_100000',
#                  '1213068_100070_100111',
#                  '1213068_100070_100112',
#                  '1213068_100070_100113',
#                  '1213068_100070_100114',
#                  '1213068_100070_100115',
#                  '1213068_100070_100116',
#                  '1213068_100070_100117',
#                  '1213068_100070_100118',
#                  '1213068_100070_100119',
#                  '1213068_100070_100120',
#                  '1213068_100070_100121',
#                  '1213068_100070_100122',
#                  '1213068_100043_100000',
#                  '1213068_100017_100000']

# a_dhw_list = ['1213068', '1213068', '1213068',
#               '1213068', '1213068', '1213068',
#               '1213068', '1213068', '1213068',
#               '1213068', '1213068', '1213068',
#               '1213068', '1213068', '1213068'] 

# h_dhw_list = ['100070','100070','100070',
#               '100070','100070','100070',
#               '100070','100070','100070',
#               '100070','100070','100070',
#               '100070','100043','100017'] 


**For MFH**

In [639]:
# ahs_scen_list = []

# a_dhw_list = []

# h_dhw_list = []

# for arch_entry_id, syst_sol_list in syst_elec_dict.items():
#     for syst_sol in syst_sol_list:      
#         for s in s_list:
#             ahs = str(arch_entry_id) + '_' + str(syst_sol) + '_' + str(s)
#             ahs_scen_list.append(ahs)    
#         break
        
# for ahs in ahs_scen_list:
#     a = ahs[:7]
#     h = ahs[8:14]
    
#     a_dhw_list.append(a)
#     h_dhw_list.append(h)
                  
                
# # print('ahs',ahs_scen_list,'a',a_dhw_list,'h',h_dhw_list)
# print('ahs_scen_list',len(ahs_scen_list))
# print('a_dhw_list',len(a_dhw_list))
# print('h_dhw_list',len(h_dhw_list))

**Scenario combination**

In [640]:
# '''ALL STES SCENARIOS'''

# def scenarios(ahs_scen_list,pv_list,b_list,a_dhw_list,h_dhw_list):
#     ''' Returns a DataFrame with a column named "Scenarios", creating all possible combinations of the 
#     "ID_list", "r_list". "s_list" and "hscen_list" lists. 
#     The format is "Building ID_PV capacity_Battery_Heating scenario"
#     It also removes the source columns
#     :param: df: <DataFrame> unedited Table 2 or 3
#     return: df: <DataFrame> Edited Tables with the column "Archetype"'''
    
#     #DataFrame with all possible combinations of the given lists
#     scenarios = pd.DataFrame(list(product(ahs_scen_list,pv_list,b_list)))
#     #print('scenarios',scenarios)

#     a_list = []
#     h_list = []
#     s_list = []
    
#     for ahs in ahs_scen_list:
#             #a = ahs[:-14]
#             s = int(ahs[-6:])
#             #h = int(ahs[-13:-6])
#             #h = int(ahs[:-6])
#             #print('a',a,'h',h,'s',s)
#             s_list.append(s)
            
#     scenarios.columns = ['ahs_scen','3','4']
#     #print('1',scenarios)
#     scenarios['0'] = a_dhw_list
#     scenarios['1'] = h_dhw_list
#     scenarios['2'] = s_list
#     scenarios = scenarios.drop(['ahs_scen'], axis=1)
#     scenarios = scenarios[['0','1','2','3','4']] 
#     #print('scenarios',scenarios)
    
#     scenarios.columns =('a','h','s','pv','b')
    
# #     scenarios['s'] = s_append_list 
#     #print('s_append_list',s_append_list)

#     scenarios['scen'] = scenarios['a'].map(str) + '_' + scenarios['h'].map(str) + '_' \
#                     + scenarios['pv'].map(str) + '_'  + scenarios['b'].map(str) + '_'  + scenarios['s'].map(str) 
    
#     scenarios = scenarios[['scen','a','h','pv','b','s']]

#     return scenarios

In [641]:
def scenarios(ahs_scen_list,pv_list,b_list):
    ''' Returns a DataFrame with a column named "Scenarios", creating all possible combinations of the 
    "ID_list", "r_list". "s_list" and "hscen_list" lists. 
    The format is "Building ID_PV capacity_Battery_Heating scenario"
    It also removes the source columns
    :param: df: <DataFrame> unedited Table 2 or 3
    return: df: <DataFrame> Edited Tables with the column "Archetype"'''
    
    #DataFrame with all possible combinations of the given lists
    scenarios = pd.DataFrame(list(product(ahs_scen_list,pv_list,b_list)))
    display('scenarios',scenarios)
    
    a_list = []
    h_list = []
    s_list = []
    
    for ahs in ahs_scen_list:
            a = ahs[:-14]
            s = int(ahs[-6:])
            h = int(ahs[-13:-7])
            #h = int(ahs[:-6])
            #print('a',a,'h',h,'s',s)
            a_list.append(a)
            h_list.append(h)
            s_list.append(s)
            
    scenarios.columns = ['ahs_scen','3','4']
    #print('1',scenarios)
    scenarios['0'] = a_list
    scenarios['1'] = h_list
    scenarios['2'] = s_list
    scenarios = scenarios.drop(['ahs_scen'], axis=1)
    scenarios = scenarios[['0','1','2','3','4']] 
#     display('scenarios',scenarios)
    
    scenarios.columns =('a','h','s','pv','b')
    
#     scenarios['s'] = s_append_list 
    #print('s_append_list',s_append_list)

    scenarios['scen'] = scenarios['a'].map(str) + '_' + scenarios['h'].map(str) + '_' \
                    + scenarios['pv'].map(str) + '_'  + scenarios['b'].map(str) + '_'  + scenarios['s'].map(str) 
    
    scenarios = scenarios[['scen','a','h','pv','b','s']]

    return scenarios

In [642]:
scen_df = scenarios(ahs_scen_list,pv_list,b_list)
scen_list = list(pd.unique(scen_df['scen'].tolist()))
# # print(scen_list)
scen_df

'scenarios'

,0,1,2
0,1030392_100056_100000,PV,LiB
1,1030392_100032_100000,PV,LiB
2,1030392_100128_100000,PV,LiB
3,1030392_100147_100000,PV,LiB
4,1133034_100056_100000,PV,LiB
...,...,...,...
796,1316897_100144_808089,PV,LiB
797,1290392_100128_820463,PV,LiB
798,1290392_100148_820463,PV,LiB
799,1292871_100135_866943,PV,LiB


,scen,a,h,pv,b,s
0,1030392_100056_PV_LiB_100000,1030392,100056,PV,LiB,100000
1,1030392_100032_PV_LiB_100000,1030392,100032,PV,LiB,100000
2,1030392_100128_PV_LiB_100000,1030392,100128,PV,LiB,100000
3,1030392_100147_PV_LiB_100000,1030392,100147,PV,LiB,100000
4,1133034_100056_PV_LiB_100000,1133034,100056,PV,LiB,100000
...,...,...,...,...,...,...
796,1316897_100144_PV_LiB_808089,1316897,100144,PV,LiB,808089
797,1290392_100128_PV_LiB_820463,1290392,100128,PV,LiB,820463
798,1290392_100148_PV_LiB_820463,1290392,100148,PV,LiB,820463
799,1292871_100135_PV_LiB_866943,1292871,100135,PV,LiB,866943


In [643]:
s_list = pd.unique(s_list).tolist()
s_list

[100000,
 100111,
 100112,
 100113,
 100114,
 100115,
 100116,
 100117,
 100118,
 100119,
 100120,
 100121,
 100122,
 204041,
 202418,
 204070,
 207277,
 203349,
 203699,
 206233,
 204401,
 203471,
 202823,
 203398,
 205390,
 204054,
 203535,
 205436,
 207719,
 203530,
 204411,
 205407,
 202505,
 206335,
 204676,
 210575,
 210500,
 204110,
 206329,
 206069,
 205871,
 206389,
 204833,
 203306,
 204216,
 203289,
 204749,
 210293,
 202994,
 203291,
 206092,
 203261,
 202245,
 204121,
 203169,
 204872,
 204746,
 204415,
 203609,
 207073,
 205891,
 203371,
 204653,
 202837,
 204215,
 202436,
 202509,
 212751,
 203345,
 204080,
 203940,
 203317,
 203911,
 207926,
 213820,
 211826,
 203620,
 201991,
 204107,
 214696,
 208750,
 207621,
 205784,
 222674,
 203686,
 208965,
 203125,
 202707,
 202783,
 207803,
 202165,
 207811,
 203882,
 202687,
 209284,
 209525,
 202882,
 212288,
 201286,
 202173,
 204419,
 202689,
 202475,
 201013,
 212932,
 202716,
 202022,
 205115,
 216735,
 816166,
 809675,
 

**GDX data set generation**

In [644]:
def par_df_to_gdx(par_df,par_list):

    par_cols_list = list(par_df)   
    par_name = par_cols_list[0]
    del par_cols_list[0]

    gdx_par_dict = {}
    for par in par_list:
        for par_col in par_cols_list:
            par_tup = (par,par_col)
            par_value = par_df.loc[par_df[par_name]== par, par_col].item()
            gdx_par_dict[par_tup] = par_value

    return gdx_par_dict

def t_df_to_gdx(t_df,t_list):

    a_cols_list = list(t_df) 
    del a_cols_list[-1:]

    gdx_t_dict = {}
    for t in t_list:
        for a in a_cols_list:
            t_tup = (t,a)
            t_value = t_df.loc[t_df['t']== t, a].item()
            gdx_t_dict[t_tup] = t_value

    return gdx_t_dict

def heat_df_to_gdx(heat_df,ah_scen_list):

    d_cols_list = list(heat_df) 
    del d_cols_list[-2:]
    del d_cols_list[0]
    
    gdx_heat_dict = {}
    for ah_scen in ah_scen_list:
#         print(ah_scen)
        for d in d_cols_list:
            a = heat_df.loc[heat_df['heat_scen']== ah_scen, 'a'].item()
            h = heat_df.loc[heat_df['heat_scen']== ah_scen, 'h'].item()
            heat_tup = (d,a,h)
            heat_value = heat_df.loc[heat_df['heat_scen']== ah_scen, d].item()
#             print(heat_tup,d,heat_value)
            gdx_heat_dict[heat_tup] = heat_value
#     print(heat_tup)

    return gdx_heat_dict

In [645]:
# display(heat_dem_th_copy)

In [646]:
# gdx_heat_dem_th_dict = heat_df_to_gdx(heat_df = heat_dem_th_copy,ah_scen_list = ah_scen_list)

In [647]:
# gdx_heat_dem_th_dict[('d1', '1260030', '100083')]

In [648]:
# gdx_dhw_cen_el_dict = heat_df_to_gdx(heat_df = dhw_cen_el_copy,ah_scen_list = ah_scen_list)

In [649]:
# gdx_dhw_cen_el_dict[('t1', '1260030', '100083')]

In [650]:
# ah_scen_list

In [651]:
# heat_dem_th_copy

In [652]:
# display('solar_df',solar_df)
# display('pv_var_df',pv_var_df)
# display('bat_df',bat_df)
# display('heat_par',heat_par)
# display('stes_par',stes_par)
# display('dhw_par',dhw_par)
# display('grid_df',grid_df)
display('price_df',price_df)
# display('elec_dem_df',elec_dem_df)
# display('heat_dem_th_copy',heat_dem_th_copy)
# display('cop_aw35_par',cop_aw35_par)
# display('cop_aw60_par',cop_aw60_par)
# display('dhw_cen_el_copy',dhw_cen_el_copy)
# display('dhw_dec_th_copy',dhw_dec_th_copy)
# display('pv_canton_df',pv_canton_df.head(24))

'price_df'

,t,price_grid,price_supply
0,t1,0.27,0.1
1,t2,0.27,0.1
2,t3,0.27,0.1
3,t4,0.27,0.1
4,t5,0.27,0.1
...,...,...,...
8755,t8756,0.27,0.1
8756,t8757,0.27,0.1
8757,t8758,0.27,0.1
8758,t8759,0.27,0.1


In [653]:
gdx_pv_dict = par_df_to_gdx(par_df = solar_df,par_list = pv_list)

gdx_pv_par_dict = par_df_to_gdx(par_df = pv_var_df,par_list = a_list)

gdx_bat_dict = par_df_to_gdx(par_df = bat_df,par_list = b_list)

gdx_heat_dict = par_df_to_gdx(par_df = heat_par,par_list = h_list)

gdx_stes_dict = par_df_to_gdx(par_df = stes_par,par_list = s_list)

gdx_dhw_dict = par_df_to_gdx(par_df = dhw_par,par_list = a_list)

gdx_grid_dict = par_df_to_gdx(par_df = grid_df,par_list = a_list)

gdx_price_dict = par_df_to_gdx(par_df = price_df,par_list = t_list)

gdx_elec_dem_dict = t_df_to_gdx(t_df = elec_dem_df,t_list = t_list)

# gdx_heat_dem_el_dict = heat_df_to_gdx(heat_df = heat_dem_el_copy,ah_scen_list = ah_scen_list)

gdx_heat_dem_th_dict = heat_df_to_gdx(heat_df = heat_dem_th_copy,ah_scen_list = ah_scen_list)

gdx_cop_35_dict = t_df_to_gdx(t_df = cop_aw35_par,t_list = t_list)

gdx_cop_60_dict = t_df_to_gdx(t_df = cop_aw60_par,t_list = t_list)

gdx_dhw_cen_el_dict = heat_df_to_gdx(heat_df = dhw_cen_el_copy,ah_scen_list = ah_scen_list)

gdx_dhw_dec_th_dict = heat_df_to_gdx(heat_df = dhw_dec_th_copy,ah_scen_list = ah_scen_list)

gdx_pv_cf_dict = t_df_to_gdx(t_df = pv_canton_df,t_list = t_list)

In [654]:
if time_res == 't48':
    if data_scen == 'base_t48':
        out_file = 'base_data_t48.gdx'
    if data_scen == 'stes_t48':
        out_file = 'stes_data_t48.gdx'
if time_res == 't8760':
    if data_scen == 'base_t8760':
        out_file = './optim_test_13_night100.gdx'
    if data_scen == 'stes_t8760':
        #out_file = './optim_test_full_run_seas0.gdx'
        out_file = str(rcp_scenario_year) + '_' + str(rcp_file_name) + '_' + str(retrofit_scen) + '_' + '100_test.gdx'
print(out_file)

2050_rcp26_Low_100_test.gdx


In [655]:
gdxout = gt.gdxrw.gdxWriter(out_file)

#Adding sets
# gdxout.add_set(gamssetname='a', toset=a[a_list], desc=a['a'])
gdxout.add_set_dc(gamssetname= 'a',domain= ['a'], toset= a_list, desc= '')
gdxout.add_set_dc(gamssetname= 'h',domain= ['h'], toset= h_list, desc= '')
gdxout.add_set_dc(gamssetname= 'pv',domain= ['pv'], toset= pv_list, desc= '')
gdxout.add_set_dc(gamssetname= 'b',domain= ['b'], toset= b_list, desc= '')
gdxout.add_set_dc(gamssetname= 's',domain= ['s'], toset= s_list, desc= '')
gdxout.add_set_dc(gamssetname= 'full_scen',domain= ['full_scen'], toset= scen_list, desc= '')

gdx_scen_list = [tuple(scen_tup) for scen_tup in scen_df.to_records(index=False)]
gdxout.add_set_dc(gamssetname= 'map_scenarios',domain = ['scen', 'a', 'h', 'pv', 'b', 's'],
               toset= gdx_scen_list, desc= '')

gdxout.add_parameter_dc(gamsparametername= 'pv_par',domain = ['pv', 'pv_char'],
               toparameter = gdx_pv_dict, desc= '')


gdxout.add_parameter_dc(gamsparametername= 'pv_var_par',domain = ['pv', 'pv_var_char'],
               toparameter = gdx_pv_par_dict, desc= '')

gdxout.add_parameter_dc(gamsparametername= 'batt_par',domain = ['b', 'b_char'],
               toparameter = gdx_bat_dict, desc= '')

gdxout.add_parameter_dc(gamsparametername= 'heat_par',domain = ['h', 'h_char'],
               toparameter = gdx_heat_dict, desc= '')

gdxout.add_parameter_dc(gamsparametername= 'stes_par',domain = ['s', 's_char'],
               toparameter = gdx_stes_dict, desc= '')

gdxout.add_parameter_dc(gamsparametername= 'dhw_par',domain = ['a', 'dhw_char'],
               toparameter = gdx_dhw_dict, desc= '')

gdxout.add_parameter_dc(gamsparametername= 'grid_par',domain = ['a', 'g_char'],
               toparameter = gdx_grid_dict, desc= '')

gdxout.add_parameter_dc(gamsparametername= 'price_el',domain = ['t', 'e_price'],
               toparameter = gdx_price_dict, desc= '')

gdxout.add_parameter_dc(gamsparametername= 'a_dem_el',domain = ['t', 'a'],
               toparameter = gdx_elec_dem_dict, desc= '')

# gdxout.add_parameter_dc(gamsparametername= 'heat_dem_el',domain = ['d', 'a','h'],
#                toparameter = gdx_heat_dem_el_dict, desc= '')

gdxout.add_parameter_dc(gamsparametername= 'heat_dem_th',domain = ['d', 'a','h'],
               toparameter = gdx_heat_dem_th_dict, desc= '')

gdxout.add_parameter_dc(gamsparametername= 'cop_aw35_par',domain = ['t', 'a'],
               toparameter = gdx_cop_35_dict, desc= '')

gdxout.add_parameter_dc(gamsparametername= 'cop_aw60_par',domain = ['t', 'a'],
               toparameter = gdx_cop_60_dict, desc= '')

gdxout.add_parameter_dc(gamsparametername= 'dhw_cen_par_el',domain = ['t', 'a','h'],
               toparameter = gdx_dhw_cen_el_dict, desc= '')

gdxout.add_parameter_dc(gamsparametername= 'dhw_dec_par_th',domain = ['t', 'a','h'],
               toparameter = gdx_dhw_dec_th_dict, desc= '')

gdxout.add_parameter_dc(gamsparametername= 'pv_cf',domain = ['t', 'a'],
               toparameter = gdx_pv_cf_dict, desc= '')


gdxout.export_gdx()

In [ ]:
# #out_file = 'scenarios_dummy_stes_test.gdx'
# with gdxpds.gdx.GdxFile() as gdx:
#     '''Creating the archetype set'''
#     gdx.append(gdxpds.gdx.GdxSymbol('a',gdxpds.gdx.GamsDataType.Set,dims=['a']))
#     data = pd.DataFrame(a_list)
#     data['Value'] = True  
#     gdx[-1].dataframe = data
    
#     '''Creating the electric heating technology set'''
#     gdx.append(gdxpds.gdx.GdxSymbol('h',gdxpds.gdx.GamsDataType.Set,dims=['h']))
#     data = pd.DataFrame(h_list)
#     data['Value'] = True
#     gdx[-1].dataframe = data
    
#     '''Creating the renewable technologies set'''
#     gdx.append(gdxpds.gdx.GdxSymbol('pv',gdxpds.gdx.GamsDataType.Set,dims=['pv']))
#     data = pd.DataFrame(pv_list)
#     data['Value'] = True
#     gdx[-1].dataframe = data
    
#     '''Creating the electric storage technologies set'''
#     gdx.append(gdxpds.gdx.GdxSymbol('b',gdxpds.gdx.GamsDataType.Set,dims=['b']))
#     data = pd.DataFrame(b_list)
#     data['Value'] = True
#     gdx[-1].dataframe = data
    
#     '''Creating the stes technology set'''
#     gdx.append(gdxpds.gdx.GdxSymbol('s',gdxpds.gdx.GamsDataType.Set,dims=['s']))
#     data = pd.DataFrame(s_list)
#     data['Value'] = True
#     gdx[-1].dataframe = data
    
    
#     '''Creating the scenario set'''
#     gdx.append(gdxpds.gdx.GdxSymbol('full_scen',gdxpds.gdx.GamsDataType.Set,dims=['full_scen']))
#     data = pd.DataFrame(scen_list)
#     data['Value'] = True
#     gdx[-1].dataframe = data
    
#     '''Creating the scenario mapping for all combinations set'''
#     gdx.append(gdxpds.gdx.GdxSymbol('map_scenarios',gdxpds.gdx.GamsDataType.Set,\
#                                     dims=['scen','a','h','pv','b','s']))
#     data = scen_df
#     data['Value'] = True
#     gdx[-1].dataframe = data
        
    
    
#     '''Creating the PV energy parameters''' 
#     for pv in pv_list:
#         #Creates a list of elements to be added to the parameter table. The list is then appended to the main list 

#         solar_list.append([str(pv),'pv_pel',solar_df.loc[solar_df['pv']== pv, 'pv_pel'].item()])
        
#     gdx.append(gdxpds.gdx.GdxSymbol('pv_par', gdxpds.gdx.GamsDataType.Parameter, dims=['pv','pv_char']))
    
#     data = pd.DataFrame(solar_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
                         
#     gdx[-1].dataframe = data
       
    
#     '''Creating the variable PV energy parameters'''
#     for a in a_list: 
#         #Creates a list of elements to be added to the parameter table. The list is then appended to the main list    
#         solar_var_list.append([str(a),'roof_era',pv_var_df.loc[pv_var_df['arch_entry_id']== a, 'roof_era'].item()])
#         solar_var_list.append([str(a),'pv_pel_max_m2',pv_var_df.loc[pv_var_df['arch_entry_id']== a, 'pv_pel_max_m2'].item()])

        
#     gdx.append(gdxpds.gdx.GdxSymbol('pv_var_par', gdxpds.gdx.GamsDataType.Parameter, dims=['a','pv_var_char']))
    
#     data = pd.DataFrame(solar_var_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
                         
#     gdx[-1].dataframe = data
    
#     '''Creating the battery parameters'''
#     for b in b_list:
#         #Creates a list of elements to be added to the parameter table. The list is then appended to the main list 

#         bat_list.append([str(b),'bat_soc_min',bat_df.loc[bat_df["b"]== b, 'bat_soc_min'].item()])
#         bat_list.append([str(b),'bat_disch_max',bat_df.loc[bat_df["b"]== b, 'bat_disch_max'].item()])
#         bat_list.append([str(b),'bat_ch_max',bat_df.loc[bat_df["b"]== b, 'bat_ch_max'].item()])
#         bat_list.append([str(b),'bat_eff',bat_df.loc[bat_df["b"]== b, 'bat_eff'].item()])
#         bat_list.append([str(b),'bat_st_cap ',bat_df.loc[bat_df["b"]== b, 'bat_st_cap '].item()])
#         bat_list.append([str(b),'bat_pw_tr',bat_df.loc[bat_df["b"]== b, 'bat_pw_tr'].item()])

        
#     gdx.append(gdxpds.gdx.GdxSymbol('batt_par', gdxpds.gdx.GamsDataType.Parameter, dims=['b','b_char']))
    
#     data = pd.DataFrame(bat_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
                         
#     gdx[-1].dataframe = data

#     #Creating the heating technology parameters
#     for h in h_list: 
#         #Creates a list of elements to be added to the parameter table. The list is then appended to the main list    
#         heat_par_list.append([str(h),'th_pel_hp',heat_par.loc[heat_par['h']== h, 'th_pel_hp'].item()])
#         heat_par_list.append([str(h),'th_efficiency',heat_par.loc[heat_par['h']== h, 'th_efficiency'].item()])
           
#     gdx.append(gdxpds.gdx.GdxSymbol('heat_par', gdxpds.gdx.GamsDataType.Parameter, dims=['h','h_char']))
    
#     data = pd.DataFrame(heat_par_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
                         
#     gdx[-1].dataframe = data 
    
#     #Creating the stes technology parameters
#     for s in s_list: 
#         #Creates a list of elements to be added to the parameter table. The list is then appended to the main list    
#         stes_par_list.append([str(s),'th_stes_volume',stes_par.loc[stes_par['s']== s, 'th_stes_volume'].item()])
#         stes_par_list.append([str(s),'th_stes_relativedailyloss',stes_par.loc[stes_par['s']== s, 'th_stes_relativedailyloss'].item()])
#         stes_par_list.append([str(s),'th_stes_placedinsidehouse',stes_par.loc[stes_par['s']== s, 'th_stes_placedinsidehouse'].item()])
#         stes_par_list.append([str(s),'th_stes_ambienttemperature',stes_par.loc[stes_par['s']== s, 'th_stes_ambienttemperature'].item()])
#         stes_par_list.append([str(s),'stes_loss_tot_th',stes_par.loc[stes_par['s']== s, 'stes_loss_tot_th'].item()])
#         stes_par_list.append([str(s),'stes_loss_tot_e1_th',stes_par.loc[stes_par['s']== s, 'stes_loss_tot_e1_th'].item()])
#         stes_par_list.append([str(s),'stes_loss_tot_e2_th',stes_par.loc[stes_par['s']== s, 'stes_loss_tot_e2_th'].item()])
#         stes_par_list.append([str(s),'stes_loss_tot_e3_th',stes_par.loc[stes_par['s']== s, 'stes_loss_tot_e3_th'].item()])
#         stes_par_list.append([str(s),'th_stes_tmax',stes_par.loc[stes_par['s']== s, 'th_stes_tmax'].item()])
#         stes_par_list.append([str(s),'th_stes_dtmax',stes_par.loc[stes_par['s']== s, 'th_stes_dtmax'].item()])
#         stes_par_list.append([str(s),'th_stes_emaxtot',stes_par.loc[stes_par['s']== s, 'th_stes_emaxtot'].item()])
#         stes_par_list.append([str(s),'th_stes_emax1',stes_par.loc[stes_par['s']== s, 'th_stes_emax1'].item()])
#         stes_par_list.append([str(s),'th_stes_emax2',stes_par.loc[stes_par['s']== s, 'th_stes_emax2'].item()])
#         stes_par_list.append([str(s),'th_stes_emax3',stes_par.loc[stes_par['s']== s, 'th_stes_emax3'].item()])
#         #stes_par_list.append([str(s),'th_stes_chargeefficiency1',stes_par.loc[stes_par['s']== s, 'th_stes_chargeefficiency1'].item()])
#         #stes_par_list.append([str(s),'th_stes_chargeefficiency2',stes_par.loc[stes_par['s']== s, 'th_stes_chargeefficiency2'].item()])
#         stes_par_list.append([str(s),'th_stes_chargeefficiency3',stes_par.loc[stes_par['s']== s, 'th_stes_chargeefficiency3'].item()])
#         stes_par_list.append([str(s),'th_stes_pmaxcharge3',stes_par.loc[stes_par['s']== s, 'th_stes_pmaxcharge3'].item()])
#         stes_par_list.append([str(s),'th_stes_socmin',stes_par.loc[stes_par['s']== s, 'th_stes_socmin'].item()])
           
#     gdx.append(gdxpds.gdx.GdxSymbol('stes_par', gdxpds.gdx.GamsDataType.Parameter, dims=['s','s_char']))
    
#     data = pd.DataFrame(stes_par_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
                         
#     gdx[-1].dataframe = data  
    
# #     #Creating the heating technology parameters
#     for a in a_list: 
#         #Creates a list of elements to be added to the parameter table. The list is then appended to the main list    
#         dhw_par_list.append([str(a),'dhw_soc_min',dhw_par.loc[dhw_par['a']== a, 'dhw_soc_min'].item()])
#         dhw_par_list.append([str(a),'dhw_eff',dhw_par.loc[dhw_par['a']== a, 'dhw_eff'].item()])
#         dhw_par_list.append([str(a),'dhw_pel',dhw_par.loc[dhw_par['a']== a, 'dhw_pel'].item()])
#         dhw_par_list.append([str(a),'dhw_st_cap',dhw_par.loc[dhw_par['a']== a, 'dhw_st_cap'].item()])
#         dhw_par_list.append([str(a),'dhw_decentral',dhw_par.loc[dhw_par['a']== a, 'dhw_decentral'].item()])
        
#     gdx.append(gdxpds.gdx.GdxSymbol('dhw_par', gdxpds.gdx.GamsDataType.Parameter, dims=['a','dhw_char']))
    
#     data = pd.DataFrame(dhw_par_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
                         
#     gdx[-1].dataframe = data
    
#     #Creating the grid constraints parameters
#     for a in a_list:
#         #Creates a list of elements to be added to the parameter table. The list is then appended to the main list 
#         grid_list.append([str(a),'grid_max_dem',grid_df.loc[grid_df['a']== a, 'grid_max_dem'].item()])
#         grid_list.append([str(a),'grid_max_sup',grid_df.loc[grid_df['a']== a, 'grid_max_sup'].item()])
        
#     gdx.append(gdxpds.gdx.GdxSymbol('grid_par', gdxpds.gdx.GamsDataType.Parameter, dims=['a','g_char']))
    
#     data = pd.DataFrame(grid_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
                         
#     gdx[-1].dataframe = data
    
    
#     '''Creating the electricity price parameters''' 
#     for t in t_list:
#         #Creates a list of elements to be added to the parameter table. The list is then appended to the main list 
#         price_list.append([str(t),'price_grid',price_df.loc[price_df['t']== t, 'price_grid'].item()])
#         price_list.append([str(t),'price_supply',price_df.loc[price_df['t']== t, 'price_supply'].item()])
    
#     gdx.append(gdxpds.gdx.GdxSymbol('price_el', gdxpds.gdx.GamsDataType.Parameter, dims=['t','e_price']))
    
#     data = pd.DataFrame(price_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
                         
#     gdx[-1].dataframe = data
    
#     '''Creating the electricity demand parameters''' 
    
#     for t in t_list:
#         #Creates a list of elements to be added to the parameter table. The list is then appended to the main list 
#         for a in a_list:
#              elec_dem_list.append([str(t),a,elec_dem_df.loc[elec_dem_df['t']== t,a].item()])
                
#     gdx.append(gdxpds.gdx.GdxSymbol('a_dem_el', gdxpds.gdx.GamsDataType.Parameter, dims=['t','a']))
    
#     data = pd.DataFrame(elec_dem_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
                         
#     gdx[-1].dataframe = data
    
    
#     for heat_scen in ah_scen_list:
       
#         for d in d_list:

#             heat_dem_el_list.append([str(d),\
#                                  heat_dem_el_copy.loc[heat_dem_el_copy['heat_scen']== heat_scen,'a'].item(),\
#                                  heat_dem_el_copy.loc[heat_dem_el_copy['heat_scen']== heat_scen,'h'].item(),\
#                                  heat_dem_el_copy.loc[heat_dem_el_copy['heat_scen']== heat_scen,d].item()])
            
                
#     gdx.append(gdxpds.gdx.GdxSymbol('heat_dem_el', gdxpds.gdx.GamsDataType.Parameter, \
#                                     dims=['d','a','h']))
    
#     data = pd.DataFrame(heat_dem_el_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
    
#     gdx[-1].dataframe = data
    
    
#     for heat_scen in ah_scen_list:
       
#         for d in d_list:

#             heat_dem_th_list.append([str(d),\
#                                  heat_dem_th_copy.loc[heat_dem_th_copy['heat_scen']== heat_scen,'a'].item(),\
#                                  heat_dem_th_copy.loc[heat_dem_th_copy['heat_scen']== heat_scen,'h'].item(),\
#                                  heat_dem_th_copy.loc[heat_dem_th_copy['heat_scen']== heat_scen,d].item()])
            
                
#     gdx.append(gdxpds.gdx.GdxSymbol('heat_dem_th', gdxpds.gdx.GamsDataType.Parameter, \
#                                     dims=['d','a','h']))
    
#     data = pd.DataFrame(heat_dem_th_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
    
#     gdx[-1].dataframe = data
        
    
    
#     '''Creating the COP AW35 demand parameters'''
#     for t in t_list:
#         #Creates a list of elements to be added to the parameter table. The list is then appended to the main list 
#         for a in a_list:
#              cop_aw35_list.append([str(t),a,cop_aw35_par.loc[cop_aw35_par['t']== t,a].item()])
                
#     gdx.append(gdxpds.gdx.GdxSymbol('cop_aw35_par', gdxpds.gdx.GamsDataType.Parameter, dims=['t','a']))
    
#     data = pd.DataFrame(cop_aw35_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
                         
#     gdx[-1].dataframe = data
    
#     '''Creating the COP AW60 demand parameters'''
#     for t in t_list:
#         #Creates a list of elements to be added to the parameter table. The list is then appended to the main list 
#         for a in a_list:
#              cop_aw60_list.append([str(t),a,cop_aw60_par.loc[cop_aw60_par['t']== t,a].item()])
                
#     gdx.append(gdxpds.gdx.GdxSymbol('cop_aw60_par', gdxpds.gdx.GamsDataType.Parameter, dims=['t','a']))
    
#     data = pd.DataFrame(cop_aw60_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
                         
#     gdx[-1].dataframe = data
     
    
#     '''Creating the centralized DHW parameters'''

#     for dhw_scen in ah_scen_list:
       
#         for t in t_list:

#             dhw_cen_list.append([str(t),\
#                                  dhw_cen_el_copy.loc[dhw_cen_el_copy['dhw_scen']== dhw_scen,'a'].item(),\
#                                  dhw_cen_el_copy.loc[dhw_cen_el_copy['dhw_scen']== dhw_scen,'h'].item(),\
#                                  dhw_cen_el_copy.loc[dhw_cen_el_copy['dhw_scen']== dhw_scen,t].item()])
            
                
#     gdx.append(gdxpds.gdx.GdxSymbol('dhw_cen_par_el', gdxpds.gdx.GamsDataType.Parameter, \
#                                     dims=['t','a','h']))
    
#     data = pd.DataFrame(dhw_cen_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
    
#     gdx[-1].dataframe = data
    
#     '''Creating the decentralized DHW parameters'''
     
#     for dhw_scen in ah_scen_list:
        
#         for t in t_list:
            
#             dhw_dec_list.append([str(t),\
#                                  dhw_dec_th_copy.loc[dhw_dec_th_copy['dhw_scen']== dhw_scen,'a'].item(),\
#                                  dhw_dec_th_copy.loc[dhw_dec_th_copy['dhw_scen']== dhw_scen,'h'].item(),\
#                                  dhw_dec_th_copy.loc[dhw_dec_th_copy['dhw_scen']== dhw_scen,t].item()])
            
                
#     gdx.append(gdxpds.gdx.GdxSymbol('dhw_dec_par_th', gdxpds.gdx.GamsDataType.Parameter, \
#                                     dims=['t','a','h']))
    
#     data = pd.DataFrame(dhw_dec_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
    
#     gdx[-1].dataframe = data
       
       
#     '''Creating the parameter for solar's capacity factor'''
#     for t in t_list:
#         #Creates a list of elements to be added to the parameter table. The list is then appended to the main list 
#         for a in a_list:
#             pvcf_list.append([str(t),a,pv_canton_df.loc[pv_canton_df['t']== t, a].item()])
    
#     gdx.append(gdxpds.gdx.GdxSymbol('pv_cf', gdxpds.gdx.GamsDataType.Parameter, dims=['t','a']))
    
#     data = pd.DataFrame(pvcf_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
                         
#     gdx[-1].dataframe = data     
    
# #     for t in t_list:
# #         #Creates a list of elements to be added to the parameter table. The list is then appended to the main list 
# #         for a in a_list:
# #              elec_dem_list.append([str(t),a,elec_dem_df.loc[elec_dem_df['t']== t,a].item()])
                
# #     gdx.append(gdxpds.gdx.GdxSymbol('a_dem_el', gdxpds.gdx.GamsDataType.Parameter, dims=['t','a']))
    
# #     data = pd.DataFrame(elec_dem_list, columns=(gdx[-1].dims + gdx[-1].value_col_names))
                         
# #     gdx[-1].dataframe = data
    

#     gdx.write(out_file)
# print("FINISHED")

**TCO calculation**

In [ ]:
dir_results = 'C:/Users/xigeby65/switchdrive/SwissStore/Model/Results/10mfh_plots'

In [ ]:
data_results_list = os.listdir(dir_results)

In [ ]:
data_results_list

In [ ]:
# build_typ_widget = widgets.Dropdown(
#     options = ['SFH','MFH'],
#     description = 'build_typ ',
#     disabled = False)
# display(build_typ_widget)

In [ ]:
# build_typ = build_typ_widget.value

In [ ]:
# for data_results in data_results_list:
#     build_typ_scen = data_results[-7:-4]
#     if build_typ_scen != build_typ:
#         data_results_list.remove(data_results)   

In [ ]:
# data_results_list

In [ ]:
data_results_widget = widgets.Dropdown(
    options = data_results_list,
    description = 'Data set ',
    disabled = False)
display(data_results_widget)

In [ ]:
data_results = data_results_widget.value
data_results

In [ ]:
data_results_path = dir_results + '/' + data_results
print(data_results_path)

In [ ]:
# '''Importing the energy cost data from the model's GDX results file'''

# elec_cost_orig = gdxpds.to_dataframes(data_results_path).get('report_cost') #Getting a specific DataFrame from the dictionary
# elec_cost_orig.rename(columns = {'Value':'elec_cost'},inplace = True) #Renaming the cost category column
# del elec_cost_orig['*']
# elec_cost_orig = elec_cost_orig.round(3) #rounding the values to 3 decimals
# elec_cost_orig

In [ ]:
gdxin = gt.gdxrw.gdxReader(data_results_path)

In [ ]:
gdx_stes_cost_inst = gdxin.rgdx(name='report_cost')
# gdx_stes_grid_spec = gdxin.rgdx(name='report_grid_demand')
gdx_stes_cost_inst

In [ ]:
'''Importing the data from the model's GDX file solution'''

gdx_stes_cost = pd.DataFrame(data=gdx_stes_cost_inst['values'].keys(), 
                             columns=gdx_stes_cost_inst['domain']) #Getting a specific DataFrame from the dictionary
gdx_stes_cost['elec_cost'] = gdx_stes_cost_inst['values'].values() #Renaming the cost category column
del gdx_stes_cost['*']
gdx_stes_cost = gdx_stes_cost.round(3) #rounding the values to 3 decimals
display(gdx_stes_cost)
elec_cost_orig = gdx_stes_cost.copy()

In [ ]:
gdx_stes_pv_inst = gdxin.rgdx(name='report_pv_pel')
# gdx_stes_grid_spec = gdxin.rgdx(name='report_grid_demand')
gdx_stes_pv_inst

In [ ]:
'''Importing the data from the model's GDX file solution'''

gdx_stes_pv = pd.DataFrame(data=gdx_stes_pv_inst['values'].keys(), 
                             columns=gdx_stes_pv_inst['domain']) #Getting a specific DataFrame from the dictionary
gdx_stes_pv['pv_pel'] = gdx_stes_pv_inst['values'].values() #Renaming the cost category column
del gdx_stes_pv['*']
gdx_stes_cost = gdx_stes_pv.round(3) #rounding the values to 3 decimals
display(gdx_stes_pv)
pv_pel_df = gdx_stes_pv.copy()

In [ ]:
full_scen_list = list(pd.unique(elec_cost_orig['full_scen']))
full_scen_list

In [ ]:
gdx_stes_grid_inst = gdxin.rgdx(name='report_grid_tot')
# gdx_stes_grid_spec = gdxin.rgdx(name='report_grid_demand')
gdx_stes_grid_inst

In [ ]:
'''Importing the data from the model's GDX file solution'''

gdx_stes_grid = pd.DataFrame(data=gdx_stes_grid_inst['values'].keys(), 
                             columns=gdx_stes_grid_inst['domain']) #Getting a specific DataFrame from the dictionary
gdx_stes_grid['val'] = gdx_stes_grid_inst['values'].values() #Renaming the cost category column
#del gdx_stes_pv['*']
gdx_stes_grid = gdx_stes_grid.round(3)#rounding the values to 3 decimals
gdx_stes_grid.rename(columns = {'*':'var'},inplace = True) 

gdx_df = pd.DataFrame()

for scen in full_scen_list:

    gdx_scen = gdx_stes_grid.copy()
    gdx_scen = gdx_scen[gdx_scen.full_scen == scen].drop(['full_scen'], axis=1).set_index('var').T
    gdx_scen['full_scen'] = scen
    
    gdx_scen = gdx_scen[['full_scen','pv_prod_tot','sum_grid_dem_tot']].reset_index(drop=True)

    gdx_df = pd.concat([gdx_df, gdx_scen], ignore_index = True)
        
#gdx_df['t'] = t_graph_list
gdx_stes_grid = gdx_df.copy()


display(gdx_stes_grid)

In [ ]:
arch_list = []
for full_scen in full_scen_list:
    arch = full_scen[:7]
    arch_list.append(arch)

In [ ]:
elec_cost_df = elec_cost_orig.copy()
scen_results_list = []
stes_results_list = []
stes_capex_list = []
stes_vol_list = []
total_elec_cost_list = []

for full_scen in full_scen_list:
    scen_name = full_scen[:14]
    stes_name = int(full_scen[-6:])
    stes_capex = table_4.loc[table_4['th_syssolution_id'] == 'th_capex_f', stes_name].item()
    stes_vol = table_4.loc[table_4['th_syssolution_id'] == 'th_stes_volume', stes_name].item()
    elec_cost = elec_cost_df.loc[elec_cost_df['full_scen'] == full_scen, 'elec_cost'].item()
    total_elec_cost = (elec_cost*discount_df).iloc[0].sum()
    
    scen_results_list.append(scen_name)
    stes_results_list.append(stes_name)
    stes_capex_list.append(stes_capex)
    stes_vol_list.append(stes_vol)
    total_elec_cost_list.append(total_elec_cost)
    
elec_cost_df['scenario'] = scen_results_list
elec_cost_df['stes'] = stes_results_list
elec_cost_df['stes_vol'] = stes_vol_list
elec_cost_df['stes_capex']=stes_capex_list
elec_cost_df['total_elec_cost']=total_elec_cost_list
elec_cost_df = elec_cost_df.drop(columns = ['elec_cost'])

elec_cost_df = pd.merge(elec_cost_df,gdx_stes_grid,on='full_scen')

elec_cost_df

In [ ]:
tco_df = pd.concat([tco_fuel_df,tco_elec_df],sort = False)
tco_df = pd.merge(elec_cost_df,tco_df,on='scenario')
#tco_df = tco_df.drop(columns = ['full_scen','scenario','stes','replacement','total_cost','energy'])
#tco_df = tco_df[['stes_vol','capex','opex','stes_capex','total_elec_cost','fuel','retrofits']]
tco_df = tco_df.drop(columns = ['energy','total_cost'])
tco_df['arch_entry_id'] = arch_list
#tco_df = pd.merge(tco_df,pv_pel_df,on='arch_entry_id')
tco_df

In [ ]:
pv_pel_df

In [ ]:
pv_capex_kWp = 2156.13
pv_opex_kWp = 94.38

pv_capex_list = []
pv_opex_list = []

pv_pel_list = list((pv_pel_df['pv_pel']))

for pv_cap in pv_pel_list:
    pv_capex = pv_cap*pv_capex_kWp
    pv_opex = ((pv_cap*pv_opex_kWp)*discount_df).iloc[0].sum()
    
    pv_capex_list.append(pv_capex)
    pv_opex_list.append(pv_opex)

pv_pel_df['pv_capex'] = pv_capex_list
pv_pel_df['pv_opex'] = pv_opex_list

pv_pel_df

In [ ]:
# pv_cap = 32.715
# pv_capex_kWp = 2541.5
# pv_opex_kWp = 70

# pv_capex = pv_cap*pv_capex_kWp
# pv_opex = ((pv_cap*pv_opex_kWp)*discount_df).iloc[0].sum()

# tco_df['pv_capex'] = pv_capex
# tco_df['pv_opex'] = pv_opex 

tco_df = pd.merge(tco_df,pv_pel_df,on='full_scen')
tco_df = tco_df.fillna(0).rename(columns={'capex':'ht_capex','opex':'ht_opex',\
                                          'pv_prod_tot':'pv_prod_yearly',\
                                          'sum_grid_dem_tot':'grid_dem_yearly'}).drop([13])
tco_df.stes = tco_df.stes.astype(str)
tco_df['pv_opex + elec_cost'] = tco_df['pv_opex'] + tco_df['total_elec_cost']
tco_df = tco_df.drop(columns=['full_scen','total_elec_cost','arch_entry_id','pv_pel','pv_opex'])
tco_df

In [ ]:
import os
  
out_csv = 'tco_pvthird.csv' 
   
#dir_csv = dir_parsed + '/' + arch 
    
dir_csv = './10MFH/'
    
#dir_cv = 'C:/Users/xigeby65/switchdrive/SwissStore/Model/Results/plots/10MFH/PVmax'
print(dir_csv)
    
if not os.path.exists(dir_csv):
    os.mkdir(dir_csv)
        
full_dir_csv = os.path.join(dir_csv,out_csv)
    
print(full_dir_csv)
    
tco_df.to_csv(full_dir_csv, sep = ',', index = False)



In [ ]:
gdx_stes_tot_inst = gdxin.rgdx(name='report_stes_tot')
gdx_stes_tot_inst

In [ ]:
'''Importing the data from the model's GDX file solution'''

gdx_stes_tot = pd.DataFrame(data=gdx_stes_tot_inst['values'].keys(), 
                             columns=gdx_stes_tot_inst['domain']) #Getting a specific DataFrame from the dictionary
gdx_stes_tot['val'] = gdx_stes_tot_inst['values'].values() #Renaming the cost category column
#del gdx_stes_pv['*']
gdx_stes_tot = gdx_stes_tot.round(3)#rounding the values to 3 decimals
gdx_stes_tot.rename(columns = {'*':'var'},inplace = True) 

gdx_df = pd.DataFrame()

scen_name_list = []
stes_name_list = []

for scen in full_scen_list:
    
    scen_name = scen[:14]
    stes_name = int(scen[-6:])
    
    scen_name_list.append(scen_name)
    stes_name_list.append(stes_name)

    gdx_scen = gdx_stes_tot.copy()
    gdx_scen = gdx_scen[gdx_scen.full_scen == scen].drop(['full_scen'], axis=1).set_index('var').T
    gdx_scen['full_scen'] = scen
    
    gdx_scen = gdx_scen[['full_scen','stes_yearly_disch__th','stes_yearly_ch_th','stes_yearly_ch_el','hp_dir_yearly_th','hp_dir_yearly_el']].reset_index(drop=True)

    gdx_df = pd.concat([gdx_df, gdx_scen], ignore_index = True)

gdx_df['scen'] = scen_name_list
gdx_df['stes'] = stes_name_list
gdx_df = gdx_df.drop(columns=['full_scen'])  
gdx_df = gdx_df[['scen','stes','stes_yearly_disch__th','stes_yearly_ch_th','stes_yearly_ch_el','hp_dir_yearly_th','hp_dir_yearly_el']]
#gdx_df['t'] = t_graph_list
gdx_stes_tot = gdx_df.copy()


display(gdx_stes_tot)

In [ ]:
import os
  
out_csv = 'stes_pvthird.csv' 
   
#dir_csv = dir_parsed + '/' + arch 
    
dir_csv = './10MFH/'
    
#dir_cv = 'C:/Users/xigeby65/switchdrive/SwissStore/Model/Results/plots/10MFH/PVmax'
print(dir_csv)
    
if not os.path.exists(dir_csv):
    os.mkdir(dir_csv)
        
full_dir_csv = os.path.join(dir_csv,out_csv)
    
print(full_dir_csv)
    
gdx_stes_tot.to_csv(full_dir_csv, sep = ',', index = False)

In [ ]:
mi_lista = []

for i in range (0,20):
    mi_lista.append(i)

print(mi_lista)

In [ ]:
#header = 'TCO for different heating and STES (MFH & PVmax & 1T_FiT = 0,05 CHF/kWh)' 
#header = 'TCO for different heating and STES (MFH & PVmax & 1T_FiT = 0,2 CHF/kWh)' 
#header = 'TCO for different heating and STES (MFH & PVmax & 2T_FiT = 0,2 CHF/kWh)' 
header = 'TCO for different heating and STES (MFH & PVmax & 2T_FiT = 0,05 CHF/kWh)' 
        
title = header 
        
#Defining plot style (https://seaborn.pydata.org/tutorial/aesthetics.html)
sns.set(rc={'figure.figsize':(15,10)}, palette = 'muted', style = 'white')
#Defining axes and title font size
sns.set_context("notebook", font_scale = 2)
        
#Plotting
plt.figure() #New plot on each loop
        
tco_plot = tco_df.transpose().T.plot(kind='bar', stacked=True).set_title(title)

#Legend configuration (loc and bbox_to_anchor work together to locate the legend) 
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize = 'small')
#Rotates the x axis labels in case they overlap
axes = plt.gca()
axes.set_ylim([0,450000])
plt.xticks(rotation=0)
#plt.xticks(np.arange(0,hours_tot,step = 5))
plt.xlabel('STES Volume')
plt.ylabel('TCO_CHF')

In [ ]:
weeks_list = [8,10,35,49]

for grid_dem_scen, grid_bal_df in gdx_stes_grid_dict.items():
    
     #'bat_ch'  'bat_disch'  'pv_sup'  'heat_hp_el'  'stes_ch_tot_hp_el'  'stes_ch_e3_el'
    
    #'dhw_cen_dem_el'  'elec_dem'  'dhw_ch_el'  'grid_dem_tot'
    
    #grid_week_df = grid_bal_df.copy().drop(columns = ['bat_ch','bat_disch','pv_sup','dhw_cen_dem_el','grid_dem_tot'])
    grid_week_df = grid_bal_df.copy().drop(columns = ['bat_ch','bat_disch','dhw_cen_dem_el','grid_dem_tot'])
    
    grid_week_df = grid_week_df[['elec_dem','pv_sup','dhw_ch_el','heat_hp_el','stes_ch_tot_hp_el','stes_ch_e3_el']]
    
    for week in weeks_list:
        
        day_end = week*168
        day_init = day_end - 48
        
        week_df = grid_week_df.loc[day_init:day_end]
        hours_tot = day_end - day_init
        
        header = 'grid demand week' 
        
        title = header + ' ' + str(week)
        
        #Defining plot style (https://seaborn.pydata.org/tutorial/aesthetics.html)
        sns.set(rc={'figure.figsize':(15,10)}, palette = 'muted', style = 'white')
        #Defining axes and title font size
        sns.set_context("notebook", font_scale = 2)
        
        #Plotting
        plt.figure() #New plot on each loop
        
        week_plot = week_df.transpose().T.plot(kind='bar', stacked=True).set_title(title)

        #Legend configuration (loc and bbox_to_anchor work together to locate the legend) 
        plt.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize = 'small')
        #Rotates the x axis labels in case they overlap
        plt.xticks(rotation=0)
        plt.xticks(np.arange(0,hours_tot,step = 5))
        plt.xlabel("hours")
        plt.ylabel("kWh")
        

In [ ]:
for file_name in data_results_list:

        stes_scen_cost_df = gdxpds.to_dataframes(dir_results).get('report_cost') #Getting a specific DataFrame from the dictionary
        stes_scen_cost_df.rename(columns = {'Value':'elec_cost'},inplace = True) #Renaming the cost category column
        del stes_scen_cost_df['*']
        stes_scen_cost_df = stes_scen_cost_df.round(3)
        
        stes_full_scen = stes_scen_cost_df.iloc[0,0]
        s = stes_full_scen[-6:]
        ah = stes_full_scen[:-16]

        stes_scen_name = '1213068_100070_' + pvscen +  '_BAT0_' + s
        #print(stes_full_scen,stes_scen_name)
                                            
        stes_scen_cost_df = stes_scen_cost_df.replace(stes_full_scen,stes_scen_name)
        
        elec_cost = stes_scen_cost_df.loc[stes_scen_cost_df['full_scen'] == stes_scen_name, 'elec_cost'].item()
        
        elec_cost_df = discount_df * elec_cost
                  
        elec_cost_final  = elec_cost_df.iloc[0].sum()
            
        stes_scen_cost_df['total_elec_cost'] = elec_cost_final
        
        th_capex_f = table_4.loc[table_4['th_syssolution_id'] == 'th_capex_f', int(s)].item()
        stes_scen_cost_df['stes_capex'] = th_capex_f
        
        cost_df = tco_elec_df.loc[tco_elec_df['scenario'] == ah].drop(columns = ['replacement','fuel','retrofits','total_cost'])
        stes_scen_cost_df ['capex'] = cost_df.loc[cost_df['scenario'] == ah, 'capex'].item()
        stes_scen_cost_df ['opex'] = cost_df.loc[cost_df['scenario'] == ah, 'opex'].item()
        
#         pv_pel = gdxpds.to_dataframes(stes_scen_path).get('pv_pel')
#         print('pv_pel',pv_pel)
#         pv_capex = pv_pel*2223.625
#         #pv_opex_df = discount_df * pv_pel * 75
#         #pv_opex_final  = pv_opex_df.iloc[0].sum()
#         stes_scen_cost_df ['pv_capex'] = pv_capex
        #stes_scen_cost_df ['pv_opex'] = pv_opex_final
        
        #stes_scen_cost_df ['total_opex'] = stes_scen_cost_df['opex'] + stes_scen_cost_df['pv_opex']
#         stes_scen_cost_df ['total_capex'] = stes_scen_cost_df['capex'] + stes_scen_cost_df['pv_capex']+ stes_scen_cost_df['stes_capex']
        
        print(stes_scen_cost_df)
        
        
        
        stes_dict_key = share + '%'
        stes_scen_dict[stes_dict_key] = stes_scen_cost_df

print(stes_scen_dict)

In [ ]:
share_list = ['10','20','50','80','100']
pvscen_list = ['PV0','PV10','PVmax']
stes_scen_cost_df = pd.DataFrame()
stes_scen_dict = {}

In [ ]:
for share in share_list:
    for pvscen in pvscen_list:
        file_name = share + '_' + 'test' + '_' + pvscen + '_results.gdx'
        stes_scen_path = 'C:/Users/hramo_000/Desktop/Model/Results/' + file_name
        stes_scen_cost_df = gdxpds.to_dataframes(stes_scen_path).get('report_cost') #Getting a specific DataFrame from the dictionary
        stes_scen_cost_df.rename(columns = {'Value':'elec_cost'},inplace = True) #Renaming the cost category column
        del stes_scen_cost_df['*']
        stes_scen_cost_df = stes_scen_cost_df.round(3)
        
        stes_full_scen = stes_scen_cost_df.iloc[0,0]
        s = stes_full_scen[-6:]
        ah = stes_full_scen[:-16]

        stes_scen_name = '1213068_100070_' + pvscen +  '_BAT0_' + s
        #print(stes_full_scen,stes_scen_name)
                                            
        stes_scen_cost_df = stes_scen_cost_df.replace(stes_full_scen,stes_scen_name)
        
        elec_cost = stes_scen_cost_df.loc[stes_scen_cost_df['full_scen'] == stes_scen_name, 'elec_cost'].item()
        
        elec_cost_df = discount_df * elec_cost
                  
        elec_cost_final  = elec_cost_df.iloc[0].sum()
            
        stes_scen_cost_df['total_elec_cost'] = elec_cost_final
        
        th_capex_f = table_4.loc[table_4['th_syssolution_id'] == 'th_capex_f', int(s)].item()
        stes_scen_cost_df['stes_capex'] = th_capex_f
        
        cost_df = tco_elec_df.loc[tco_elec_df['scenario'] == ah].drop(columns = ['replacement','fuel','retrofits','total_cost'])
        stes_scen_cost_df ['capex'] = cost_df.loc[cost_df['scenario'] == ah, 'capex'].item()
        stes_scen_cost_df ['opex'] = cost_df.loc[cost_df['scenario'] == ah, 'opex'].item()
        
#         pv_pel = gdxpds.to_dataframes(stes_scen_path).get('pv_pel')
#         print('pv_pel',pv_pel)
#         pv_capex = pv_pel*2223.625
#         #pv_opex_df = discount_df * pv_pel * 75
#         #pv_opex_final  = pv_opex_df.iloc[0].sum()
#         stes_scen_cost_df ['pv_capex'] = pv_capex
        #stes_scen_cost_df ['pv_opex'] = pv_opex_final
        
        #stes_scen_cost_df ['total_opex'] = stes_scen_cost_df['opex'] + stes_scen_cost_df['pv_opex']
#         stes_scen_cost_df ['total_capex'] = stes_scen_cost_df['capex'] + stes_scen_cost_df['pv_capex']+ stes_scen_cost_df['stes_capex']
        
        print(stes_scen_cost_df)
        
        
        
        stes_dict_key = share + '%'
        stes_scen_dict[stes_dict_key] = stes_scen_cost_df

print(stes_scen_dict)

In [ ]:
for stes_scen, stes_cost_df in stes_scen_dict.items():
    
    if stes_scen = '10_PV10'

In [ ]:
data_scen_list =['base_data_t48','stes_data_t48','base_data_t8760','stes_data_t8760','stes_1223552_100071_t8760','base_1223552_100071_t8760']

In [ ]:
data_scen_widget = widgets.Dropdown(
    options = data_scen_list,
    description = 'Scenario ',
    disabled = False)
display(data_scen_widget)

In [ ]:
data_scen = data_scen_widget.value